# APTOS 2019 Blindness Detection – Medal Plan

Goal: Win a medal via strong CV and fast CNN baseline → iterate with error-driven improvements.

Pipeline overview:
- Validation: 5-fold StratifiedKFold on diagnosis, fixed seed, save folds to disk; monitor OOF QWK.
- Metric: Optimize quadratic weighted kappa via post-training threshold search on OOF.
- Model: timm pretrained CNN (start: tf_efficientnet_b3_ns, fallback: resnet50). Head: regression (single logit) with MSE/Huber; inference uses learned thresholds.
- Image size: start 384px, then 512px if time allows. Mixed precision, GPU mandatory.
- Augmentations (Albumentations): Horizontal/vertical flip, random rotate/shift, RandomResizedCrop, brightness/contrast, CLAHE (light), blur/noise light, Cutout optional. Normalize to ImageNet.
- Preprocessing v0: simple center-crop and resize (no heavy circle crop yet). v1: fast fundus crop via threshold mask.
- Training: AdamW, cosine schedule w/ warmup, early stopping (patience 3), epochs ~10-12 per fold (stop early).
- TTA: simple (hflip) at inference after solid CV.
- Logging: print fold idx, epoch, train/valid loss, valid QWK, elapsed time. Save best per fold, OOF preds, test preds, thresholds.
- Repro: fix seeds, deterministic ops where feasible.

Iteration plan:
1) Sanity: Verify GPU (install torch), list data counts, class distribution; create folds.
2) Baseline: EfficientNet-B3 384px, regression + OOF thresholding. Quick 2-fold smoke test (2-3 epochs) to validate pipeline.
3) Full 5-fold run; cache OOF/test. Submit baseline.
4) Improvements:
   - Better preprocessing: fast circular mask crop; light CLAHE.
   - Stronger aug schedule; try BCE-with-ordinal targets vs regression.
   - Image size 512; lr/weight-decay tuning; label smoothing for CE variant.
   - TTA on LB; blend seeds/models.
5) Error analysis: OOF confusion, per-class QWK bins; adjust thresholds/augs.

Safety/efficiency:
- Subsample for smoke runs; early stop aggressively.
- While training, run expert review on strategy and CV; do not wait on unproductive runs.

Artifacts:
- folds.csv, oof.npy, oof_targets.npy, oof_logits.npy, thresholds.npy, test_preds.npy, submission.csv

Next: Get expert feedback before heavy runs; then environment check and installs.

In [ ]:
# Environment setup: install GPU PyTorch and libs; verify GPU
import sys, subprocess, time
def pip_install(args):
    print('Installing:', ' '.join(args), flush=True)
    r = subprocess.run([sys.executable, '-m', 'pip', 'install'] + args, check=False)
    print('Return code:', r.returncode, flush=True)

# Install PyTorch CUDA 12.1 first to avoid CPU wheels
pip_install(['--upgrade', 'pip'])
pip_install(['--index-url', 'https://download.pytorch.org/whl/cu121', 'torch', 'torchvision', 'torchaudio'])
# Core libs
pip_install(['timm', 'albumentations>=1.4.0', 'opencv-python-headless', 'scikit-learn', 'pandas', 'numpy', 'tqdm', 'scipy'])

import torch
print(f'GPU Available: {torch.cuda.is_available()}', flush=True)
print(f'GPU Count: {torch.cuda.device_count()}', flush=True)
if torch.cuda.is_available():
    print(f'GPU Name: {torch.cuda.get_device_name(0)}', flush=True)
    props = torch.cuda.get_device_properties(0)
    print(f'GPU Memory: {props.total_memory / 1024**3:.1f} GB', flush=True)

In [2]:
# Data load, CV folds, and fast 512px cache with circle-crop + Ben filter + CLAHE
import os, math, random, json, gc, shutil, glob, time
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

SEED = 42
random.seed(SEED); np.random.seed(SEED)

DATA_DIR = Path('.')
TRAIN_DIR = DATA_DIR / 'train_images'
TEST_DIR = DATA_DIR / 'test_images'
FOLDS_CSV = DATA_DIR / 'folds.csv'
CACHE_DIR = DATA_DIR / 'cache512'
CACHE_TRAIN = CACHE_DIR / 'train'
CACHE_TEST = CACHE_DIR / 'test'
IMG_SIZE = 512

# Read CSVs
train_df = pd.read_csv(DATA_DIR / 'train.csv')
test_df = pd.read_csv(DATA_DIR / 'test.csv')
print('Train shape:', train_df.shape, 'Test shape:', test_df.shape, flush=True)
print('Train head:\n', train_df.head(), flush=True)
print('Class distribution (train):\n', train_df['diagnosis'].value_counts().sort_index(), flush=True)

# Create 5-fold stratified splits (fixed and reusable)
if FOLDS_CSV.exists():
    folds_df = pd.read_csv(FOLDS_CSV)
    print('Loaded existing folds.csv with shape', folds_df.shape, flush=True)
else:
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    folds = np.zeros(len(train_df), dtype=np.int32)
    for fold, (_, val_idx) in enumerate(skf.split(train_df['id_code'], train_df['diagnosis'])):
        folds[val_idx] = fold
    folds_df = train_df.copy()
    folds_df['fold'] = folds
    folds_df.to_csv(FOLDS_CSV, index=False)
    print('Saved folds.csv with shape', folds_df.shape, flush=True)
print(folds_df['fold'].value_counts().sort_index(), flush=True)

# --- Fast fundus preprocessing ---
def circle_crop_ben_clahe(img_bgr: np.ndarray, size: int = 512, clahe_p: float = 0.4) -> np.ndarray:
    h, w = img_bgr.shape[:2]
    # Make a quick mask for fundus
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    # Normalize a bit to stabilize threshold
    gray_blur = cv2.GaussianBlur(gray, (0,0), 3)
    thr = cv2.threshold(gray_blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    # Invert if background is white-ish
    if np.mean(thr) > 127:
        thr = cv2.bitwise_not(thr)
    # Morph close to fill holes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
    thr = cv2.morphologyEx(thr, cv2.MORPH_CLOSE, kernel, iterations=2)
    contours, _ = cv2.findContours(thr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        cnt = max(contours, key=cv2.contourArea)
        x, y, bw, bh = cv2.boundingRect(cnt)
    else:
        # Fallback center square
        side = min(h, w)
        x = (w - side) // 2; y = (h - side) // 2; bw = side; bh = side
    crop = img_bgr[y:y+bh, x:x+bw]
    ch, cw = crop.shape[:2]
    # Pad to square
    side = max(ch, cw)
    pad_top = (side - ch) // 2; pad_bottom = side - ch - pad_top
    pad_left = (side - cw) // 2; pad_right = side - cw - pad_left
    crop_sq = cv2.copyMakeBorder(crop, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=(0,0,0))
    # Resize
    img = cv2.resize(crop_sq, (size, size), interpolation=cv2.INTER_AREA)
    # Ben Graham enhancement (unsharp-like)
    blur = cv2.GaussianBlur(img, (0,0), size/30)  # sigma proportional to size
    img = cv2.addWeighted(img, 4, blur, -4, 128)
    img = np.clip(img, 0, 255).astype(np.uint8)
    # Light CLAHE on L channel
    if random.random() < clahe_p:
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        L, A, B = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        L = clahe.apply(L)
        lab = cv2.merge([L, A, B])
        img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return img

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

ensure_dir(CACHE_TRAIN); ensure_dir(CACHE_TEST)

def cache_split(df: pd.DataFrame, src_dir: Path, dst_dir: Path, id_col: str, limit: int | None = None):
    paths = []
    for iid in df[id_col].values.tolist():
        src = src_dir / f"{iid}.png"
        dst = dst_dir / f"{iid}.png"
        paths.append((str(src), str(dst)))
    if limit is not None:
        paths = paths[:limit]
    cnt_exist = sum(os.path.exists(d) for _, d in paths)
    print(f'{dst_dir.name}: {cnt_exist}/{len(paths)} already cached', flush=True)
    to_process = [(s, d) for s, d in paths if not os.path.exists(d)]
    print(f'Processing {len(to_process)} images into {dst_dir} ...', flush=True)
    t0 = time.time()
    for i, (src, dst) in enumerate(tqdm(to_process, total=len(to_process))):
        img = cv2.imread(src, cv2.IMREAD_COLOR)
        if img is None:
            # Skip or copy as-is if missing
            continue
        img = circle_crop_ben_clahe(img, size=IMG_SIZE, clahe_p=0.4)
        cv2.imwrite(dst, img, [cv2.IMWRITE_PNG_COMPRESSION, 3])
        if (i+1) % 200 == 0:
            print(f'Cached {i+1}/{len(to_process)}; elapsed {time.time()-t0:.1f}s', flush=True)
    print(f'Done {dst_dir.name}; total elapsed {time.time()-t0:.1f}s', flush=True)

# Full cache run at 512px for all images (per expert advice)
SMOKE_LIMIT = None  # set to None for full cache run
cache_split(train_df, TRAIN_DIR, CACHE_TRAIN, id_col='id_code', limit=SMOKE_LIMIT)
cache_split(test_df, TEST_DIR, CACHE_TEST, id_col='id_code', limit=SMOKE_LIMIT)
print('Cache complete. Proceed to training.', flush=True)

Train shape: (3295, 2) Test shape: (367, 1)


Train head:
         id_code  diagnosis
0  2a2274bcb00a          0
1  eda29a9d78f3          0
2  789c60cba801          0
3  a07efb1ecfc0          0
4  44855f666225          2


Class distribution (train):
 diagnosis
0    1628
1     340
2     896
3     176
4     255
Name: count, dtype: int64


Loaded existing folds.csv with shape (3295, 3)


fold
0    659
1    659
2    659
3    659
4    659
Name: count, dtype: int64


...

Processing 0 images into cache512/test ...


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Done test; total elapsed 0.0s


Cache complete. Proceed to training.


In [3]:
# Training scaffold: Dataset, Augs, Model (EffNet-B4 @512), CV loop, QWK + threshold optimization
import math, os, time, json, gc, copy, random
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import timm
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from scipy.optimize import minimize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
IMG_DIR_TRAIN = CACHE_TRAIN
IMG_DIR_TEST = CACHE_TEST
NUM_CLASSES = 5

# Ensure writable HF cache (timm pretrained weights) to avoid read-only /app/.cache errors
HF_CACHE_DIR = Path('./hf_cache')
HF_CACHE_DIR.mkdir(parents=True, exist_ok=True)
os.environ['HF_HOME'] = str(HF_CACHE_DIR)
os.environ['HUGGINGFACE_HUB_CACHE'] = str(HF_CACHE_DIR)
os.environ['XDG_CACHE_HOME'] = str(HF_CACHE_DIR)
os.environ['HF_HUB_CACHE'] = str(HF_CACHE_DIR)
# Help CUDA allocator avoid fragmentation per PyTorch docs
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

def seed_everything(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

# Hyperparams (full run defaults)
BATCH_SIZE = 8  # start safer; OOM logic can drop to 4 or 2 with accumulation
EPOCHS_FULL = 15
LR = 2e-4
WD = 1e-5
PATIENCE = 3
NUM_WORKERS_TRAIN = 0
NUM_WORKERS_TEST = 6
BACKBONE = 'tf_efficientnet_b4_ns'

# Albumentations pipelines (conservative, per expert advice)
MEAN = (0.485, 0.456, 0.406); STD = (0.229, 0.224, 0.225)
train_tfms = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, border_mode=cv2.BORDER_REFLECT_101, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.7),
    A.RandomGamma(gamma_limit=(90, 110), p=0.4),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])
valid_tfms = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

class DRDataset(Dataset):
    def __init__(self, df: pd.DataFrame, img_dir: Path, transforms=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.transforms = transforms
        self.has_target = 'diagnosis' in df.columns
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_dir / f"{row['id_code']}.png"
        img = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
        if img is None:
            # Fallback: read from original dir if cache missing
            orig = TRAIN_DIR / f"{row['id_code']}.png"
            img = cv2.imread(str(orig), cv2.IMREAD_COLOR)
            if img is None:
                img = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Hard enforce size to avoid variable tensor sizes even if transforms fail
        if img.shape[0] != IMG_SIZE or img.shape[1] != IMG_SIZE:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
        if self.transforms:
            img = self.transforms(image=img)['image']
        if self.has_target:
            target = float(row['diagnosis'])
            return img, torch.tensor(target, dtype=torch.float32)
        else:
            return img, row['id_code']

class RegHeadModel(nn.Module):
    def __init__(self, backbone_name='tf_efficientnet_b4_ns', pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=0, global_pool='avg', cache_dir=str(HF_CACHE_DIR))
        # Try to enable gradient checkpointing to save memory
        try:
            if hasattr(self.backbone, 'set_grad_checkpointing'):
                self.backbone.set_grad_checkpointing(True)
        except Exception:
            pass
        in_ch = self.backbone.num_features
        self.head = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(in_ch, 1)
        )
    def forward(self, x):
        feats = self.backbone(x)
        out = self.head(feats).squeeze(1)
        return out

def qwk(y_true, y_pred_cls):
    return cohen_kappa_score(y_true, y_pred_cls, weights='quadratic')

def preds_to_classes(preds, thresholds):
    th0, th1, th2, th3 = thresholds
    return np.digitize(preds, bins=[th0, th1, th2, th3])

def optimize_thresholds(oof_targets, oof_preds, init=[0.5,1.5,2.5,3.5]):
    y = np.asarray(oof_targets).astype(float)
    p = np.asarray(oof_preds).astype(float)
    def _loss(th):
        th = np.sort(th)
        th = np.clip(th, -1.0, 4.0)
        cls = preds_to_classes(p, th)
        return -qwk(y, cls)
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i] - th[i-1] < 0.05:
            th[i] = th[i-1] + 0.05
    return th

def get_loaders(tr_df, va_df, batch_size=16, num_workers=0):
    dtr = DRDataset(tr_df, IMG_DIR_TRAIN, transforms=train_tfms)
    dva = DRDataset(va_df, IMG_DIR_TRAIN, transforms=valid_tfms)
    dl_tr = DataLoader(dtr, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=True, persistent_workers=False)
    dl_va = DataLoader(dva, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=False)
    return dl_tr, dl_va

def validate(model, dl, loss_fn):
    model.eval()
    preds = []; targs = []; val_loss = 0.0; n = 0
    with torch.no_grad():
        for xb, yb in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            yb = yb.to(device, non_blocking=True)
            with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16):
                out = model(xb)
                loss = loss_fn(out, yb)
            bs = xb.size(0)
            val_loss += loss.item()*bs; n += bs
            preds.append(out.detach().float().cpu().numpy())
            targs.append(yb.detach().float().cpu().numpy())
    preds = np.concatenate(preds); targs = np.concatenate(targs)
    return val_loss/n, preds, targs

def tta_predict(model, dl):
    model.eval()
    preds = []
    with torch.no_grad():
        for xb, _ in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16):
                p1 = model(xb)
                p2 = model(torch.flip(xb, dims=[-1]))
                p = (p1 + p2) / 2.0
            preds.append(p.detach().float().cpu().numpy())
    return np.concatenate(preds)

def train_one_fold(fold, folds_df, epochs=12, lr=2e-4, wd=1e-5, batch_size=16, patience=3, backbone='tf_efficientnet_b4_ns', num_workers=0):
    print(f'\n===== Fold {fold} / {folds_df["fold"].nunique()} =====', flush=True)
    if torch.cuda.is_available():
        try:
            torch.cuda.reset_peak_memory_stats()
        except Exception:
            pass
        print('Peak GB before fold:', f'{torch.cuda.max_memory_allocated()/1024**3:.2f}', flush=True)
    tr_df = folds_df[folds_df['fold'] != fold][['id_code','diagnosis']].reset_index(drop=True)
    va_df = folds_df[folds_df['fold'] == fold][['id_code','diagnosis']].reset_index(drop=True)
    dl_tr, dl_va = get_loaders(tr_df, va_df, batch_size=batch_size, num_workers=num_workers)
    # Use pretrained=True now that cache dir is writable
    model = RegHeadModel(backbone_name=backbone, pretrained=True).to(device)
    model = model.to(memory_format=torch.channels_last)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    loss_fn = nn.HuberLoss(delta=1.0)
    scaler = torch.cuda.amp.GradScaler(enabled=True)
    total_steps = epochs * len(dl_tr)
    def lr_lambda(step):
        if step < len(dl_tr):
            return (step+1)/len(dl_tr)
        progress = (step - len(dl_tr)) / max(1, total_steps - len(dl_tr))
        return 0.5 * (1 + math.cos(math.pi * progress))
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    best_qwk = -1.0; best_state = None; best_preds = None; best_targs = None; no_improve = 0
    global_step = 0; t_start = time.time()
    accum_steps = max(1, 16 // batch_size)
    optimizer.zero_grad(set_to_none=True)
    for epoch in range(1, epochs+1):
        model.train()
        tr_loss = 0.0; n = 0; t0 = time.time()
        for it, (xb, yb) in enumerate(dl_tr):
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last); yb = yb.to(device, non_blocking=True)
            with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16):
                out = model(xb)
                loss = loss_fn(out, yb) / accum_steps
            scaler.scale(loss).backward()
            if (it + 1) % accum_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)
                scheduler.step()
            bs = xb.size(0); tr_loss += (loss.item() * accum_steps)*bs; n += bs; global_step += 1
            if (it+1) % 50 == 0:
                print(f'Epoch {epoch} it {it+1}/{len(dl_tr)} tr_loss {(tr_loss/n):.4f} elapsed {time.time()-t0:.1f}s', flush=True)
        val_loss, v_preds, v_targs = validate(model, dl_va, loss_fn)
        tmp_cls = preds_to_classes(v_preds, [0.5,1.5,2.5,3.5])
        val_q = qwk(v_targs, tmp_cls)
        print(f'Epoch {epoch}: tr_loss {(tr_loss/n):.4f} val_loss {val_loss:.4f} val_qwk {val_q:.4f} epoch_time {time.time()-t0:.1f}s total_elapsed {time.time()-t_start:.1f}s', flush=True)
        if val_q > best_qwk:
            best_qwk = val_q; best_state = copy.deepcopy(model.state_dict()); best_preds = v_preds.copy(); best_targs = v_targs.copy(); no_improve = 0
        else:
            no_improve += 1
        if no_improve >= patience:
            print('Early stopping triggered', flush=True); break
    model.load_state_dict(best_state)
    return model, best_preds, best_targs

def run_cv(folds_df, backbone='tf_efficientnet_b4_ns', epochs=12, batch_size=8, num_workers=0):
    # Fix index alignment for OOF
    folds_df = folds_df.reset_index(drop=True).copy()
    n_folds = folds_df['fold'].nunique()
    oof_preds = np.zeros(len(folds_df), dtype=np.float32)
    oof_targs = folds_df['diagnosis'].values.astype(float)
    models = []
    for fold in range(n_folds):
        try:
            torch.cuda.empty_cache()
            if torch.cuda.is_available():
                torch.cuda.reset_peak_memory_stats()
        except Exception:
            pass
        gc.collect()
        current_bs = batch_size
        for attempt in range(6):
            print(f'Attempt {attempt+1}: trying batch_size={current_bs}', flush=True)
            try:
                fm, v_preds, v_targs = train_one_fold(fold, folds_df, epochs=epochs, lr=LR, wd=WD, batch_size=current_bs, patience=PATIENCE, backbone=backbone, num_workers=num_workers)
                break
            except (RuntimeError, torch.cuda.OutOfMemoryError, torch.OutOfMemoryError) as e:
                msg = str(e).lower()
                if 'out of memory' in msg or 'cuda out of memory' in msg:
                    print(f'CUDA OOM on fold {fold} with batch_size={current_bs}. Reducing and retrying (attempt {attempt+1})...', flush=True)
                    # Aggressive cleanup
                    try:
                        del fm
                    except Exception:
                        pass
                    gc.collect()
                    try:
                        torch.cuda.empty_cache()
                        if torch.cuda.is_available():
                            torch.cuda.reset_peak_memory_stats()
                    except Exception:
                        pass
                    current_bs = max(2, current_bs - 2)
                    if attempt >= 5:
                        raise
                    continue
                else:
                    raise
        models.append(fm)
        va_idx = np.where(folds_df['fold'].values == fold)[0]
        oof_preds[va_idx] = v_preds
        fold_q = qwk(v_targs, preds_to_classes(v_preds, [0.5,1.5,2.5,3.5]))
        print(f'Fold {fold} val QWK (default th): {fold_q:.4f}', flush=True)
    th = optimize_thresholds(oof_targs, oof_preds)
    oof_q = qwk(oof_targs, preds_to_classes(oof_preds, th))
    print('Optimized thresholds:', th, 'OOF QWK:', f'{oof_q:.5f}', flush=True)
    np.save('oof_preds.npy', oof_preds); np.save('oof_targets.npy', oof_targs); np.save('thresholds.npy', th)
    return models, th, oof_q

def predict_test(models, thresholds, batch_size=16):
    test_ids = test_df['id_code'].values
    dte = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms)
    dl_te = DataLoader(dte, batch_size=batch_size*2, shuffle=False, num_workers=NUM_WORKERS_TEST, pin_memory=True, persistent_workers=False)
    all_preds = []
    for m_i, m in enumerate(models):
        print(f'Inference model {m_i+1}/{len(models)}', flush=True)
        preds = tta_predict(m, dl_te)
        all_preds.append(preds)
    reg_pred = np.mean(np.stack(all_preds, axis=0), axis=0)
    cls = preds_to_classes(reg_pred, thresholds).astype(int)
    sub = pd.DataFrame({'id_code': test_ids, 'diagnosis': cls})
    sub.to_csv('submission.csv', index=False)
    np.save('test_reg_preds.npy', reg_pred)
    print('Saved submission.csv and test_reg_preds.npy')
    return sub

# --- Full CV run (smoke off) ---
SMOKE_TRAIN = False
if SMOKE_TRAIN:
    folds_use = folds_df.copy()
    folds_use = folds_use[folds_use['fold'].isin([0,1])].copy()
    models, thres, oof_q = run_cv(folds_use, backbone=BACKBONE, epochs=2, batch_size=8, num_workers=0)
    print('Smoke OOF QWK (partial):', oof_q)
else:
    # Optional one-time warmup to force weight download before CV
    try:
        _ = timm.create_model(BACKBONE, pretrained=True, num_classes=0, global_pool='avg', cache_dir=str(HF_CACHE_DIR))
        del _
    except Exception as e:
        print('Pretrained weight warmup failed (will retry inside folds):', e)
    models, thres, oof_q = run_cv(folds_df, backbone=BACKBONE, epochs=EPOCHS_FULL, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS_TRAIN)

print('Training scaffold ready.')

/app/.pip-target/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


Attempt 1: trying batch_size=8



===== Fold 0 / 5 =====


Peak GB before fold: 0.00


/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


...

Epoch 10: tr_loss 0.1005 val_loss 0.1715 val_qwk 0.8423 epoch_time 140.6s total_elapsed 1392.7s


Early stopping triggered


Fold 4 val QWK (default th): 0.8745


Optimized thresholds: [0.51432359 1.49038092 2.48714289 3.27959726] OOF QWK: 0.86938


Training scaffold ready.


In [4]:
# Inference and submission after CV completes
print('OOF QWK (from CV):', oof_q if 'oof_q' in globals() else 'N/A', flush=True)
sub = predict_test(models, thres, batch_size=BATCH_SIZE)
print('Submission head:\n', sub.head(), flush=True)
import os
print('submission.csv exists:', os.path.exists('submission.csv'), flush=True)

OOF QWK (from CV): 0.8693825133407611


Inference model 1/5


/tmp/ipykernel_680/1833179421.py:169: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16):


Inference model 2/5


Inference model 3/5


Inference model 4/5


Inference model 5/5


Saved submission.csv and test_reg_preds.npy
Submission head:
         id_code  diagnosis
0  b460ca9fa26f          0
1  6cee2e148520          0
2  ca6842bfcbc9          1
3  6cbc3dad809c          2
4  a9bc2f892cb3          0


submission.csv exists: True


In [5]:
# Train EfficientNet-B5 @512, then blend with B4; re-optimize thresholds on blended OOF; create blended submission
import shutil, os
print('Preparing for B5 run + blend with B4', flush=True)

# Rename current B4 artifacts to keep them
if os.path.exists('oof_preds.npy'):
    shutil.move('oof_preds.npy', 'oof_preds_b4.npy')
if os.path.exists('oof_targets.npy') and not os.path.exists('oof_targets_b4.npy'):
    shutil.copy('oof_targets.npy', 'oof_targets_b4.npy')  # targets identical
if os.path.exists('thresholds.npy'):
    shutil.move('thresholds.npy', 'thresholds_b4.npy')
if os.path.exists('test_reg_preds.npy'):
    shutil.move('test_reg_preds.npy', 'test_reg_preds_b4.npy')

# Train B5 with safer batch size
BACKBONE_B5 = 'tf_efficientnet_b5_ns'
BATCH_SIZE_B5 = 8  # OOM logic will reduce if needed
print('Starting B5 training...', flush=True)
models_b5, thres_b5, oof_q_b5 = run_cv(folds_df, backbone=BACKBONE_B5, epochs=EPOCHS_FULL, batch_size=BATCH_SIZE_B5, num_workers=NUM_WORKERS_TRAIN)
print('B5 OOF QWK:', oof_q_b5, flush=True)

# Inference for B5 to cache its test preds
print('Running B5 inference...', flush=True)
sub_b5 = predict_test(models_b5, thres_b5, batch_size=BATCH_SIZE_B5)

# Load OOFs and blend (equal weights). Re-optimize thresholds on blended OOF.
oof_b4 = np.load('oof_preds_b4.npy')
oof_b5 = np.load('oof_preds.npy')  # latest from B5 run
oof_t = np.load('oof_targets_b4.npy')
oof_blend = 0.5 * oof_b4 + 0.5 * oof_b5
th_blend = optimize_thresholds(oof_t, oof_blend)
oof_q_blend = qwk(oof_t, preds_to_classes(oof_blend, th_blend))
print('Blended OOF QWK:', f'{oof_q_blend:.5f}', 'Thresholds:', th_blend, flush=True)
np.save('thresholds_blend.npy', th_blend)

# Blend test predictions and create submission
preds_b4 = np.load('test_reg_preds_b4.npy')
preds_b5 = np.load('test_reg_preds.npy')
preds_blend = 0.5 * preds_b4 + 0.5 * preds_b5
cls_blend = preds_to_classes(preds_blend, th_blend).astype(int)
sub_blend = pd.DataFrame({'id_code': test_df['id_code'].values, 'diagnosis': cls_blend})
sub_blend.to_csv('submission.csv', index=False)
np.save('test_reg_preds_blend.npy', preds_blend)
print('Saved blended submission.csv and test_reg_preds_blend.npy', flush=True)
print('Head of blended submission:\n', sub_blend.head(), flush=True)

Preparing for B5 run + blend with B4


Starting B5 training...


Attempt 1: trying batch_size=8



===== Fold 0 / 5 =====


Peak GB before fold: 0.67


...

Inference model 5/5


Saved submission.csv and test_reg_preds.npy


Blended OOF QWK: 0.87970 Thresholds: [0.56248219 1.49132383 2.58129375 3.13354049]


Saved blended submission.csv and test_reg_preds_blend.npy


Head of blended submission:
         id_code  diagnosis
0  b460ca9fa26f          0
1  6cee2e148520          0
2  ca6842bfcbc9          1
3  6cbc3dad809c          2
4  a9bc2f892cb3          0


In [6]:
# Stronger TTA inference for both B4 and B5, then blend with thresholds_blend.npy
import numpy as np, pandas as pd, torch
from torch.utils.data import DataLoader

def multi_tta_predict(models, dl):
    preds_accum = []
    for m_i, model in enumerate(models):
        model.eval()
        preds_model = []
        with torch.no_grad():
            for xb, _ in dl:
                xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
                with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16):
                    p0 = model(xb)
                    p1 = model(torch.flip(xb, dims=[-1]))  # hflip
                    p2 = model(torch.flip(xb, dims=[-2]))  # vflip
                    x90 = xb.transpose(-1, -2)  # rot90 via transpose + hflip
                    p3 = model(x90)
                    x270 = torch.flip(x90, dims=[-1])
                    p4 = model(x270)
                    p = (p0 + p1 + p2 + p3 + p4) / 5.0
                preds_model.append(p.detach().float().cpu().numpy())
        preds_model = np.concatenate(preds_model)
        preds_accum.append(preds_model)
        print(f'TTA inference done for model {m_i+1}/{len(models)}', flush=True)
    return np.mean(np.stack(preds_accum, axis=0), axis=0)

print('Running stronger TTA + blend...', flush=True)
dte = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms)
dl_te = DataLoader(dte, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=NUM_WORKERS_TEST, pin_memory=True, persistent_workers=False)

# B4 TTA
preds_b4_tta = multi_tta_predict(models, dl_te)
np.save('test_reg_preds_b4_tta.npy', preds_b4_tta)

# B5 TTA
preds_b5_tta = multi_tta_predict(models_b5, dl_te)
np.save('test_reg_preds_b5_tta.npy', preds_b5_tta)

# Blend and threshold (use thresholds_blend optimized on OOF blend)
th_blend = np.load('thresholds_blend.npy')
preds_blend_tta = 0.5 * preds_b4_tta + 0.5 * preds_b5_tta
cls_blend_tta = preds_to_classes(preds_blend_tta, th_blend).astype(int)
sub_blend_tta = pd.DataFrame({'id_code': test_df['id_code'].values, 'diagnosis': cls_blend_tta})
sub_blend_tta.to_csv('submission.csv', index=False)
np.save('test_reg_preds_blend_tta.npy', preds_blend_tta)
print('Saved TTA-blended submission.csv and test_reg_preds_blend_tta.npy', flush=True)
print('Head of TTA-blended submission:\n', sub_blend_tta.head(), flush=True)

Running stronger TTA + blend...


/tmp/ipykernel_680/724826451.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16):


TTA inference done for model 1/5


TTA inference done for model 2/5


TTA inference done for model 3/5


...

TTA inference done for model 3/5


TTA inference done for model 4/5


TTA inference done for model 5/5


Saved TTA-blended submission.csv and test_reg_preds_blend_tta.npy


Head of TTA-blended submission:
         id_code  diagnosis
0  b460ca9fa26f          0
1  6cee2e148520          0
2  ca6842bfcbc9          1
3  6cbc3dad809c          2
4  a9bc2f892cb3          0


In [11]:
# RRC + EMA strengthened recipe and new B5@512 seed run
import math, time, copy, gc, os
import numpy as np
import pandas as pd
import albumentations as A
import cv2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from timm.utils import ModelEmaV2
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import minimize
from albumentations.pytorch import ToTensorV2

print('Setting up strengthened recipe: RRC + EMA for B5@512', flush=True)

# Augmentations per expert advice (fix params for current Albumentations)
MEAN = (0.485, 0.456, 0.406); STD = (0.229, 0.224, 0.225)
train_tfms_rrc = A.Compose([
    A.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.85, 1.0), ratio=(0.95, 1.05), interpolation=cv2.INTER_AREA),
    A.HorizontalFlip(p=0.5),
    A.Affine(scale=(0.95, 1.05), translate_percent=(0.0, 0.06), rotate=(-15, 15), border_mode=cv2.BORDER_REFLECT_101, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.7),
    A.RandomGamma(gamma_limit=(90,110), p=0.4),
    A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=8, val_shift_limit=8, p=0.3),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])
valid_tfms_rrc = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

def preds_to_classes_local(preds, thresholds):
    th0, th1, th2, th3 = thresholds
    return np.digitize(preds, bins=[th0, th1, th2, th3])

def qwk_local(y_true, y_pred_cls):
    return cohen_kappa_score(y_true, y_pred_cls, weights='quadratic')

def optimize_thresholds_local(oof_targets, oof_preds, init=[0.5,1.5,2.5,3.5]):
    y = np.asarray(oof_targets).astype(float)
    p = np.asarray(oof_preds).astype(float)
    def _loss(th):
        th = np.sort(th)
        th = np.clip(th, -1.0, 4.0)
        cls = preds_to_classes_local(p, th)
        return -qwk_local(y, cls)
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i] - th[i-1] < 0.05:
            th[i] = th[i-1] + 0.05
    return th

def get_loaders_rrc(tr_df, va_df, batch_size=16, num_workers=0):
    dtr = DRDataset(tr_df, IMG_DIR_TRAIN, transforms=train_tfms_rrc)
    dva = DRDataset(va_df, IMG_DIR_TRAIN, transforms=valid_tfms_rrc)
    dl_tr = DataLoader(dtr, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=True, persistent_workers=False)
    dl_va = DataLoader(dva, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=False)
    return dl_tr, dl_va

def validate_with_model(model, dl, loss_fn):
    model.eval()
    preds = []; targs = []; val_loss = 0.0; n = 0
    with torch.no_grad():
        for xb, yb in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            yb = yb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                out = model(xb)
                loss = loss_fn(out, yb)
            bs = xb.size(0)
            val_loss += loss.item()*bs; n += bs
            preds.append(out.detach().float().cpu().numpy())
            targs.append(yb.detach().float().cpu().numpy())
    preds = np.concatenate(preds); targs = np.concatenate(targs)
    return val_loss/n, preds, targs

def train_one_fold_ema(fold, folds_df, epochs=12, lr=2e-4, wd=1e-5, batch_size=16, patience=3, backbone='tf_efficientnet_b5_ns', num_workers=0, ema_decay=0.9996):
    print(f'\n===== [RRC+EMA] Fold {fold} / {folds_df["fold"].nunique()} =====', flush=True)
    if torch.cuda.is_available():
        try: torch.cuda.reset_peak_memory_stats()
        except Exception: pass
        print('Peak GB before fold:', f'{torch.cuda.max_memory_allocated()/1024**3:.2f}', flush=True)
    tr_df = folds_df[folds_df['fold'] != fold][['id_code','diagnosis']].reset_index(drop=True)
    va_df = folds_df[folds_df['fold'] == fold][['id_code','diagnosis']].reset_index(drop=True)
    dl_tr, dl_va = get_loaders_rrc(tr_df, va_df, batch_size=batch_size, num_workers=num_workers)
    model = RegHeadModel(backbone_name=backbone, pretrained=True).to(device)
    model = model.to(memory_format=torch.channels_last)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    loss_fn = nn.HuberLoss(delta=1.0)
    scaler = torch.amp.GradScaler('cuda', enabled=True)
    ema = ModelEmaV2(model, decay=ema_decay, device=None)
    total_steps = epochs * len(dl_tr)
    def lr_lambda(step):
        if step < len(dl_tr):
            return (step+1)/len(dl_tr)
        progress = (step - len(dl_tr)) / max(1, total_steps - len(dl_tr))
        return 0.5 * (1 + math.cos(math.pi * progress))
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    best_metric = float('inf'); best_state = None; best_preds = None; best_targs = None; no_improve = 0
    global_step = 0; t_start = time.time()
    accum_steps = max(1, 16 // batch_size)
    optimizer.zero_grad(set_to_none=True)
    for epoch in range(1, epochs+1):
        model.train()
        tr_loss = 0.0; n = 0; t0 = time.time()
        for it, (xb, yb) in enumerate(dl_tr):
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last); yb = yb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                out = model(xb)
                loss = loss_fn(out, yb) / accum_steps
            scaler.scale(loss).backward()
            if (it + 1) % accum_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)
                scheduler.step()
                ema.update(model)
            bs = xb.size(0); tr_loss += (loss.item() * accum_steps)*bs; n += bs; global_step += 1
            if (it+1) % 50 == 0:
                print(f'Epoch {epoch} it {it+1}/{len(dl_tr)} tr_loss {(tr_loss/n):.4f} elapsed {time.time()-t0:.1f}s', flush=True)
        # Validate with current (non-EMA) model for stability; still save EMA weights
        val_loss_m, v_preds_m, v_targs = validate_with_model(model, dl_va, loss_fn)
        cls_def = preds_to_classes_local(v_preds_m, [0.5,1.5,2.5,3.5])
        val_q_def = qwk_local(v_targs, cls_def)
        print(f'Epoch {epoch}: tr_loss {(tr_loss/n):.4f} val_loss {val_loss_m:.4f} val_qwk(def) {val_q_def:.4f} preds[min/mean/max]={v_preds_m.min():.3f}/{v_preds_m.mean():.3f}/{v_preds_m.max():.3f} epoch_time {time.time()-t0:.1f}s total_elapsed {time.time()-t_start:.1f}s', flush=True)
        # Early stopping on val_loss
        if val_loss_m < best_metric:
            best_metric = val_loss_m; best_state = copy.deepcopy(ema.module.state_dict()); best_preds = v_preds_m.copy(); best_targs = v_targs.copy(); no_improve = 0
        else:
            no_improve += 1
        if no_improve >= patience:
            print('Early stopping triggered', flush=True); break
    # Load best EMA weights and return EMA model
    ema.module.load_state_dict(best_state)
    return ema.module, best_preds, best_targs

def run_cv_ema(folds_df, backbone='tf_efficientnet_b5_ns', epochs=15, batch_size=8, num_workers=0):
    folds_df = folds_df.reset_index(drop=True).copy()
    n_folds = folds_df['fold'].nunique()
    oof_preds = np.zeros(len(folds_df), dtype=np.float32)
    oof_targs = folds_df['diagnosis'].values.astype(float)
    models_local = []
    for fold in range(n_folds):
        try:
            torch.cuda.empty_cache()
            if torch.cuda.is_available():
                torch.cuda.reset_peak_memory_stats()
        except Exception:
            pass
        gc.collect()
        current_bs = batch_size
        for attempt in range(6):
            print(f'Attempt {attempt+1}: trying batch_size={current_bs}', flush=True)
            try:
                fm, v_preds, v_targs = train_one_fold_ema(fold, folds_df, epochs=epochs, lr=LR, wd=WD, batch_size=current_bs, patience=PATIENCE, backbone=backbone, num_workers=num_workers)
                break
            except (RuntimeError, torch.cuda.OutOfMemoryError, torch.OutOfMemoryError) as e:
                msg = str(e).lower()
                if 'out of memory' in msg or 'cuda out of memory' in msg:
                    print(f'CUDA OOM on fold {fold} with batch_size={current_bs}. Reducing and retrying (attempt {attempt+1})...', flush=True)
                    try: del fm
                    except Exception: pass
                    gc.collect()
                    try:
                        torch.cuda.empty_cache()
                        if torch.cuda.is_available():
                            torch.cuda.reset_peak_memory_stats()
                    except Exception:
                        pass
                    current_bs = max(2, current_bs - 2)
                    if attempt >= 5:
                        raise
                    continue
                else:
                    raise
        models_local.append(fm)
        va_idx = np.where(folds_df['fold'].values == fold)[0]
        oof_preds[va_idx] = v_preds
        fold_q = qwk_local(v_targs, preds_to_classes_local(v_preds, [0.5,1.5,2.5,3.5]))
        print(f'[RRC+EMA] Fold {fold} val QWK (default th): {fold_q:.4f}', flush=True)
    th = optimize_thresholds_local(oof_targs, oof_preds)
    oof_q = qwk_local(oof_targs, preds_to_classes_local(oof_preds, th))
    print('[RRC+EMA] Optimized thresholds:', th, 'OOF QWK:', f'{oof_q:.5f}', flush=True)
    np.save('oof_preds_b5_seed2025_rrc_ema.npy', oof_preds); np.save('oof_targets_b5_seed2025_rrc_ema.npy', oof_targs); np.save('thresholds_b5_seed2025_rrc_ema.npy', th)
    return models_local, th, oof_q

def bootstrap_thresholds(oof_targets, oof_preds, n_bootstrap=100, base=[0.5,1.5,2.5,3.5]):
    N = len(oof_targets)
    cuts = []
    for i in range(n_bootstrap):
        idx = np.random.randint(0, N, size=N)
        th = optimize_thresholds_local(oof_targets[idx], oof_preds[idx], init=base)
        cuts.append(th)
        if (i+1) % 20 == 0:
            print(f'Bootstrap {i+1}/{n_bootstrap}', flush=True)
    cuts = np.array(cuts)
    med = np.median(cuts, axis=0)
    med = np.clip(med, 0.3, 3.7)
    for j in range(1,4):
        if med[j] - med[j-1] < 0.1:
            med[j] = med[j-1] + 0.1
    base_arr = np.array([0.5,1.5,2.5,3.5])
    med = 0.9*med + 0.1*base_arr
    return med

# Launch B5@512 with new seed and RRC+EMA
SEED_NEW = 2025
seed_everything(SEED_NEW)
BACKBONE_NEW = 'tf_efficientnet_b5_ns'
EPOCHS_NEW = 15
BATCH_SIZE_NEW = 8
print('Starting RRC+EMA B5@512 training, seed', SEED_NEW, flush=True)
models_b5_seed2025_ema, th_b5_seed2025_ema, oof_q_b5_seed2025_ema = run_cv_ema(folds_df, backbone=BACKBONE_NEW, epochs=EPOCHS_NEW, batch_size=BATCH_SIZE_NEW, num_workers=NUM_WORKERS_TRAIN)
print('B5@512 RRC+EMA OOF QWK:', oof_q_b5_seed2025_ema, flush=True)

Setting up strengthened recipe: RRC + EMA for B5@512


Starting RRC+EMA B5@512 training, seed 2025


Attempt 1: trying batch_size=8



===== [RRC+EMA] Fold 0 / 5 =====


Peak GB before fold: 2.61


...

Epoch 12: tr_loss 0.0582 val_loss 0.1662 val_qwk(def) 0.8505 preds[min/mean/max]=-0.090/1.034/4.094 epoch_time 190.9s total_elapsed 2295.9s


Early stopping triggered


[RRC+EMA] Fold 4 val QWK (default th): 0.8712


[RRC+EMA] Optimized thresholds: [0.57085114 1.37378998 2.37971497 3.43718414] OOF QWK: 0.87391


B5@512 RRC+EMA OOF QWK: 0.8739131318130468


In [14]:
# Prep: 640px cache, B4@640 RRC+EMA 3-fold run, and NNLS-weighted blend + bootstrapped thresholds
import os, time, gc, copy, math
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from timm.utils import ModelEmaV2
from scipy.optimize import nnls, minimize
from sklearn.metrics import cohen_kappa_score

print('Preparing 640px pipeline (cache + B4 3-fold + blend)', flush=True)

# 640 cache setup
CACHE640_DIR = Path('cache640'); CACHE640_TRAIN = CACHE640_DIR / 'train'; CACHE640_TEST = CACHE640_DIR / 'test'
CACHE640_DIR.mkdir(parents=True, exist_ok=True); CACHE640_TRAIN.mkdir(parents=True, exist_ok=True); CACHE640_TEST.mkdir(parents=True, exist_ok=True)
IMG_SIZE_640 = 640

def circle_crop_ben_clahe_sz(img_bgr: np.ndarray, size: int = 512, clahe_p: float = 0.4) -> np.ndarray:
    h, w = img_bgr.shape[:2]
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.GaussianBlur(gray, (0,0), 3)
    thr = cv2.threshold(gray_blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    if np.mean(thr) > 127: thr = cv2.bitwise_not(thr)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
    thr = cv2.morphologyEx(thr, cv2.MORPH_CLOSE, kernel, iterations=2)
    contours, _ = cv2.findContours(thr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        cnt = max(contours, key=cv2.contourArea)
        x, y, bw, bh = cv2.boundingRect(cnt)
    else:
        side = min(h, w); x = (w - side) // 2; y = (h - side) // 2; bw = side; bh = side
    crop = img_bgr[y:y+bh, x:x+bw]
    ch, cw = crop.shape[:2]; side = max(ch, cw)
    pad_top = (side - ch) // 2; pad_bottom = side - ch - pad_top
    pad_left = (side - cw) // 2; pad_right = side - cw - pad_left
    crop_sq = cv2.copyMakeBorder(crop, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=(0,0,0))
    img = cv2.resize(crop_sq, (size, size), interpolation=cv2.INTER_AREA)
    blur = cv2.GaussianBlur(img, (0,0), size/30)
    img = cv2.addWeighted(img, 4, blur, -4, 128)
    img = np.clip(img, 0, 255).astype(np.uint8)
    if np.random.rand() < 0.4:
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        L, A_, B_ = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        L = clahe.apply(L)
        lab = cv2.merge([L, A_, B_])
        img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return img

def cache_split_sz(df: pd.DataFrame, src_dir: Path, dst_dir: Path, id_col: str, size: int, limit: int | None = None):
    paths = [(str(src_dir / f"{iid}.png"), str(dst_dir / f"{iid}.png")) for iid in df[id_col].values.tolist()]
    if limit is not None: paths = paths[:limit]
    cnt_exist = sum(os.path.exists(d) for _, d in paths)
    print(f'{dst_dir.name}: {cnt_exist}/{len(paths)} already cached at {size}px', flush=True)
    to_process = [(s, d) for s, d in paths if not os.path.exists(d)]
    t0 = time.time()
    for i, (src, dst) in enumerate(to_process):
        img = cv2.imread(src, cv2.IMREAD_COLOR)
        if img is None: continue
        img = circle_crop_ben_clahe_sz(img, size=size, clahe_p=0.4)
        cv2.imwrite(dst, img, [cv2.IMWRITE_PNG_COMPRESSION, 3])
        if (i+1) % 200 == 0:
            print(f'Cached {i+1}/{len(to_process)}; elapsed {time.time()-t0:.1f}s', flush=True)
    print(f'Done {dst_dir.name}; total elapsed {time.time()-t0:.1f}s', flush=True)

# Local dataset for arbitrary size/dir
class LocalDRDataset(Dataset):
    def __init__(self, df: pd.DataFrame, img_dir: Path, img_size: int, transforms=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.img_size = img_size
        self.transforms = transforms
        self.has_target = 'diagnosis' in df.columns
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_dir / f"{row['id_code']}.png"
        img = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
        if img is None:
            img = np.zeros((self.img_size, self.img_size, 3), dtype=np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if img.shape[0] != self.img_size or img.shape[1] != self.img_size:
            img = cv2.resize(img, (self.img_size, self.img_size), interpolation=cv2.INTER_AREA)
        if self.transforms:
            img = self.transforms(image=img)['image']
        if self.has_target:
            return img, torch.tensor(float(row['diagnosis']), dtype=torch.float32)
        else:
            return img, row['id_code']

# Transforms for 640 with RRC + light augs (tightened per expert advice)
MEAN = (0.485, 0.456, 0.406); STD = (0.229, 0.224, 0.225)
train_tfms_640 = A.Compose([
    A.RandomResizedCrop(size=(IMG_SIZE_640, IMG_SIZE_640), scale=(0.9, 1.0), ratio=(0.98, 1.02), interpolation=cv2.INTER_AREA),
    A.HorizontalFlip(p=0.5),
    A.Affine(scale=(0.95, 1.05), translate_percent=(0.0, 0.04), rotate=(-10, 10), border_mode=cv2.BORDER_REFLECT_101, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.7),
    A.RandomGamma(gamma_limit=(90,110), p=0.4),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])
valid_tfms_640 = A.Compose([
    A.Resize(IMG_SIZE_640, IMG_SIZE_640),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

def preds_to_classes_blend(preds, thresholds):
    th0, th1, th2, th3 = thresholds
    return np.digitize(preds, bins=[th0, th1, th2, th3])

def qwk_func(y_true, y_pred_cls):
    return cohen_kappa_score(y_true, y_pred_cls, weights='quadratic')

def optimize_thresholds_generic(y, p, init=[0.5,1.5,2.5,3.5]):
    y = np.asarray(y).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, -1.0, 4.0)
        cls = preds_to_classes_blend(p, th)
        return -qwk_func(y, cls)
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i] - th[i-1] < 0.05: th[i] = th[i-1] + 0.05
    return th

def bootstrap_thresholds_generic(y, p, n_bootstrap=100, base=[0.5,1.5,2.5,3.5]):
    N = len(y); cuts = []
    for i in range(n_bootstrap):
        idx = np.random.randint(0, N, size=N)
        th = optimize_thresholds_generic(y[idx], p[idx], init=base)
        cuts.append(th)
        if (i+1) % 20 == 0: print(f'Bootstrap {i+1}/{n_bootstrap}', flush=True)
    cuts = np.array(cuts); med = np.median(cuts, axis=0); med = np.clip(med, 0.3, 3.7)
    for j in range(1,4):
        if med[j] - med[j-1] < 0.1: med[j] = med[j-1] + 0.1
    base_arr = np.array([0.5,1.5,2.5,3.5])
    return 0.9*med + 0.1*base_arr

def get_loaders_640(tr_df, va_df, batch_size=8, num_workers=0):
    dtr = LocalDRDataset(tr_df, CACHE640_TRAIN, IMG_SIZE_640, transforms=train_tfms_640)
    dva = LocalDRDataset(va_df, CACHE640_TRAIN, IMG_SIZE_640, transforms=valid_tfms_640)
    dl_tr = DataLoader(dtr, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=True, persistent_workers=False)
    dl_va = DataLoader(dva, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=False)
    return dl_tr, dl_va

def validate_model_generic(model, dl, loss_fn):
    model.eval(); preds = []; targs = []; val_loss = 0.0; n = 0
    with torch.no_grad():
        for xb, yb in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last); yb = yb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                out = model(xb); loss = loss_fn(out, yb)
            bs = xb.size(0); val_loss += loss.item()*bs; n += bs
            preds.append(out.detach().float().cpu().numpy()); targs.append(yb.detach().float().cpu().numpy())
    preds = np.concatenate(preds); targs = np.concatenate(targs)
    return val_loss/n, preds, targs

def train_one_fold_b4_640_rrc_ema(fold, folds_df, epochs=15, lr=2e-4, wd=1e-5, batch_size=4, patience=3, ema_decay=0.9996):
    print(f'\n===== [B4@640 RRC+EMA] Fold {fold} =====', flush=True)
    tr_df = folds_df[folds_df['fold'] != fold][['id_code','diagnosis']].reset_index(drop=True)
    va_df = folds_df[folds_df['fold'] == fold][['id_code','diagnosis']].reset_index(drop=True)
    dl_tr, dl_va = get_loaders_640(tr_df, va_df, batch_size=batch_size, num_workers=0)
    model = RegHeadModel(backbone_name='tf_efficientnet_b4_ns', pretrained=True).to(device)
    model = model.to(memory_format=torch.channels_last)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    loss_fn = nn.HuberLoss(delta=1.0)
    scaler = torch.amp.GradScaler('cuda', enabled=True)
    ema = ModelEmaV2(model, decay=ema_decay, device=None)
    total_steps = epochs * len(dl_tr)
    def lr_lambda(step):
        if step < len(dl_tr): return (step+1)/len(dl_tr)
        progress = (step - len(dl_tr)) / max(1, total_steps - len(dl_tr))
        return 0.5 * (1 + math.cos(math.pi * progress))
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
    best_loss = float('inf'); best_state = None; best_preds = None; best_targs = None; no_improve = 0
    accum_steps = max(1, 16 // batch_size); t_start = time.time()
    optimizer.zero_grad(set_to_none=True)
    for epoch in range(1, epochs+1):
        model.train(); tr_loss = 0.0; n = 0; t0 = time.time()
        for it, (xb, yb) in enumerate(dl_tr):
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last); yb = yb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                out = model(xb); loss = loss_fn(out, yb) / accum_steps
            scaler.scale(loss).backward()
            if (it + 1) % accum_steps == 0:
                scaler.step(optimizer); scaler.update(); optimizer.zero_grad(set_to_none=True); scheduler.step(); ema.update(model)
            bs = xb.size(0); tr_loss += (loss.item()*accum_steps)*bs; n += bs
            if (it+1) % 50 == 0:
                print(f'Epoch {epoch} it {it+1}/{len(dl_tr)} tr_loss {(tr_loss/n):.4f} elapsed {time.time()-t0:.1f}s', flush=True)
        vloss, vpreds, vtargs = validate_model_generic(model, dl_va, loss_fn)
        vq = qwk_func(vtargs, preds_to_classes_blend(vpreds, [0.5,1.5,2.5,3.5]))
        print(f'Epoch {epoch}: tr_loss {(tr_loss/n):.4f} val_loss {vloss:.4f} val_qwk(def) {vq:.4f} epoch_time {time.time()-t0:.1f}s total_elapsed {time.time()-t_start:.1f}s', flush=True)
        if vloss < best_loss:
            best_loss = vloss; best_state = copy.deepcopy(ema.module.state_dict()); best_preds = vpreds.copy(); best_targs = vtargs.copy(); no_improve = 0
        else:
            no_improve += 1
        if no_improve >= patience:
            print('Early stopping triggered', flush=True); break
    ema.module.load_state_dict(best_state)
    return ema.module, best_preds, best_targs

def run_cv_b4_640_rrc_ema(folds_df, epochs=15, batch_size=4, folds_to_run=(0,1,2)):
    folds_df = folds_df.reset_index(drop=True).copy()
    n = len(folds_df); oof_preds = np.full(n, np.nan, dtype=np.float32); oof_targs = folds_df['diagnosis'].values.astype(float)
    models_out = [];
    for fold in folds_to_run:
        try:
            torch.cuda.empty_cache();
            if torch.cuda.is_available(): torch.cuda.reset_peak_memory_stats()
        except Exception: pass
        gc.collect()
        current_bs = batch_size
        for attempt in range(6):
            print(f'Attempt {attempt+1}: trying batch_size={current_bs}', flush=True)
            try:
                fm, v_preds, v_targs = train_one_fold_b4_640_rrc_ema(fold, folds_df, epochs=epochs, lr=LR, wd=WD, batch_size=current_bs, patience=PATIENCE, ema_decay=0.9996)
                break
            except (RuntimeError, torch.cuda.OutOfMemoryError, torch.OutOfMemoryError) as e:
                if 'out of memory' in str(e).lower():
                    print(f'CUDA OOM on fold {fold} with batch_size={current_bs}. Reducing and retrying...', flush=True)
                    try: del fm
                    except Exception: pass
                    gc.collect()
                    try: torch.cuda.empty_cache(); torch.cuda.reset_peak_memory_stats()
                    except Exception: pass
                    current_bs = max(2, current_bs - 2);
                    if attempt >= 5: raise
                    continue
                else:
                    raise
        models_out.append(fm)
        va_idx = np.where(folds_df['fold'].values == fold)[0]
        oof_preds[va_idx] = v_preds
        fold_q = qwk_func(v_targs, preds_to_classes_blend(v_preds, [0.5,1.5,2.5,3.5]))
        print(f'[B4@640] Fold {fold} val QWK (default th): {fold_q:.4f}', flush=True)
    # Save partial OOF (NaNs where not run)
    np.save('oof_preds_b4_640_rrc_ema.npy', oof_preds); np.save('oof_targets_b4_640_rrc_ema.npy', oof_targs)
    return models_out, oof_preds, oof_targs

def nnls_blend_weights(y, preds_list, mask=None):
    y = np.asarray(y).astype(float)
    mats = []
    for p in preds_list:
        mats.append(np.asarray(p).reshape(-1,1))
    X = np.concatenate(mats, axis=1)
    if mask is not None:
        X = X[mask]; y = y[mask]
    w, _ = nnls(X, y)
    if w.sum() > 0:
        w = w / w.sum()
    else:
        w = np.ones_like(w) / len(w)
    # Cap and floor weights per expert advice to avoid dominance / collapse
    w = np.clip(w, 0.05, 0.8)
    w = w / w.sum()
    return w

def prepare_and_run_b4_640_and_blend():
    # 1) Cache 640 if needed
    cache_split_sz(train_df, TRAIN_DIR, CACHE640_TRAIN, id_col='id_code', size=IMG_SIZE_640, limit=None)
    cache_split_sz(test_df, TEST_DIR, CACHE640_TEST, id_col='id_code', size=IMG_SIZE_640, limit=None)
    print('640 cache ready.', flush=True)
    # 2) Train B4@640 for 3 folds
    models_b4_640, oof_b4_640, oof_t = run_cv_b4_640_rrc_ema(folds_df, epochs=15, batch_size=4, folds_to_run=(0,1,2))
    # 3) Load existing OOFs
    paths = {
        'b4_512': 'oof_preds_b4.npy',
        'b5_512': 'oof_preds.npy',
        'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
        'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    }
    loaded = {};
    for k, p in paths.items():
        if os.path.exists(p): loaded[k] = np.load(p)
    # Align mask where all selected models have non-nan preds
    keys = list(loaded.keys())
    arrs = [loaded[k] for k in keys]
    mask = np.ones_like(oof_t, dtype=bool)
    for a in arrs:
        mask &= np.isfinite(a)
    # 4) Fit NNLS blend weights on intersection
    weights = nnls_blend_weights(oof_t, arrs, mask=mask)
    print('NNLS blend weights (', keys, '):', weights, flush=True)
    # 5) Create blended OOF and optimize + bootstrap thresholds
    blend_oof = np.zeros_like(oof_t, dtype=float)
    for w, a in zip(weights, arrs):
        a_fill = np.where(np.isfinite(a), a, 0.0)
        blend_oof += w * a_fill
    th_opt = optimize_thresholds_generic(oof_t[mask], blend_oof[mask])
    oof_q_opt = qwk_func(oof_t[mask], preds_to_classes_blend(blend_oof[mask], th_opt))
    print('Blend OOF QWK (opt on mask):', f'{oof_q_opt:.5f}', 'th:', th_opt, flush=True)
    th_boot = bootstrap_thresholds_generic(oof_t[mask], blend_oof[mask], n_bootstrap=100, base=[0.5,1.5,2.5,3.5])
    print('Bootstrapped thresholds:', th_boot, flush=True)
    np.save('blend_weights_keys.npy', np.array(keys, dtype=object)); np.save('blend_weights_vals.npy', weights); np.save('thresholds_blend_boot.npy', th_boot)
    # 6) TTA inference for B4@640 (simple hflip) and final blended submission
    dte640 = LocalDRDataset(test_df[['id_code']].copy(), CACHE640_TEST, IMG_SIZE_640, transforms=valid_tfms_640)
    dl_te640 = DataLoader(dte640, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
    def tta_predict_simple(models, dl):
        preds = [];
        with torch.no_grad():
            for xb, _ in dl:
                xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
                with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                    p0 = 0; cnt = 0
                    for m in models:
                        m.eval()
                        p0m = m(xb); p1m = m(torch.flip(xb, dims=[-1]))
                        p0 += (p0m + p1m) / 2.0; cnt += 1
                    p = p0 / max(1, cnt)
                preds.append(p.detach().float().cpu().numpy())
        return np.concatenate(preds)
    preds_b4_640 = tta_predict_simple(models_b4_640, dl_te640)
    np.save('test_reg_preds_b4_640_rrc_ema.npy', preds_b4_640)
    # Load existing test preds
    test_paths = {
        'b4_512': 'test_reg_preds_b4_tta.npy' if os.path.exists('test_reg_preds_b4_tta.npy') else 'test_reg_preds_b4.npy',
        'b5_512': 'test_reg_preds_b5_tta.npy' if os.path.exists('test_reg_preds_b5_tta.npy') else 'test_reg_preds.npy',
    }
    test_loaded = []
    for k in keys:
        if k == 'b4_640':
            test_loaded.append(preds_b4_640)
        elif k in test_paths and os.path.exists(test_paths[k]):
            test_loaded.append(np.load(test_paths[k]))
        elif k in ('b4_512','b5_512') and os.path.exists('test_reg_preds_blend_tta.npy'):
            pass
        elif k == 'b5_512_rrcema':
            pass
    # If some test preds missing (e.g., new EMA B5), fallback to equal weights among available
    if len(test_loaded) != len(keys):
        print('Warning: some test preds missing for keys', keys, 'using available ones only for final blend now.', flush=True)
        weights_use = weights[:len(test_loaded)];
        if weights_use.sum() > 0: weights_use = weights_use / weights_use.sum()
    else:
        weights_use = weights
    # Blend available test preds in consistent key order
    final_test_list = []
    for k in keys:
        if k == 'b4_640': final_test_list.append(preds_b4_640)
        elif k in test_paths and os.path.exists(test_paths[k]): final_test_list.append(np.load(test_paths[k]))
    if len(final_test_list) == 0:
        print('No test predictions available to blend.', flush=True); return
    w_adj = np.array(weights[:len(final_test_list)])
    if w_adj.sum() == 0: w_adj = np.ones_like(w_adj)/len(w_adj)
    w_adj = w_adj / w_adj.sum()
    blend_test = np.zeros_like(final_test_list[0], dtype=float)
    for w, a in zip(w_adj, final_test_list): blend_test += w * a
    th_final = np.load('thresholds_blend_boot.npy') if os.path.exists('thresholds_blend_boot.npy') else th_opt
    cls = preds_to_classes_blend(blend_test, th_final).astype(int)
    sub = pd.DataFrame({'id_code': test_df['id_code'].values, 'diagnosis': cls})
    sub.to_csv('submission.csv', index=False)
    np.save('test_reg_preds_blend_final.npy', blend_test)
    print('Saved submission.csv with NNLS-weighted blend and bootstrapped thresholds.', flush=True)

print('Cell ready. After B5 RRC+EMA completes, execute prepare_and_run_b4_640_and_blend() to continue medal push.', flush=True)

Preparing 640px pipeline (cache + B4 3-fold + blend)


Cell ready. After B5 RRC+EMA completes, execute prepare_and_run_b4_640_and_blend() to continue medal push.


In [15]:
# Post-B5(EMA) inference + trigger 640 pipeline
import numpy as np, torch, pandas as pd
from torch.utils.data import DataLoader

def hflip_tta_predict(models, dl):
    preds = [];
    with torch.no_grad():
        for xb, _ in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                p_sum = 0; cnt = 0
                for m in models:
                    m.eval()
                    p0 = m(xb); p1 = m(torch.flip(xb, dims=[-1]))
                    p_sum += (p0 + p1) / 2.0; cnt += 1
                p = p_sum / max(1, cnt)
            preds.append(p.detach().float().cpu().numpy())
    return np.concatenate(preds)

def run_b5_seed2025_ema_test_infer_if_ready():
    if 'models_b5_seed2025_ema' not in globals():
        print('EMA B5 models not present yet; skip test inference.', flush=True);
        return False
    try:
        dte = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms)
        dl_te = DataLoader(dte, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
        preds = hflip_tta_predict(models_b5_seed2025_ema, dl_te)
        np.save('test_reg_preds_b5_seed2025_rrc_ema.npy', preds)
        print('Saved test_reg_preds_b5_seed2025_rrc_ema.npy', flush=True)
        return True
    except Exception as e:
        print('B5 EMA test inference failed:', e, flush=True);
        return False

print('If B5@512 RRC+EMA run finished, we will cache its test preds, then launch 640 pipeline.', flush=True)
ok = run_b5_seed2025_ema_test_infer_if_ready()
if ok:
    try:
        prepare_and_run_b4_640_and_blend()
    except NameError:
        print('Cell 8 not executed yet; run it first to define prepare_and_run_b4_640_and_blend()', flush=True)

If B5@512 RRC+EMA run finished, we will cache its test preds, then launch 640 pipeline.


Saved test_reg_preds_b5_seed2025_rrc_ema.npy


train: 0/3295 already cached at 640px


Cached 200/3295; elapsed 40.1s


Cached 400/3295; elapsed 79.3s


...

Bootstrap 60/100


Bootstrap 80/100


Bootstrap 100/100


Bootstrapped thresholds: [0.52041135 1.50418103 2.55775329 3.28651022]


Saved submission.csv with NNLS-weighted blend and bootstrapped thresholds.


In [16]:
# Utility: Recompute hflip-only test preds for B4@512 and B5@512, then finalize blend with NNLS weights
import numpy as np, pandas as pd, torch
from torch.utils.data import DataLoader

def hflip_only_predict(models, dl):
    preds = []
    with torch.no_grad():
        for xb, _ in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                s = 0; c = 0
                for m in models:
                    m.eval()
                    p0 = m(xb); p1 = m(torch.flip(xb, dims=[-1]))
                    s += (p0 + p1) / 2.0; c += 1
                p = s / max(1, c)
            preds.append(p.detach().float().cpu().numpy())
    return np.concatenate(preds)

def recompute_hflip_test_preds_for_512_models():
    made = {}
    try:
        dte = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms)
        dl = DataLoader(dte, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
        if 'models' in globals() and isinstance(models, list) and len(models) > 0:
            p_b4 = hflip_only_predict(models, dl)
            np.save('test_reg_preds_b4_hflip.npy', p_b4)
            print('Saved test_reg_preds_b4_hflip.npy', flush=True)
            made['b4_512'] = 'test_reg_preds_b4_hflip.npy'
        else:
            print('models (B4@512) not found; skipping', flush=True)
        if 'models_b5' in globals() and isinstance(models_b5, list) and len(models_b5) > 0:
            p_b5 = hflip_only_predict(models_b5, dl)
            np.save('test_reg_preds_b5_hflip.npy', p_b5)
            print('Saved test_reg_preds_b5_hflip.npy', flush=True)
            made['b5_512'] = 'test_reg_preds_b5_hflip.npy'
        else:
            print('models_b5 (B5@512) not found; skipping', flush=True)
    except Exception as e:
        print('Recompute hflip test preds failed:', e, flush=True)
    return made

def finalize_blend_with_standardized_hflip():
    # Load NNLS weights and keys from previous step
    if not (os.path.exists('blend_weights_keys.npy') and os.path.exists('blend_weights_vals.npy')):
        print('Blend weights not found. Run prepare_and_run_b4_640_and_blend() first.', flush=True);
        return
    keys = np.load('blend_weights_keys.npy', allow_pickle=True).tolist()
    weights = np.load('blend_weights_vals.npy')
    # Thresholds
    th = np.load('thresholds_blend_boot.npy') if os.path.exists('thresholds_blend_boot.npy') else np.array([0.5,1.5,2.5,3.5], dtype=float)
    # Collect test preds per key with hflip-only preference
    pred_map = {}
    for k in keys:
        if k == 'b4_512':
            if os.path.exists('test_reg_preds_b4_hflip.npy'):
                pred_map[k] = np.load('test_reg_preds_b4_hflip.npy')
            elif os.path.exists('test_reg_preds_b4_tta.npy'):
                pred_map[k] = np.load('test_reg_preds_b4_tta.npy')
            elif os.path.exists('test_reg_preds_b4.npy'):
                pred_map[k] = np.load('test_reg_preds_b4.npy')
        elif k == 'b5_512':
            if os.path.exists('test_reg_preds_b5_hflip.npy'):
                pred_map[k] = np.load('test_reg_preds_b5_hflip.npy')
            elif os.path.exists('test_reg_preds_b5_tta.npy'):
                pred_map[k] = np.load('test_reg_preds_b5_tta.npy')
            elif os.path.exists('test_reg_preds.npy'):
                pred_map[k] = np.load('test_reg_preds.npy')
        elif k == 'b5_512_rrcema':
            # already saved as hflip-only earlier
            if os.path.exists('test_reg_preds_b5_seed2025_rrc_ema.npy'):
                pred_map[k] = np.load('test_reg_preds_b5_seed2025_rrc_ema.npy')
        elif k == 'b4_640':
            if os.path.exists('test_reg_preds_b4_640_rrc_ema.npy'):
                pred_map[k] = np.load('test_reg_preds_b4_640_rrc_ema.npy')
    # Verify availability
    missing = [k for k in keys if k not in pred_map]
    if missing:
        print('Missing test preds for keys:', missing, flush=True)
    used_keys = [k for k in keys if k in pred_map]
    mats = [pred_map[k] for k in used_keys]
    if not mats:
        print('No test preds to blend.', flush=True); return
    w = weights[:len(mats)]
    if w.sum() <= 0: w = np.ones_like(w) / len(w)
    w = w / w.sum()
    blend = np.zeros_like(mats[0], dtype=float)
    for wi, arr in zip(w, mats): blend += wi * arr
    # Threshold to classes
    def preds_to_classes_blend(preds, thresholds):
        th0, th1, th2, th3 = thresholds
        return np.digitize(preds, bins=[th0, th1, th2, th3])
    cls = preds_to_classes_blend(blend, th).astype(int)
    sub = pd.DataFrame({'id_code': test_df['id_code'].values, 'diagnosis': cls})
    sub.to_csv('submission.csv', index=False)
    np.save('test_reg_preds_blend_final.npy', blend)
    print('Final standardized hflip-only blend saved to submission.csv', flush=True)

print('Cell ready: after B4@640 completes and initial blend is done, run:', flush=True)
print('- recompute_hflip_test_preds_for_512_models()', flush=True)
print('- finalize_blend_with_standardized_hflip()', flush=True)

Cell ready: after B4@640 completes and initial blend is done, run:


- recompute_hflip_test_preds_for_512_models()


- finalize_blend_with_standardized_hflip()


In [17]:
# Recompute hflip-only preds for 512px models and finalize standardized blend
made = recompute_hflip_test_preds_for_512_models()
print('Made:', made, flush=True)
finalize_blend_with_standardized_hflip()

Saved test_reg_preds_b4_hflip.npy


Saved test_reg_preds_b5_hflip.npy


Made: {'b4_512': 'test_reg_preds_b4_hflip.npy', 'b5_512': 'test_reg_preds_b5_hflip.npy'}


Final standardized hflip-only blend saved to submission.csv


In [18]:
# SEResNeXt50_32x4d @512 (3 folds) with RRC(scale=(0.92,1.0)) + EMA, then refit NNLS with caps [0.05,0.7], bootstrap thresholds (n=180)
import os, gc, copy, time, math, numpy as np, pandas as pd, torch, torch.nn as nn, albumentations as A, cv2
from torch.utils.data import DataLoader
from timm.utils import ModelEmaV2
from scipy.optimize import nnls, minimize
from sklearn.metrics import cohen_kappa_score

print('Starting SEResNeXt50_32x4d@512 (3 folds) RRC+EMA...', flush=True)

# Transforms: RRC tightened to (0.92, 1.0)
MEAN = (0.485, 0.456, 0.406); STD = (0.229, 0.224, 0.225)
train_tfms_serx = A.Compose([
    A.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.92, 1.0), ratio=(0.95, 1.05), interpolation=cv2.INTER_AREA),
    A.HorizontalFlip(p=0.5),
    A.Affine(scale=(0.95, 1.05), translate_percent=(0.0, 0.05), rotate=(-12, 12), border_mode=cv2.BORDER_REFLECT_101, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.7),
    A.RandomGamma(gamma_limit=(90,110), p=0.4),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])
valid_tfms_serx = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

def get_loaders_serx(tr_df, va_df, batch_size=8, num_workers=0):
    dtr = DRDataset(tr_df, IMG_DIR_TRAIN, transforms=train_tfms_serx)
    dva = DRDataset(va_df, IMG_DIR_TRAIN, transforms=valid_tfms_serx)
    dl_tr = DataLoader(dtr, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=True, persistent_workers=False)
    dl_va = DataLoader(dva, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=False)
    return dl_tr, dl_va

def validate_serx(model, dl, loss_fn):
    model.eval(); preds = []; targs = []; val_loss = 0.0; n = 0
    with torch.no_grad():
        for xb, yb in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last); yb = yb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                out = model(xb); loss = loss_fn(out, yb)
            bs = xb.size(0); val_loss += loss.item()*bs; n += bs
            preds.append(out.detach().float().cpu().numpy()); targs.append(yb.detach().float().cpu().numpy())
    preds = np.concatenate(preds); targs = np.concatenate(targs)
    return val_loss/max(1,n), preds, targs

def train_one_fold_serx_ema(fold, folds_df, epochs=15, lr=2e-4, wd=1e-5, batch_size=8, patience=3, ema_decay=0.9996):
    print(f"\n===== [SEResNeXt50@512 RRC+EMA] Fold {fold} =====", flush=True)
    tr_df = folds_df[folds_df['fold'] != fold][['id_code','diagnosis']].reset_index(drop=True)
    va_df = folds_df[folds_df['fold'] == fold][['id_code','diagnosis']].reset_index(drop=True)
    dl_tr, dl_va = get_loaders_serx(tr_df, va_df, batch_size=batch_size, num_workers=0)
    model = RegHeadModel(backbone_name='seresnext50_32x4d', pretrained=True).to(device)
    model = model.to(memory_format=torch.channels_last)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    loss_fn = nn.HuberLoss(delta=1.0)
    scaler = torch.amp.GradScaler('cuda', enabled=True)
    ema = ModelEmaV2(model, decay=ema_decay, device=None)
    total_steps = epochs * len(dl_tr)
    def lr_lambda(step):
        if step < len(dl_tr): return (step+1)/len(dl_tr)
        progress = (step - len(dl_tr)) / max(1, total_steps - len(dl_tr))
        return 0.5 * (1 + math.cos(math.pi * progress))
    sch = torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda)
    best_loss = float('inf'); best_state = None; best_preds = None; best_targs = None; no_imp = 0
    accum = max(1, 16 // batch_size); t0_all = time.time(); opt.zero_grad(set_to_none=True)
    for ep in range(1, epochs+1):
        model.train(); tr_loss=0.0; n=0; t0=time.time()
        for it,(xb,yb) in enumerate(dl_tr):
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last); yb = yb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                out = model(xb); loss = loss_fn(out, yb)/accum
            scaler.scale(loss).backward()
            if (it+1)%accum==0:
                scaler.step(opt); scaler.update(); opt.zero_grad(set_to_none=True); sch.step(); ema.update(model)
            bs = xb.size(0); tr_loss += (loss.item()*accum)*bs; n += bs
            if (it+1)%50==0: print(f'Epoch {ep} it {it+1}/{len(dl_tr)} tr_loss {tr_loss/max(1,n):.4f} elapsed {time.time()-t0:.1f}s', flush=True)
        vloss, vpreds, vtargs = validate_serx(model, dl_va, loss_fn)
        vq = cohen_kappa_score(vtargs, np.digitize(vpreds, bins=[0.5,1.5,2.5,3.5]), weights='quadratic')
        print(f'Epoch {ep}: tr_loss {tr_loss/max(1,n):.4f} val_loss {vloss:.4f} val_qwk(def) {vq:.4f} epoch_time {time.time()-t0:.1f}s total_elapsed {time.time()-t0_all:.1f}s', flush=True)
        if vloss < best_loss:
            best_loss = vloss; best_state = copy.deepcopy(ema.module.state_dict()); best_preds = vpreds.copy(); best_targs = vtargs.copy(); no_imp = 0
        else:
            no_imp += 1
        if no_imp >= patience:
            print('Early stopping triggered', flush=True); break
    ema.module.load_state_dict(best_state)
    return ema.module, best_preds, best_targs

def run_cv_serx_ema(folds_df, folds_to_run=(0,1,2), epochs=15, batch_size=8):
    folds_df = folds_df.reset_index(drop=True).copy()
    n = len(folds_df); oof_preds = np.full(n, np.nan, dtype=np.float32); oof_targs = folds_df['diagnosis'].values.astype(float)
    models_out = []
    for fold in folds_to_run:
        try:
            torch.cuda.empty_cache()
            if torch.cuda.is_available(): torch.cuda.reset_peak_memory_stats()
        except Exception:
            pass
        gc.collect()
        bs = batch_size
        for attempt in range(5):
            print(f'Attempt {attempt+1}: trying batch_size={bs}', flush=True)
            try:
                m, vpreds, vtargs = train_one_fold_serx_ema(fold, folds_df, epochs=epochs, lr=2e-4, wd=1e-5, batch_size=bs, patience=3, ema_decay=0.9996)
                break
            except (RuntimeError, torch.cuda.OutOfMemoryError, torch.OutOfMemoryError) as e:
                if 'out of memory' in str(e).lower():
                    print(f'CUDA OOM on fold {fold} with batch_size={bs}. Reducing...', flush=True)
                    try: del m
                    except Exception: pass
                    gc.collect()
                    try: torch.cuda.empty_cache(); torch.cuda.reset_peak_memory_stats()
                    except Exception: pass
                    bs = max(2, bs-2); continue
                else:
                    raise
        models_out.append(m)
        va_idx = np.where(folds_df['fold'].values == fold)[0]
        oof_preds[va_idx] = vpreds
        fold_q = cohen_kappa_score(vtargs, np.digitize(vpreds, bins=[0.5,1.5,2.5,3.5]), weights='quadratic')
        print(f'[SEResNeXt50@512] Fold {fold} val QWK (default th): {fold_q:.4f}', flush=True)
    np.save('oof_preds_serx50_512_rrc_ema.npy', oof_preds); np.save('oof_targets_serx50_512_rrc_ema.npy', oof_targs)
    return models_out, oof_preds, oof_targs

def hflip_tta_predict_mean(models, dl):
    preds = []
    with torch.no_grad():
        for xb, _ in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                s = 0; c = 0
                for m in models:
                    m.eval(); p0 = m(xb); p1 = m(torch.flip(xb, dims=[-1]))
                    s += (p0 + p1) / 2.0; c += 1
                p = s / max(1,c)
            preds.append(p.detach().float().cpu().numpy())
    return np.concatenate(preds)

def qwk_func(y_true, y_pred_cls):
    return cohen_kappa_score(y_true, y_pred_cls, weights='quadratic')

def preds_to_classes_blend(preds, thresholds):
    th0, th1, th2, th3 = thresholds
    return np.digitize(preds, bins=[th0, th1, th2, th3])

def optimize_thresholds_generic(y, p, init=[0.5,1.5,2.5,3.5]):
    y = np.asarray(y).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, -1.0, 4.0)
        cls = preds_to_classes_blend(p, th)
        return -qwk_func(y, cls)
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i] - th[i-1] < 0.05: th[i] = th[i-1] + 0.05
    return th

def bootstrap_thresholds_generic(y, p, n_bootstrap=180, base=[0.5,1.5,2.5,3.5]):
    N = len(y); cuts = []
    for i in range(n_bootstrap):
        idx = np.random.randint(0, N, size=N)
        th = optimize_thresholds_generic(y[idx], p[idx], init=base)
        cuts.append(th)
        if (i+1)%30==0: print(f'Bootstrap {i+1}/{n_bootstrap}', flush=True)
    cuts = np.array(cuts); med = np.median(cuts, axis=0); med = np.clip(med, 0.3, 3.7)
    for j in range(1,4):
        if med[j] - med[j-1] < 0.12: med[j] = med[j-1] + 0.12
    base_arr = np.array([0.5,1.5,2.5,3.5])
    return 0.9*med + 0.1*base_arr

def nnls_blend_weights_capped(y, preds_list, mask=None, lo=0.05, hi=0.7):
    y = np.asarray(y).astype(float); mats = [np.asarray(p).reshape(-1,1) for p in preds_list]
    X = np.concatenate(mats, axis=1)
    if mask is not None: X = X[mask]; y = y[mask]
    w, _ = nnls(X, y)
    if w.sum() > 0: w = w / w.sum()
    else: w = np.ones_like(w) / len(w)
    w = np.clip(w, lo, hi)
    w = w / w.sum()
    return w

# 1) Train SEResNeXt50 (3 folds)
models_serx, oof_serx, oof_t = run_cv_serx_ema(folds_df, folds_to_run=(0,1,2), epochs=15, batch_size=8)

# 2) Inference for SEResNeXt50 (hflip-only TTA) on test
dte_serx = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms_serx)
dl_te_serx = DataLoader(dte_serx, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
preds_serx = hflip_tta_predict_mean(models_serx, dl_te_serx)
np.save('test_reg_preds_serx50_512_rrc_ema.npy', preds_serx)
print('Saved test_reg_preds_serx50_512_rrc_ema.npy', flush=True)

# 3) Refit NNLS on OOF intersection including SEResNeXt50 with caps [0.05,0.7]
paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
}
loaded_keys = []; loaded_arrs = []; mask = np.ones_like(oof_t, dtype=bool)
for k,p in paths_oof.items():
    if os.path.exists(p):
        arr = np.load(p); loaded_keys.append(k); loaded_arrs.append(arr); mask &= np.isfinite(arr)
    else:
        print('Missing OOF for', k, '->', p, flush=True)
weights = nnls_blend_weights_capped(oof_t, loaded_arrs, mask=mask, lo=0.05, hi=0.7)
print('NNLS weights (capped) for', loaded_keys, ':', weights, flush=True)

# 4) Create blended OOF, bootstrap thresholds (n=180)
blend_oof = np.zeros_like(oof_t, dtype=float)
for w, a in zip(weights, loaded_arrs):
    a_fill = np.where(np.isfinite(a), a, 0.0); blend_oof += w * a_fill
th_opt = optimize_thresholds_generic(oof_t[mask], blend_oof[mask])
oof_q_opt = qwk_func(oof_t[mask], preds_to_classes_blend(blend_oof[mask], th_opt))
print('Blend OOF QWK (opt on mask):', f'{oof_q_opt:.5f}', 'th:', th_opt, flush=True)
th_boot = bootstrap_thresholds_generic(oof_t[mask], blend_oof[mask], n_bootstrap=180, base=[0.5,1.5,2.5,3.5])
print('Bootstrapped thresholds (n=180):', th_boot, flush=True)
np.save('blend_weights_keys.npy', np.array(loaded_keys, dtype=object)); np.save('blend_weights_vals.npy', weights); np.save('thresholds_blend_boot.npy', th_boot)

# 5) Finalize standardized hflip-only test blend across all available keys
pred_map = {}
for k in loaded_keys:
    if k == 'b4_512' and os.path.exists('test_reg_preds_b4_hflip.npy'): pred_map[k] = np.load('test_reg_preds_b4_hflip.npy')
    elif k == 'b5_512' and os.path.exists('test_reg_preds_b5_hflip.npy'): pred_map[k] = np.load('test_reg_preds_b5_hflip.npy')
    elif k == 'b5_512_rrcema' and os.path.exists('test_reg_preds_b5_seed2025_rrc_ema.npy'): pred_map[k] = np.load('test_reg_preds_b5_seed2025_rrc_ema.npy')
    elif k == 'b4_640' and os.path.exists('test_reg_preds_b4_640_rrc_ema.npy'): pred_map[k] = np.load('test_reg_preds_b4_640_rrc_ema.npy')
    elif k == 'serx50_512_rrcema' and os.path.exists('test_reg_preds_serx50_512_rrc_ema.npy'): pred_map[k] = np.load('test_reg_preds_serx50_512_rrc_ema.npy')

used_keys = [k for k in loaded_keys if k in pred_map]
mats = [pred_map[k] for k in used_keys]
if len(mats) == 0:
    print('No test preds available to blend.', flush=True)
else:
    w_use = weights[:len(mats)]
    if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
    w_use = w_use / w_use.sum()
    blend_test = np.zeros_like(mats[0], dtype=float)
    for wi, arr in zip(w_use, mats): blend_test += wi * arr
    cls = preds_to_classes_blend(blend_test, th_boot).astype(int)
    sub = pd.DataFrame({'id_code': test_df['id_code'].values, 'diagnosis': cls})
    sub.to_csv('submission.csv', index=False)
    np.save('test_reg_preds_blend_final.npy', blend_test)
    print('Final submission.csv written with SEResNeXt50 included.', flush=True)

Starting SEResNeXt50_32x4d@512 (3 folds) RRC+EMA...


Attempt 1: trying batch_size=8



===== [SEResNeXt50@512 RRC+EMA] Fold 0 =====


Epoch 1 it 50/329 tr_loss 0.8071 elapsed 17.4s


Epoch 1 it 100/329 tr_loss 0.7231 elapsed 35.1s


...

Bootstrap 120/180


Bootstrap 150/180


Bootstrap 180/180


Bootstrapped thresholds (n=180): [0.53542452 1.53205458 2.44638209 3.23597427]


Final submission.csv written with SEResNeXt50 included.


In [19]:
# SEResNeXt50_32x4d second seed @512 (3 folds) with RRC(scale=(0.92,1.0)) + EMA, then refit NNLS (caps [0.05,0.7]) and bootstrap thresholds (n=180)
import os, numpy as np, pandas as pd, torch
from torch.utils.data import DataLoader

print('Starting SEResNeXt50_32x4d@512 second seed (3 folds) RRC+EMA...', flush=True)
seed_everything(2026)

# 1) Train SEResNeXt50 second seed (3 folds)
models_serx_s2, oof_serx_s2, oof_t = run_cv_serx_ema(folds_df, folds_to_run=(0,1,2), epochs=15, batch_size=8)
np.save('oof_preds_serx50_512_rrc_ema_seed2026.npy', oof_serx_s2); np.save('oof_targets_serx50_512_rrc_ema_seed2026.npy', oof_t)

# 2) Inference for SEResNeXt50 second seed (hflip-only TTA) on test
dte_serx2 = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms_serx)
dl_te_serx2 = DataLoader(dte_serx2, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
preds_serx2 = hflip_tta_predict_mean(models_serx_s2, dl_te_serx2)
np.save('test_reg_preds_serx50_512_rrc_ema_seed2026.npy', preds_serx2)
print('Saved test_reg_preds_serx50_512_rrc_ema_seed2026.npy', flush=True)

# 3) Refit NNLS on OOF intersection including both SEResNeXt seeds with caps [0.05,0.7]
paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
}
loaded_keys = []; loaded_arrs = []; mask = np.ones_like(oof_t, dtype=bool)
for k,p in paths_oof.items():
    if os.path.exists(p):
        arr = np.load(p); loaded_keys.append(k); loaded_arrs.append(arr); mask &= np.isfinite(arr)
    else:
        print('Missing OOF for', k, '->', p, flush=True)
weights = nnls_blend_weights_capped(oof_t, loaded_arrs, mask=mask, lo=0.05, hi=0.7)
print('NNLS weights (capped) for', loaded_keys, ':', weights, flush=True)

# 4) Create blended OOF, bootstrap thresholds (n=180)
blend_oof = np.zeros_like(oof_t, dtype=float)
for w, a in zip(weights, loaded_arrs):
    a_fill = np.where(np.isfinite(a), a, 0.0); blend_oof += w * a_fill
th_opt = optimize_thresholds_generic(oof_t[mask], blend_oof[mask])
oof_q_opt = qwk_func(oof_t[mask], preds_to_classes_blend(blend_oof[mask], th_opt))
print('Blend OOF QWK (opt on mask):', f'{oof_q_opt:.5f}', 'th:', th_opt, flush=True)
th_boot = bootstrap_thresholds_generic(oof_t[mask], blend_oof[mask], n_bootstrap=180, base=[0.5,1.5,2.5,3.5])
print('Bootstrapped thresholds (n=180):', th_boot, flush=True)
np.save('blend_weights_keys.npy', np.array(loaded_keys, dtype=object)); np.save('blend_weights_vals.npy', weights); np.save('thresholds_blend_boot.npy', th_boot)

# 5) Finalize standardized hflip-only test blend across all available keys (including s2) and write submission
pred_map = {}
for k in loaded_keys:
    if k == 'b4_512' and os.path.exists('test_reg_preds_b4_hflip.npy'): pred_map[k] = np.load('test_reg_preds_b4_hflip.npy')
    elif k == 'b5_512' and os.path.exists('test_reg_preds_b5_hflip.npy'): pred_map[k] = np.load('test_reg_preds_b5_hflip.npy')
    elif k == 'b5_512_rrcema' and os.path.exists('test_reg_preds_b5_seed2025_rrc_ema.npy'): pred_map[k] = np.load('test_reg_preds_b5_seed2025_rrc_ema.npy')
    elif k == 'b4_640' and os.path.exists('test_reg_preds_b4_640_rrc_ema.npy'): pred_map[k] = np.load('test_reg_preds_b4_640_rrc_ema.npy')
    elif k == 'serx50_512_rrcema' and os.path.exists('test_reg_preds_serx50_512_rrc_ema.npy'): pred_map[k] = np.load('test_reg_preds_serx50_512_rrc_ema.npy')
    elif k == 'serx50_512_rrcema_s2' and os.path.exists('test_reg_preds_serx50_512_rrc_ema_seed2026.npy'): pred_map[k] = np.load('test_reg_preds_serx50_512_rrc_ema_seed2026.npy')
used_keys = [k for k in loaded_keys if k in pred_map]
mats = [pred_map[k] for k in used_keys]
if len(mats) == 0:
    print('No test preds available to blend.', flush=True)
else:
    w_use = weights[:len(mats)]
    if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
    w_use = w_use / w_use.sum()
    blend_test = np.zeros_like(mats[0], dtype=float)
    for wi, arr in zip(w_use, mats): blend_test += wi * arr
    cls = preds_to_classes_blend(blend_test, th_boot).astype(int)
    sub = pd.DataFrame({'id_code': test_df['id_code'].values, 'diagnosis': cls})
    sub.to_csv('submission.csv', index=False)
    np.save('test_reg_preds_blend_final.npy', blend_test)
    print('Final submission.csv written with SEResNeXt50 second seed included.', flush=True)

Starting SEResNeXt50_32x4d@512 second seed (3 folds) RRC+EMA...


Attempt 1: trying batch_size=8



===== [SEResNeXt50@512 RRC+EMA] Fold 0 =====


Epoch 1 it 50/329 tr_loss 0.7859 elapsed 17.5s


Epoch 1 it 100/329 tr_loss 0.7754 elapsed 35.2s


...

Bootstrap 120/180


Bootstrap 150/180


Bootstrap 180/180


Bootstrapped thresholds (n=180): [0.53216035 1.51247651 2.47348362 3.39321307]


Final submission.csv written with SEResNeXt50 second seed included.


In [21]:
# Level-2 Stacker (LightGBM multiclass) on OOF regression features; hflip-only test; bootstrap thresholds; write submission
import os, numpy as np, pandas as pd
from sklearn.metrics import cohen_kappa_score
import lightgbm as lgb

print('L2 stacking: building feature matrices from OOF/test preds...', flush=True)
oof_t = np.load('oof_targets_b4.npy') if os.path.exists('oof_targets_b4.npy') else np.load('oof_targets.npy')
folds_df_l2 = pd.read_csv('folds.csv')

# Define model keys and corresponding file paths (OOF and Test)
keys = [
    ('b4_512', 'oof_preds_b4.npy', 'test_reg_preds_b4_hflip.npy'),
    ('b5_512', 'oof_preds.npy', 'test_reg_preds_b5_hflip.npy'),
    ('b5_512_rrcema', 'oof_preds_b5_seed2025_rrc_ema.npy', 'test_reg_preds_b5_seed2025_rrc_ema.npy'),
    ('b4_640', 'oof_preds_b4_640_rrc_ema.npy', 'test_reg_preds_b4_640_rrc_ema.npy'),
    ('serx50_512_rrcema', 'oof_preds_serx50_512_rrc_ema.npy', 'test_reg_preds_serx50_512_rrc_ema.npy'),
    ('serx50_512_rrcema_s2', 'oof_preds_serx50_512_rrc_ema_seed2026.npy', 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy'),
]

feat_oof_list = []
feat_te_list = []
used_keys = []
for name, oof_path, te_path in keys:
    if os.path.exists(oof_path) and os.path.exists(te_path):
        oof_arr = np.load(oof_path)
        te_arr = np.load(te_path)
        feat_oof_list.append(oof_arr.reshape(-1, 1))
        feat_te_list.append(te_arr.reshape(-1, 1))
        used_keys.append(name)
    else:
        print('Skipping key due to missing files:', name, '->', oof_path, te_path, flush=True)

if len(feat_oof_list) == 0:
    raise RuntimeError('No features available for stacking.')

X_oof_full = np.concatenate(feat_oof_list, axis=1).astype(np.float32)
X_te_full = np.concatenate(feat_te_list, axis=1).astype(np.float32)
mask = np.isfinite(X_oof_full).all(axis=1) & np.isfinite(oof_t)
print('Features used:', used_keys, 'X_oof shape:', X_oof_full.shape, 'mask sum:', int(mask.sum()), flush=True)

# L2 LightGBM classifier with 5 classes
params = dict(objective='multiclass', num_class=5, learning_rate=0.03, n_estimators=2000,
              num_leaves=63, max_depth=-1, feature_fraction=0.8, bagging_fraction=0.8, bagging_freq=1,
              reg_lambda=1.0, verbosity=-1)

X = X_oof_full[mask]
y = oof_t[mask].astype(int)
X_te = X_te_full.copy()

# 5-fold CV consistent with original folds
oof_proba = np.zeros((len(X_oof_full), 5), dtype=np.float32)
te_proba_acc = np.zeros((len(X_te), 5), dtype=np.float32)

for f in range(5):
    va_idx_all = (folds_df_l2['fold'].values == f)
    va_mask_idx = np.where(mask & va_idx_all)[0]
    tr_mask_idx = np.where(mask & (~va_idx_all))[0]
    if len(va_mask_idx) == 0 or len(tr_mask_idx) == 0:
        continue
    X_tr, y_tr = X_oof_full[tr_mask_idx], oof_t[tr_mask_idx].astype(int)
    X_va, y_va = X_oof_full[va_mask_idx], oof_t[va_mask_idx].astype(int)
    clf = lgb.LGBMClassifier(**params)
    # Remove unsupported verbose kwarg; use callbacks for early stopping and silent logging
    clf.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], eval_metric='multi_logloss', callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)])
    oof_proba[va_mask_idx] = clf.predict_proba(X_va)
    te_proba_acc += clf.predict_proba(X_te)
    y_va_pred = np.argmax(oof_proba[va_mask_idx], axis=1)
    print(f'[L2] Fold {f} done; val size {len(va_mask_idx)}', flush=True)

te_proba = te_proba_acc / 5.0

# Convert probabilities to expected value scalar for thresholds
classes = np.arange(5, dtype=np.float32).reshape(1, -1)
oof_reg = (oof_proba * classes).sum(axis=1)
te_reg = (te_proba * classes).sum(axis=1)

def qwk_func(y_true, y_pred_cls):
    return cohen_kappa_score(y_true, y_pred_cls, weights='quadratic')

def preds_to_classes_blend(preds, thresholds):
    th0, th1, th2, th3 = thresholds
    return np.digitize(preds, bins=[th0, th1, th2, th3])

from scipy.optimize import minimize
def optimize_thresholds_generic(y, p, init=[0.5,1.5,2.5,3.5]):
    y = np.asarray(y).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, -1.0, 4.0)
        cls = preds_to_classes_blend(p, th)
        return -qwk_func(y, cls)
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i] - th[i-1] < 0.05: th[i] = th[i-1] + 0.05
    return th

def bootstrap_thresholds_generic(y, p, n_bootstrap=200, base=[0.5,1.5,2.5,3.5]):
    N = len(y); cuts = []
    for i in range(n_bootstrap):
        idx = np.random.randint(0, N, size=N)
        th = optimize_thresholds_generic(y[idx], p[idx], init=base)
        cuts.append(th)
        if (i+1)%30==0: print(f'Bootstrap {i+1}/{n_bootstrap}', flush=True)
    cuts = np.array(cuts); med = np.median(cuts, axis=0); med = np.clip(med, 0.3, 3.7)
    for j in range(1,4):
        if med[j] - med[j-1] < 0.12: med[j] = med[j-1] + 0.12
    base_arr = np.array([0.5,1.5,2.5,3.5])
    return 0.9*med + 0.1*base_arr

# Optimize thresholds on masked OOF expected values
th_opt = optimize_thresholds_generic(oof_t[mask], oof_reg[mask])
oof_q_opt = qwk_func(oof_t[mask], preds_to_classes_blend(oof_reg[mask], th_opt))
print('L2 OOF QWK (opt on mask):', f'{oof_q_opt:.5f}', 'th:', th_opt, flush=True)
th_boot = bootstrap_thresholds_generic(oof_t[mask], oof_reg[mask], n_bootstrap=200, base=[0.5,1.5,2.5,3.5])
print('L2 bootstrapped thresholds (n=200):', th_boot, flush=True)
np.save('l2_used_keys.npy', np.array(used_keys, dtype=object)); np.save('l2_oof_reg.npy', oof_reg); np.save('l2_te_reg.npy', te_reg); np.save('l2_thresholds_boot.npy', th_boot)

# Final test classes and submission
cls_final = preds_to_classes_blend(te_reg, th_boot).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls_final})
sub.to_csv('submission.csv', index=False)
print('L2 stacker submission.csv written.', flush=True)

L2 stacking: building feature matrices from OOF/test preds...


Features used: ['b4_512', 'b5_512', 'b5_512_rrcema', 'b4_640', 'serx50_512_rrcema', 'serx50_512_rrcema_s2'] X_oof shape: (3295, 6) mask sum: 1977


Training until validation scores don't improve for 100 rounds


Early stopping, best iteration is:
[80]	valid_0's multi_logloss: 0.603583
[L2] Fold 0 done; val size 659


Training until validation scores don't improve for 100 rounds


...

Bootstrap 120/200


Bootstrap 150/200


Bootstrap 180/200


L2 bootstrapped thresholds (n=200): [0.53360907 1.59051364 2.33572592 3.20070596]


L2 stacker submission.csv written.


In [22]:
# Ordinal Head Fine-tuning for B5@512 (RRC+EMA) per expert recipe
import os, gc, copy, time, math, numpy as np, pandas as pd, torch, torch.nn as nn, albumentations as A, cv2
from torch.utils.data import DataLoader
from timm.utils import ModelEmaV2
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import nnls, minimize
from albumentations.pytorch import ToTensorV2

print('Starting B5@512 Ordinal head fine-tuning (5 folds)...', flush=True)

# Reuse globals: folds_df, IMG_SIZE, IMG_DIR_TRAIN, device, seed_everything, HF_CACHE_DIR
MEAN = (0.485, 0.456, 0.406); STD = (0.229, 0.224, 0.225)
train_tfms_ord = A.Compose([
    A.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.85, 1.0), ratio=(0.95, 1.05), interpolation=cv2.INTER_AREA),
    A.HorizontalFlip(p=0.5),
    A.Affine(scale=(0.95, 1.05), translate_percent=(0.0, 0.06), rotate=(-15, 15), border_mode=cv2.BORDER_REFLECT_101, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.7),
    A.RandomGamma(gamma_limit=(90,110), p=0.4),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])
valid_tfms_ord = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

class OrdinalDRDataset(Dataset):
    def __init__(self, df: pd.DataFrame, img_dir: Path, transforms=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.transforms = transforms
        self.has_target = 'diagnosis' in df.columns
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_dir / f"{row['id_code']}.png"
        img = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
        if img is None:
            img = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if img.shape[0] != IMG_SIZE or img.shape[1] != IMG_SIZE:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
        if self.transforms:
            img = self.transforms(image=img)['image']
        if self.has_target:
            y = int(row['diagnosis'])
            return img, torch.tensor(y, dtype=torch.long)
        else:
            return img, row['id_code']

class OrdinalHeadModel(nn.Module):
    def __init__(self, backbone_name='tf_efficientnet_b5_ns', pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=0, global_pool='avg', cache_dir=str(HF_CACHE_DIR))
        try:
            if hasattr(self.backbone, 'set_grad_checkpointing'):
                self.backbone.set_grad_checkpointing(True)
        except Exception:
            pass
        in_ch = self.backbone.num_features
        self.dropout = nn.Dropout(0.25)
        self.head = nn.Linear(in_ch, 4)  # 4 ordinal logits
    def forward(self, x):
        feat = self.backbone(x)
        feat = self.dropout(feat)
        logits = self.head(feat)
        return logits  # raw logits

def make_targets_ordinal(y_long: torch.Tensor) -> torch.Tensor:
    y = y_long.float()
    t = torch.stack([(y >= 1.0), (y >= 2.0), (y >= 3.0), (y >= 4.0)], dim=1).float()
    return t

def get_pos_weight_from_train(df_all: pd.DataFrame) -> torch.Tensor:
    y = df_all['diagnosis'].values.astype(int)
    N = len(y)
    pos = []
    for k in range(4):
        s = (y >= (k+1)).sum()
        s = max(1, int(s))
        neg = N - s
        pos.append(neg / s)
    pw = torch.tensor(pos, dtype=torch.float32, device=device)
    return pw

def qwk_from_ev(y_true, ev, thresholds):
    y = np.asarray(y_true).astype(float)
    th0, th1, th2, th3 = thresholds
    cls = np.digitize(ev, bins=[th0, th1, th2, th3])
    return cohen_kappa_score(y, cls, weights='quadratic')

def optimize_thresholds_ev(y, ev, init=[0.5,1.5,2.5,3.5]):
    y = np.asarray(y).astype(float); p = np.asarray(ev).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, -0.5, 4.5)
        th0, th1, th2, th3 = th
        cls = np.digitize(p, bins=[th0, th1, th2, th3])
        return -cohen_kappa_score(y, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':1500, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i] - th[i-1] < 0.05:
            th[i] = th[i-1] + 0.05
    return th

def get_loaders_ord(tr_df, va_df, batch_size=8, num_workers=0):
    dtr = OrdinalDRDataset(tr_df, IMG_DIR_TRAIN, transforms=train_tfms_ord)
    dva = OrdinalDRDataset(va_df, IMG_DIR_TRAIN, transforms=valid_tfms_ord)
    dl_tr = DataLoader(dtr, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=True, persistent_workers=False)
    dl_va = DataLoader(dva, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=False)
    return dl_tr, dl_va

def validate_ord(model, dl, loss_fn):
    model.eval(); val_loss=0.0; n=0; all_logits=[]; all_y=[]
    with torch.no_grad():
        for xb, yb in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            yb = yb.to(device, non_blocking=True)
            yt = make_targets_ordinal(yb)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                logits = model(xb)
                loss = loss_fn(logits, yt)
            bs = xb.size(0); val_loss += loss.item()*bs; n += bs
            all_logits.append(logits.detach().float().cpu().numpy()); all_y.append(yb.detach().long().cpu().numpy())
    logits = np.concatenate(all_logits); y = np.concatenate(all_y)
    probs = 1.0 / (1.0 + np.exp(-logits))
    ev = probs.sum(axis=1)
    return val_loss/max(1,n), logits, probs, ev, y

def hflip_tta_logits(models, dl):
    preds = []
    with torch.no_grad():
        for xb, _ in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                s = 0; c = 0
                for m in models:
                    m.eval()
                    l0 = m(xb); l1 = m(torch.flip(xb, dims=[-1]))
                    s += (l0 + l1) / 2.0; c += 1
                logits = s / max(1,c)
            preds.append(logits.detach().float().cpu().numpy())
    return np.concatenate(preds)

def train_fold_b5_ord(fold, folds_df, batch_size=8, ema_decay=0.9996, patience=2, epochs_head=1, epochs_unfreeze=4):
    print(f"\n===== [B5 Ordinal] Fold {fold} =====", flush=True)
    tr_df = folds_df[folds_df['fold'] != fold][['id_code','diagnosis']].reset_index(drop=True)
    va_df = folds_df[folds_df['fold'] == fold][['id_code','diagnosis']].reset_index(drop=True)
    dl_tr, dl_va = get_loaders_ord(tr_df, va_df, batch_size=batch_size, num_workers=0)
    model = OrdinalHeadModel('tf_efficientnet_b5_ns', pretrained=True).to(device)
    model = model.to(memory_format=torch.channels_last)
    ema = ModelEmaV2(model, decay=ema_decay, device=None)
    # pos_weight from full train
    pos_weight = get_pos_weight_from_train(folds_df)
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction='mean')
    scaler = torch.amp.GradScaler('cuda', enabled=True)

    def run_phase(opt, sch, max_epochs, freeze_backbone=False):
        best_loss = float('inf'); best_state=None; best_logits=None; best_probs=None; best_ev=None; best_y=None; no_imp=0
        if freeze_backbone:
            for p in model.backbone.parameters(): p.requires_grad_(False)
            model.backbone.eval()
        else:
            for p in model.backbone.parameters(): p.requires_grad_(True)
            model.backbone.train()
        for ep in range(1, max_epochs+1):
            model.train(); tr_loss=0.0; n=0; t0=time.time()
            for it, (xb, yb) in enumerate(dl_tr):
                xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
                yb = yb.to(device, non_blocking=True)
                yt = make_targets_ordinal(yb)
                with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                    logits = model(xb); loss = loss_fn(logits, yt)
                scaler.scale(loss).backward()
                scaler.step(opt); scaler.update(); opt.zero_grad(set_to_none=True);
                if sch is not None: sch.step()
                ema.update(model)
                bs = xb.size(0); tr_loss += loss.item()*bs; n += bs
                if (it+1) % 50 == 0:
                    print(f'  it {it+1}/{len(dl_tr)} tr_loss {tr_loss/max(1,n):.4f} elapsed {time.time()-t0:.1f}s', flush=True)
            vloss, vlogits, vprobs, vev, vy = validate_ord(model, dl_va, loss_fn)
            q_def = qwk_from_ev(vy, vev, [0.5,1.5,2.5,3.5])
            print(f'  Epoch {ep}: tr_loss {tr_loss/max(1,n):.4f} val_loss {vloss:.4f} val_qwk(def) {q_def:.4f}', flush=True)
            if vloss < best_loss:
                best_loss = vloss; best_state = copy.deepcopy(ema.module.state_dict());
                best_logits = vlogits.copy(); best_probs = vprobs.copy(); best_ev = vev.copy(); best_y = vy.copy(); no_imp = 0
            else:
                no_imp += 1
            if no_imp >= patience:
                print('  Early stopping.', flush=True); break
        ema.module.load_state_dict(best_state)
        model.load_state_dict(best_state, strict=False)
        return best_logits, best_probs, best_ev, best_y

    # Phase A: head-only
    head_params = list(model.head.parameters()) + list(model.dropout.parameters())
    optA = torch.optim.AdamW(head_params, lr=1e-3, weight_decay=1e-5)
    stepsA = max(1, epochs_head) * len(dl_tr)
    def lr_lambda_a(step):
        warm = len(dl_tr)
        if step < warm: return (step+1)/warm
        prog = (step - warm) / max(1, stepsA - warm)
        return 0.5*(1+math.cos(math.pi*prog))
    schA = torch.optim.lr_scheduler.LambdaLR(optA, lr_lambda_a)
    _ = run_phase(optA, schA, max_epochs=epochs_head, freeze_backbone=True)

    # Phase B: unfreeze all with small LR on backbone, higher on head
    params = [
        {'params': model.backbone.parameters(), 'lr': 3e-5},
        {'params': model.head.parameters(), 'lr': 1e-4},
        {'params': model.dropout.parameters(), 'lr': 1e-4},
    ]
    optB = torch.optim.AdamW(params, lr=1e-4, weight_decay=1e-5)
    stepsB = max(1, epochs_unfreeze) * len(dl_tr)
    def lr_lambda_b(step):
        warm = len(dl_tr)
        if step < warm: return (step+1)/warm
        prog = (step - warm) / max(1, stepsB - warm)
        return 0.5*(1+math.cos(math.pi*prog))
    schB = torch.optim.lr_scheduler.LambdaLR(optB, lr_lambda_b)
    vlogits, vprobs, vev, vy = run_phase(optB, schB, max_epochs=epochs_unfreeze, freeze_backbone=False)
    return ema.module, vlogits, vprobs, vev, vy

def run_cv_b5_ordinal(folds_df, batch_size=8):
    folds_df = folds_df.reset_index(drop=True).copy()
    n = len(folds_df); n_folds = folds_df['fold'].nunique()
    oof_ev = np.zeros(n, dtype=np.float32)
    oof_probs4 = np.zeros((n,4), dtype=np.float32)
    y_all = folds_df['diagnosis'].values.astype(int)
    models_ord = []
    for f in range(n_folds):
        try:
            torch.cuda.empty_cache()
            if torch.cuda.is_available(): torch.cuda.reset_peak_memory_stats()
        except Exception: pass
        gc.collect()
        m, vlogits, vprobs, vev, vy = train_fold_b5_ord(f, folds_df, batch_size=batch_size, ema_decay=0.9996, patience=2, epochs_head=1, epochs_unfreeze=4)
        models_ord.append(m)
        va_idx = np.where(folds_df['fold'].values == f)[0]
        oof_ev[va_idx] = vev.astype(np.float32)
        oof_probs4[va_idx] = vprobs.astype(np.float32)
        q_def = qwk_from_ev(vy, vev, [0.5,1.5,2.5,3.5])
        print(f'[B5 Ordinal] Fold {f} val QWK(def): {q_def:.4f}', flush=True)
    th = optimize_thresholds_ev(y_all, oof_ev)
    oof_q = qwk_from_ev(y_all, oof_ev, th)
    print('B5 Ordinal OOF QWK (opt):', f'{oof_q:.5f}', 'th:', th, flush=True)
    np.save('oof_ev_b5_ordinal.npy', oof_ev); np.save('oof_probs4_b5_ordinal.npy', oof_probs4); np.save('oof_targets_b5_ordinal.npy', y_all); np.save('thresholds_b5_ordinal.npy', th)
    return models_ord, oof_ev, oof_probs4, y_all, th, oof_q

def infer_test_b5_ordinal(models_ord):
    dte = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms_ord)
    dl_te = DataLoader(dte, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
    logits = hflip_tta_logits(models_ord, dl_te)
    probs = 1.0 / (1.0 + np.exp(-logits))
    ev = probs.sum(axis=1).astype(np.float32)
    np.save('test_ev_b5_ordinal.npy', ev); np.save('test_probs4_b5_ordinal.npy', probs.astype(np.float32))
    print('Saved test_ev_b5_ordinal.npy and test_probs4_b5_ordinal.npy', flush=True)
    return ev, probs

def nnls_blend_ord_with_existing():
    print('Blending ordinal EV with existing regression EVs via NNLS...', flush=True)
    # Load OOF targets (use main folds targets)
    y = np.load('oof_targets_b4.npy') if os.path.exists('oof_targets_b4.npy') else np.load('oof_targets.npy')
    # Load available OOF EV sources
    mats = []; keys = []
    def add_oof(path, key):
        if os.path.exists(path):
            arr = np.load(path); mats.append(arr.reshape(-1,1)); keys.append(key)
            return True
        return False
    add_oof('oof_preds_b4.npy', 'b4_512')
    add_oof('oof_preds.npy', 'b5_512')
    add_oof('oof_preds_b5_seed2025_rrc_ema.npy', 'b5_512_rrcema')
    add_oof('oof_preds_b4_640_rrc_ema.npy', 'b4_640')
    add_oof('oof_preds_serx50_512_rrc_ema.npy', 'serx50_512_rrcema')
    add_oof('oof_preds_serx50_512_rrc_ema_seed2026.npy', 'serx50_512_rrcema_s2')
    add_oof('oof_ev_b5_ordinal.npy', 'b5_512_ordinal')
    if len(mats) < 2:
        print('Not enough OOF sources to blend.')
        return None, None, None, None
    X = np.concatenate(mats, axis=1)
    mask = np.isfinite(X).all(axis=1) & np.isfinite(y)
    w, _ = nnls(X[mask], y[mask].astype(float))
    if w.sum() > 0: w = w / w.sum()
    w = np.clip(w, 0.05, 0.8); w = w / w.sum()
    blend_oof = (X * w.reshape(1,-1)).sum(axis=1)
    th = optimize_thresholds_ev(y[mask], blend_oof[mask])
    oof_q = qwk_from_ev(y[mask], blend_oof[mask], th)
    print('NNLS blend (incl ordinal) OOF QWK:', f'{oof_q:.5f}', 'weights:', w, 'keys:', keys, flush=True)
    np.save('blend_weights_keys.npy', np.array(keys, dtype=object)); np.save('blend_weights_vals.npy', w);
    # Build test blend in same key order
    test_list = []
    test_map = {
        'b4_512': 'test_reg_preds_b4_hflip.npy',
        'b5_512': 'test_reg_preds_b5_hflip.npy',
        'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
        'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
        'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
        'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
        'b5_512_ordinal': 'test_ev_b5_ordinal.npy',
    }
    for k in keys:
        pth = test_map.get(k, None)
        if pth and os.path.exists(pth):
            test_list.append(np.load(pth).reshape(-1))
        else:
            print('Missing test preds for key', k, '->', pth, flush=True)
    if len(test_list) != len(keys):
        print('Warning: some test preds missing; re-normalizing weights over available.', flush=True)
    avail = min(len(test_list), len(w))
    w_use = w[:avail];
    if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
    w_use = w_use / w_use.sum()
    blend_test = np.zeros_like(test_list[0], dtype=float)
    for wi, arr in zip(w_use, test_list[:avail]):
        blend_test += wi * arr
    return blend_oof, blend_test, w, th

# Execute training and inference
seed_everything(42)
models_b5_ordinal, oof_ev_b5_ord, oof_probs4_b5_ord, y_all_b5_ord, th_b5_ord, oof_q_b5_ord = run_cv_b5_ordinal(folds_df, batch_size=8)
print('B5 Ordinal OOF done. OOF QWK:', oof_q_b5_ord, flush=True)
test_ev_b5_ord, test_probs4_b5_ord = infer_test_b5_ordinal(models_b5_ordinal)

# Blend with existing models via NNLS and write submission
blend_oof_ev, blend_test_ev, w_blend, th_blend = nnls_blend_ord_with_existing()
if blend_oof_ev is not None:
    # Bootstrap thresholds on blended OOF EV
    def bootstrap_thresholds_ev(y, p, n_bootstrap=180, base=[0.5,1.5,2.5,3.5]):
        N = len(y); cuts = []
        for i in range(n_bootstrap):
            idx = np.random.randint(0, N, size=N)
            th = optimize_thresholds_ev(y[idx], p[idx], init=base)
            cuts.append(th)
            if (i+1)%30==0: print(f'Bootstrap {i+1}/{n_bootstrap}', flush=True)
        cuts = np.array(cuts); med = np.median(cuts, axis=0); med = np.clip(med, 0.3, 3.7)
        for j in range(1,4):
            if med[j] - med[j-1] < 0.1: med[j] = med[j-1] + 0.1
        base_arr = np.array([0.5,1.5,2.5,3.5])
        return 0.9*med + 0.1*base_arr
    y_ref = np.load('oof_targets_b4.npy') if os.path.exists('oof_targets_b4.npy') else np.load('oof_targets.npy')
    mask_ref = np.isfinite(blend_oof_ev) & np.isfinite(y_ref)
    th_boot = bootstrap_thresholds_ev(y_ref[mask_ref], blend_oof_ev[mask_ref], n_bootstrap=180)
    np.save('thresholds_blend_boot.npy', th_boot)
    cls = np.digitize(blend_test_ev, bins=[th_boot[0], th_boot[1], th_boot[2], th_boot[3]]).astype(int)
    sub = pd.DataFrame({'id_code': test_df['id_code'].values, 'diagnosis': cls})
    sub.to_csv('submission.csv', index=False)
    print('Submission with ordinal-enhanced NNLS blend written to submission.csv', flush=True)
else:
    print('Skipping submission write due to missing blend.', flush=True)

Starting B5@512 Ordinal head fine-tuning (5 folds)...



===== [B5 Ordinal] Fold 0 =====


/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b5_ns to current tf_efficientnet_b5.ns_jft_in1k.
  model = create_fn(


  it 50/329 tr_loss 1.0178 elapsed 11.7s


  it 100/329 tr_loss 0.9991 elapsed 23.2s


...

Bootstrap 90/180


Bootstrap 120/180


Bootstrap 150/180


Bootstrap 180/180


Submission with ordinal-enhanced NNLS blend written to submission.csv


In [26]:
# Pseudo-OOF fill for 3-fold models + L2 XGBoost stacker (EV-only) per expert plan
import os, gc, numpy as np, pandas as pd, torch
from torch.utils.data import DataLoader

print('Starting pseudo-OOF fill and L2 XGBoost stacker...', flush=True)

# 1) Build pseudo-OOF for 3-fold families by averaging their models over entire TRAIN (hflip-only TTA)
def hflip_tta_predict_models(models, dl):
    preds = []
    with torch.no_grad():
        for xb, _ in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                s = 0; c = 0
                for m in models:
                    m.eval()
                    p0 = m(xb); p1 = m(torch.flip(xb, dims=[-1]))
                    s += (p0 + p1) / 2.0; c += 1
                p = s / max(1,c)
            preds.append(p.detach().float().cpu().numpy())
    return np.concatenate(preds)

def infer_train_for_models(models):
    dtr = DRDataset(folds_df[['id_code']].copy(), IMG_DIR_TRAIN, transforms=valid_tfms)
    dl_tr_all = DataLoader(dtr, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
    return hflip_tta_predict_models(models, dl_tr_all).reshape(-1)

pseudo_cache = {}
if 'models_b4_640' in globals() and isinstance(models_b4_640, list) and len(models_b4_640) > 0:
    print('Generating pseudo-OOF for b4_640...', flush=True)
    pseudo_cache['b4_640'] = infer_train_for_models(models_b4_640)
    np.save('pseudo_oof_b4_640.npy', pseudo_cache['b4_640'])
else:
    print('models_b4_640 not in memory; will skip pseudo-OOF for b4_640 if missing.', flush=True)
if 'models_serx' in globals() and isinstance(models_serx, list) and len(models_serx) > 0:
    print('Generating pseudo-OOF for serx50_512_rrcema...', flush=True)
    pseudo_cache['serx50_512_rrcema'] = infer_train_for_models(models_serx)
    np.save('pseudo_oof_serx50_512_rrc_ema.npy', pseudo_cache['serx50_512_rrcema'])
else:
    print('models_serx not in memory; will skip pseudo-OOF for serx50.', flush=True)
if 'models_serx_s2' in globals() and isinstance(models_serx_s2, list) and len(models_serx_s2) > 0:
    print('Generating pseudo-OOF for serx50_512_rrcema_s2...', flush=True)
    pseudo_cache['serx50_512_rrcema_s2'] = infer_train_for_models(models_serx_s2)
    np.save('pseudo_oof_serx50_512_rrc_ema_seed2026.npy', pseudo_cache['serx50_512_rrcema_s2'])
else:
    print('models_serx_s2 not in memory; will skip pseudo-OOF for serx50 s2.', flush=True)

# 2) Load OOF arrays and fill NaNs (folds 3/4) with pseudo-OOF; if NaNs remain, DROP that feature for stability
oof_t = np.load('oof_targets_b4.npy') if os.path.exists('oof_targets_b4.npy') else np.load('oof_targets.npy')
feat_specs = [
    ('b4_512', 'oof_preds_b4.npy', None),
    ('b5_512', 'oof_preds.npy', None),
    ('b5_512_rrcema', 'oof_preds_b5_seed2025_rrc_ema.npy', None),
    ('b4_640', 'oof_preds_b4_640_rrc_ema.npy', 'pseudo_oof_b4_640.npy'),
    ('serx50_512_rrcema', 'oof_preds_serx50_512_rrc_ema.npy', 'pseudo_oof_serx50_512_rrc_ema.npy'),
    ('serx50_512_rrcema_s2', 'oof_preds_serx50_512_rrc_ema_seed2026.npy', 'pseudo_oof_serx50_512_rrc_ema_seed2026.npy'),
    ('b5_512_ordinal', 'oof_ev_b5_ordinal.npy', None),
]
X_cols = []
for name, oof_path, pseudo_path in feat_specs:
    if not os.path.exists(oof_path):
        print('Missing OOF for', name, '->', oof_path, 'skipping feature.', flush=True)
        continue
    arr = np.load(oof_path).reshape(-1)
    if np.any(~np.isfinite(arr)) and pseudo_path is not None and os.path.exists(pseudo_path):
        pseudo = np.load(pseudo_path).reshape(-1)
        mask_nan = ~np.isfinite(arr)
        arr[mask_nan] = pseudo[mask_nan]
    if np.any(~np.isfinite(arr)):
        print('Skipping feature due to remaining NaNs:', name, flush=True)
        continue
    X_cols.append((name, arr))

if len(X_cols) < 3:
    raise RuntimeError('Too few features collected for L2.')

X_oof = np.stack([c[1] for c in X_cols], axis=1).astype(np.float32)
names = [c[0] for c in X_cols]
mask_full = np.isfinite(X_oof).all(axis=1) & np.isfinite(oof_t)
print('L2 features:', names, 'X_oof shape:', X_oof.shape, 'full mask sum:', int(mask_full.sum()), flush=True)

# 3) Build test feature matrix in same order
test_map = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
    'b5_512_ordinal': 'test_ev_b5_ordinal.npy',
}
X_te_list = []
for n in names:
    p = test_map.get(n, None)
    if p is None or not os.path.exists(p):
        raise RuntimeError(f'Missing test preds for {n}: {p}')
    X_te_list.append(np.load(p).reshape(-1))
X_te = np.stack(X_te_list, axis=1).astype(np.float32)
print('Test feature shape:', X_te.shape, flush=True)

# 4) XGBoost GPU multiclass stacker, CV by original folds
try:
    import xgboost as xgb
except Exception as e:
    import sys, subprocess
    print('Installing xgboost...', flush=True)
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'xgboost'], check=False)
    import xgboost as xgb

params = dict(
    objective='multi:softprob', num_class=5,
    tree_method='gpu_hist', predictor='gpu_predictor',
    n_estimators=2000, learning_rate=0.05,
    max_depth=3, min_child_weight=2,
    subsample=0.8, colsample_bytree=0.8,
    reg_lambda=1.0, reg_alpha=0.0,
    random_state=42, n_jobs=-1
)

folds_ref = pd.read_csv('folds.csv')
X = X_oof[mask_full]
y = oof_t[mask_full].astype(int)
oof_proba = np.zeros((len(X_oof), 5), dtype=np.float32)
te_proba_acc = np.zeros((len(X_te), 5), dtype=np.float32)

for f in range(5):
    va_mask_idx = np.where((folds_ref['fold'].values == f) & mask_full)[0]
    tr_mask_idx = np.where((folds_ref['fold'].values != f) & mask_full)[0]
    if len(va_mask_idx) == 0 or len(tr_mask_idx) == 0:
        print(f'[L2-XGB] Skip fold {f} due to empty split.', flush=True);
        continue
    X_tr, y_tr = X_oof[tr_mask_idx], oof_t[tr_mask_idx].astype(int)
    X_va, y_va = X_oof[va_mask_idx], oof_t[va_mask_idx].astype(int)
    clf = xgb.XGBClassifier(**params)
    # Compatibility path: train without eval_set/early stopping due to version API differences
    clf.fit(X_tr, y_tr)
    oof_proba[va_mask_idx] = clf.predict_proba(X_va)
    te_proba_acc += clf.predict_proba(X_te)
    print(f'[L2-XGB] Fold {f} done; val size {len(va_mask_idx)}', flush=True)

te_proba = te_proba_acc / 5.0

# 5) Convert probs to EV, optimize/boot thresholds, write submission
classes = np.arange(5, dtype=np.float32).reshape(1,-1)
oof_ev = (oof_proba * classes).sum(axis=1)
te_ev = (te_proba * classes).sum(axis=1)

from sklearn.metrics import cohen_kappa_score
def preds_to_classes_ev(preds, th):
    return np.digitize(preds, bins=[th[0], th[1], th[2], th[3]])
from scipy.optimize import minimize
def optimize_thresholds_ev(y, p, init=[0.5,1.5,2.5,3.5]):
    y = np.asarray(y).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, -0.5, 4.5)
        cls = preds_to_classes_ev(p, th)
        return -cohen_kappa_score(y, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i]-th[i-1] < 0.1: th[i] = th[i-1] + 0.1
    return th

th_opt = optimize_thresholds_ev(oof_t[mask_full], oof_ev[mask_full])
oof_q = cohen_kappa_score(oof_t[mask_full], preds_to_classes_ev(oof_ev[mask_full], th_opt), weights='quadratic')
print('L2-XGB OOF QWK (opt):', f'{oof_q:.5f}', 'th:', th_opt, flush=True)

def bootstrap_thresholds_ev(y, p, n_bootstrap=200, base=[0.5,1.5,2.5,3.5]):
    N = len(y); cuts = []
    for i in range(n_bootstrap):
        idx = np.random.randint(0, N, size=N)
        th = optimize_thresholds_ev(y[idx], p[idx], init=base)
        cuts.append(th)
        if (i+1)%30==0: print(f'Bootstrap {i+1}/{n_bootstrap}', flush=True)
    cuts = np.array(cuts); med = np.median(cuts, axis=0); med = np.clip(med, 0.3, 3.7)
    for j in range(1,4):
        if med[j]-med[j-1] < 0.1: med[j] = med[j-1] + 0.1
    base_arr = np.array([0.5,1.5,2.5,3.5])
    return 0.9*med + 0.1*base_arr

th_boot = bootstrap_thresholds_ev(oof_t[mask_full], oof_ev[mask_full], n_bootstrap=200)
np.save('l2xgb_used_keys.npy', np.array(names, dtype=object))
np.save('l2xgb_oof_ev.npy', oof_ev); np.save('l2xgb_te_ev.npy', te_ev); np.save('l2xgb_thresholds_boot.npy', th_boot)

cls = preds_to_classes_ev(te_ev, th_boot).astype(int)
sub = pd.DataFrame({'id_code': test_df['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
print('L2-XGB submission.csv written.', flush=True)

Starting pseudo-OOF fill and L2 XGBoost stacker...


models_b4_640 not in memory; will skip pseudo-OOF for b4_640 if missing.


Generating pseudo-OOF for serx50_512_rrcema...


Generating pseudo-OOF for serx50_512_rrcema_s2...


Skipping feature due to remaining NaNs: b4_640


...

Bootstrap 90/200


Bootstrap 120/200


Bootstrap 150/200


Bootstrap 180/200


L2-XGB submission.csv written.


In [27]:
# Rebuild best NNLS blend (regression EVs only) and write submission
import os, numpy as np, pandas as pd
from scipy.optimize import nnls, minimize
from sklearn.metrics import cohen_kappa_score

print('Recomputing NNLS blend over regression EVs (no ordinal)...', flush=True)
oof_t = np.load('oof_targets_b4.npy') if os.path.exists('oof_targets_b4.npy') else np.load('oof_targets.npy')
paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
}
keys = []; arrs = []; mask = np.ones_like(oof_t, dtype=bool)
for k, p in paths_oof.items():
    if os.path.exists(p):
        a = np.load(p).reshape(-1); keys.append(k); arrs.append(a); mask &= np.isfinite(a)
    else:
        print('Missing OOF for', k, '->', p, flush=True)
if len(arrs) < 2: raise RuntimeError('Not enough OOF sources to blend')
X = np.concatenate([a.reshape(-1,1) for a in arrs], axis=1)
w, _ = nnls(X[mask], oof_t[mask].astype(float))
if w.sum() > 0: w = w / w.sum()
w = np.clip(w, 0.05, 0.7); w = w / w.sum()
blend_oof = (X * w.reshape(1,-1)).sum(axis=1)
def preds_to_classes_ev(preds, th):
    return np.digitize(preds, bins=[th[0], th[1], th[2], th[3]])
def optimize_thresholds_ev(y, p, init=[0.5,1.5,2.5,3.5]):
    y = np.asarray(y).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, -0.5, 4.5)
        cls = preds_to_classes_ev(p, th)
        return -cohen_kappa_score(y, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i]-th[i-1] < 0.1: th[i] = th[i-1] + 0.1
    return th
th_opt = optimize_thresholds_ev(oof_t[mask], blend_oof[mask])
oof_q = cohen_kappa_score(oof_t[mask], preds_to_classes_ev(blend_oof[mask], th_opt), weights='quadratic')
print('NNLS OOF QWK (opt on mask):', f'{oof_q:.5f}', 'weights:', w, 'keys:', keys, flush=True)

def bootstrap_thresholds_ev(y, p, n_bootstrap=180, base=[0.5,1.5,2.5,3.5]):
    N = len(y); cuts = []
    for i in range(n_bootstrap):
        idx = np.random.randint(0, N, size=N)
        th = optimize_thresholds_ev(y[idx], p[idx], init=base)
        cuts.append(th)
        if (i+1)%30==0: print(f'Bootstrap {i+1}/{n_bootstrap}', flush=True)
    cuts = np.array(cuts); med = np.median(cuts, axis=0); med = np.clip(med, 0.3, 3.7)
    for j in range(1,4):
        if med[j]-med[j-1] < 0.1: med[j] = med[j-1] + 0.1
    base_arr = np.array([0.5,1.5,2.5,3.5])
    return 0.9*med + 0.1*base_arr
th_boot = bootstrap_thresholds_ev(oof_t[mask], blend_oof[mask], n_bootstrap=180)
np.save('blend_weights_keys.npy', np.array(keys, dtype=object)); np.save('blend_weights_vals.npy', w); np.save('thresholds_blend_boot.npy', th_boot)

# Build test blend in same key order using standardized hflip-only preds
test_map = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
}
test_list = []; used_keys = []
for k in keys:
    p = test_map.get(k, None)
    if p and os.path.exists(p):
        test_list.append(np.load(p).reshape(-1)); used_keys.append(k)
    else:
        print('Missing test preds for', k, '->', p, flush=True)
if len(test_list) == 0: raise RuntimeError('No test preds available for NNLS blend')
w_use = w[:len(test_list)]
if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
w_use = w_use / w_use.sum()
blend_test = np.zeros_like(test_list[0], dtype=float)
for wi, arr in zip(w_use, test_list): blend_test += wi * arr
th = th_boot
cls = np.digitize(blend_test, bins=[th[0], th[1], th[2], th[3]]).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
print('Best-regression NNLS submission.csv written.', flush=True)

Recomputing NNLS blend over regression EVs (no ordinal)...


NNLS OOF QWK (opt on mask): 0.88151 weights: [0.12077895 0.17904818 0.31066866 0.11245652 0.22942863 0.04761905] keys: ['b4_512', 'b5_512', 'b5_512_rrcema', 'b4_640', 'serx50_512_rrcema', 'serx50_512_rrcema_s2']


Bootstrap 30/180


Bootstrap 60/180


Bootstrap 90/180


Bootstrap 120/180


Bootstrap 150/180


Bootstrap 180/180


Best-regression NNLS submission.csv written.


In [28]:
# ResNet200d @512 (3 folds) with RRC+EMA, then refit NNLS blend and write submission
import os, gc, copy, time, math, numpy as np, pandas as pd, torch, torch.nn as nn, albumentations as A, cv2
from torch.utils.data import DataLoader
from albumentations.pytorch import ToTensorV2
from timm.utils import ModelEmaV2
from scipy.optimize import nnls, minimize
from sklearn.metrics import cohen_kappa_score

print('Starting resnet200d@512 (3 folds) RRC+EMA...', flush=True)

MEAN = (0.485, 0.456, 0.406); STD = (0.229, 0.224, 0.225)
train_tfms_r200d = A.Compose([
    A.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.88, 1.0), ratio=(0.95, 1.05), interpolation=cv2.INTER_AREA),
    A.HorizontalFlip(p=0.5),
    A.Affine(scale=(0.95, 1.05), translate_percent=(0.0, 0.05), rotate=(-12, 12), border_mode=cv2.BORDER_REFLECT_101, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.7),
    A.RandomGamma(gamma_limit=(90,110), p=0.4),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])
valid_tfms_r200d = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

def get_loaders_r200d(tr_df, va_df, batch_size=8, num_workers=0):
    dtr = DRDataset(tr_df, IMG_DIR_TRAIN, transforms=train_tfms_r200d)
    dva = DRDataset(va_df, IMG_DIR_TRAIN, transforms=valid_tfms_r200d)
    dl_tr = DataLoader(dtr, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=True, persistent_workers=False)
    dl_va = DataLoader(dva, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=False)
    return dl_tr, dl_va

def validate_r200d(model, dl, loss_fn):
    model.eval(); preds=[]; targs=[]; val_loss=0.0; n=0
    with torch.no_grad():
        for xb, yb in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last); yb = yb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                out = model(xb); loss = loss_fn(out, yb)
            bs = xb.size(0); val_loss += loss.item()*bs; n += bs
            preds.append(out.detach().float().cpu().numpy()); targs.append(yb.detach().float().cpu().numpy())
    preds = np.concatenate(preds); targs = np.concatenate(targs)
    return val_loss/max(1,n), preds, targs

def train_one_fold_r200d_ema(fold, folds_df, epochs=12, lr=2e-4, wd=1e-5, batch_size=8, patience=2, ema_decay=0.9996):
    print(f"\n===== [resnet200d@512 RRC+EMA] Fold {fold} =====", flush=True)
    tr_df = folds_df[folds_df['fold'] != fold][['id_code','diagnosis']].reset_index(drop=True)
    va_df = folds_df[folds_df['fold'] == fold][['id_code','diagnosis']].reset_index(drop=True)
    dl_tr, dl_va = get_loaders_r200d(tr_df, va_df, batch_size=batch_size, num_workers=0)
    model = RegHeadModel(backbone_name='resnet200d', pretrained=True).to(device)
    model = model.to(memory_format=torch.channels_last)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    loss_fn = nn.HuberLoss(delta=1.0)
    scaler = torch.amp.GradScaler('cuda', enabled=True)
    ema = ModelEmaV2(model, decay=ema_decay, device=None)
    total_steps = epochs * len(dl_tr)
    def lr_lambda(step):
        if step < len(dl_tr): return (step+1)/len(dl_tr)
        prog = (step - len(dl_tr)) / max(1, total_steps - len(dl_tr))
        return 0.5*(1+math.cos(math.pi*prog))
    sch = torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda)
    best_loss = float('inf'); best_state=None; best_preds=None; best_targs=None; no_imp=0; accum = max(1, 16 // batch_size)
    opt.zero_grad(set_to_none=True)
    for ep in range(1, epochs+1):
        model.train(); tr_loss=0.0; n=0; t0=time.time()
        for it,(xb,yb) in enumerate(dl_tr):
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last); yb = yb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                out = model(xb); loss = loss_fn(out, yb)/accum
            scaler.scale(loss).backward()
            if (it+1)%accum==0:
                scaler.step(opt); scaler.update(); opt.zero_grad(set_to_none=True); sch.step(); ema.update(model)
            bs = xb.size(0); tr_loss += (loss.item()*accum)*bs; n += bs
            if (it+1)%50==0: print(f'Epoch {ep} it {it+1}/{len(dl_tr)} tr_loss {tr_loss/max(1,n):.4f} elapsed {time.time()-t0:.1f}s', flush=True)
        vloss, vpreds, vtargs = validate_r200d(model, dl_va, loss_fn)
        vq = cohen_kappa_score(vtargs, np.digitize(vpreds, bins=[0.5,1.5,2.5,3.5]), weights='quadratic')
        print(f'Epoch {ep}: tr_loss {tr_loss/max(1,n):.4f} val_loss {vloss:.4f} val_qwk(def) {vq:.4f} epoch_time {time.time()-t0:.1f}s', flush=True)
        if vloss < best_loss:
            best_loss = vloss; best_state = copy.deepcopy(ema.module.state_dict()); best_preds = vpreds.copy(); best_targs = vtargs.copy(); no_imp=0
        else:
            no_imp += 1
        if no_imp >= patience:
            print('Early stopping triggered', flush=True); break
    ema.module.load_state_dict(best_state)
    return ema.module, best_preds, best_targs

def run_cv_r200d_ema(folds_df, folds_to_run=(0,1,2), epochs=12, batch_size=8):
    folds_df = folds_df.reset_index(drop=True).copy()
    n = len(folds_df); oof_preds = np.full(n, np.nan, dtype=np.float32); oof_targs = folds_df['diagnosis'].values.astype(float)
    models_out = []
    for fold in folds_to_run:
        try:
            torch.cuda.empty_cache()
            if torch.cuda.is_available(): torch.cuda.reset_peak_memory_stats()
        except Exception: pass
        gc.collect()
        bs = batch_size
        for attempt in range(5):
            print(f'Attempt {attempt+1}: trying batch_size={bs}', flush=True)
            try:
                m, vpreds, vtargs = train_one_fold_r200d_ema(fold, folds_df, epochs=epochs, lr=2e-4, wd=1e-5, batch_size=bs, patience=2, ema_decay=0.9996)
                break
            except (RuntimeError, torch.cuda.OutOfMemoryError, torch.OutOfMemoryError) as e:
                if 'out of memory' in str(e).lower():
                    print(f'CUDA OOM on fold {fold} with batch_size={bs}. Reducing...', flush=True)
                    try: del m
                    except Exception: pass
                    gc.collect()
                    try: torch.cuda.empty_cache(); torch.cuda.reset_peak_memory_stats()
                    except Exception: pass
                    bs = max(2, bs-2); continue
                else:
                    raise
        models_out.append(m)
        va_idx = np.where(folds_df['fold'].values == fold)[0]
        oof_preds[va_idx] = vpreds
        fold_q = cohen_kappa_score(vtargs, np.digitize(vpreds, bins=[0.5,1.5,2.5,3.5]), weights='quadratic')
        print(f'[resnet200d@512] Fold {fold} val QWK (default th): {fold_q:.4f}', flush=True)
    np.save('oof_preds_r200d_512_rrc_ema.npy', oof_preds); np.save('oof_targets_r200d_512_rrc_ema.npy', oof_targs)
    return models_out, oof_preds, oof_targs

def nnls_blend_with_r200d():
    print('Refitting NNLS with resnet200d added...', flush=True)
    paths_oof = {
        'b4_512': 'oof_preds_b4.npy',
        'b5_512': 'oof_preds.npy',
        'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
        'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
        'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
        'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
        'r200d_512_rrcema': 'oof_preds_r200d_512_rrc_ema.npy',
    }
    y = np.load('oof_targets_b4.npy') if os.path.exists('oof_targets_b4.npy') else np.load('oof_targets.npy')
    keys=[]; arrs=[]; mask = np.isfinite(y).copy()
    for k,p in paths_oof.items():
        if os.path.exists(p):
            a = np.load(p).reshape(-1); keys.append(k); arrs.append(a); mask &= np.isfinite(a)
        else:
            print('Missing OOF for', k, '->', p, flush=True)
    if len(arrs) < 3:
        print('Too few arrays for NNLS. Abort blend.', flush=True); return None, None, None, None, None
    X = np.concatenate([a.reshape(-1,1) for a in arrs], axis=1)
    w, _ = nnls(X[mask], y[mask].astype(float))
    if w.sum() > 0: w = w / w.sum()
    w = np.clip(w, 0.05, 0.7); w = w / w.sum()
    blend_oof = (X * w.reshape(1,-1)).sum(axis=1)
    def preds_to_classes_ev(preds, th):
        return np.digitize(preds, bins=[th[0], th[1], th[2], th[3]])
    def optimize_thresholds_ev(y, p, init=[0.5,1.5,2.5,3.5]):
        y = np.asarray(y).astype(float); p = np.asarray(p).astype(float)
        def _loss(th):
            th = np.sort(th); th = np.clip(th, -0.5, 4.5)
            cls = preds_to_classes_ev(p, th)
            return -cohen_kappa_score(y, cls, weights='quadratic')
        res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000, 'xatol':1e-3, 'fatol':1e-3})
        th = np.sort(res.x)
        for i in range(1,4):
            if th[i]-th[i-1] < 0.1: th[i] = th[i-1] + 0.1
        return th
    th_opt = optimize_thresholds_ev(y[mask], blend_oof[mask])
    oof_q = cohen_kappa_score(y[mask], preds_to_classes_ev(blend_oof[mask], th_opt), weights='quadratic')
    print('NNLS (with r200d) OOF QWK:', f'{oof_q:.5f}', 'weights:', w, 'keys:', keys, flush=True)
    # Build test blend
    test_map = {
        'b4_512': 'test_reg_preds_b4_hflip.npy',
        'b5_512': 'test_reg_preds_b5_hflip.npy',
        'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
        'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
        'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
        'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
        'r200d_512_rrcema': 'test_reg_preds_r200d_512_rrc_ema.npy',
    }
    test_list=[]; used_keys=[]
    for k in keys:
        p = test_map.get(k, None)
        if p and os.path.exists(p): test_list.append(np.load(p).reshape(-1)); used_keys.append(k)
        else: print('Missing test preds for', k, '->', p, flush=True)
    if len(test_list) == 0:
        print('No test preds to blend.', flush=True); return None, None, None, None, None
    w_use = w[:len(test_list)]
    if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
    w_use = w_use / w_use.sum()
    blend_test = np.zeros_like(test_list[0], dtype=float)
    for wi, arr in zip(w_use, test_list): blend_test += wi * arr
    return blend_oof, blend_test, w, th_opt, keys

def tta_predict_hflip_mean(models, dl):
    preds = [];
    with torch.no_grad():
        for xb, _ in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                s = 0; c = 0
                for m in models:
                    m.eval(); p0 = m(xb); p1 = m(torch.flip(xb, dims=[-1]));
                    s += (p0 + p1)/2.0; c += 1
                p = s / max(1,c)
            preds.append(p.detach().float().cpu().numpy())
    return np.concatenate(preds)

# Execute: train resnet200d 3-folds, infer test, then NNLS-blend
seed_everything(1337)
models_r200d, oof_r200d, oof_t_r200d = run_cv_r200d_ema(folds_df, folds_to_run=(0,1,2), epochs=12, batch_size=8)
dte_r200d = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms_r200d)
dl_te_r200d = DataLoader(dte_r200d, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
preds_r200d = tta_predict_hflip_mean(models_r200d, dl_te_r200d)
np.save('test_reg_preds_r200d_512_rrc_ema.npy', preds_r200d)
blend_oof_r200d, blend_test_r200d, w_r200d, th_r200d, keys_r200d = nnls_blend_with_r200d()
if blend_oof_r200d is not None:
    # Bootstrap thresholds for final stability
    def bootstrap_thresholds_ev(y, p, n_bootstrap=180, base=[0.5,1.5,2.5,3.5]):
        N = len(y); cuts=[]
        for i in range(n_bootstrap):
            idx = np.random.randint(0, N, size=N)
            # Reuse optimizer above
            def opt_th(yy, pp, init=base):
                def _loss(th):
                    th = np.sort(th); th = np.clip(th, -0.5, 4.5)
                    cls = np.digitize(pp, bins=[th[0], th[1], th[2], th[3]])
                    return -cohen_kappa_score(yy, cls, weights='quadratic')
                res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':1200, 'xatol':1e-3, 'fatol':1e-3})
                th = np.sort(res.x)
                for j in range(1,4):
                    if th[j]-th[j-1] < 0.1: th[j] = th[j-1] + 0.1
                return th
            thb = opt_th(oof_t_r200d[np.isfinite(blend_oof_r200d)], blend_oof_r200d[np.isfinite(blend_oof_r200d)], init=base)
            cuts.append(thb)
            if (i+1)%30==0: print(f'Bootstrap {i+1}/{n_bootstrap}', flush=True)
        cuts = np.array(cuts); med = np.median(cuts, axis=0); med = np.clip(med, 0.3, 3.7)
        for j in range(1,4):
            if med[j]-med[j-1] < 0.1: med[j] = med[j-1] + 0.1
        base_arr = np.array([0.5,1.5,2.5,3.5])
        return 0.9*med + 0.1*base_arr
    th_boot = bootstrap_thresholds_ev(oof_t_r200d, blend_oof_r200d, n_bootstrap=180)
    cls = np.digitize(blend_test_r200d, bins=[th_boot[0], th_boot[1], th_boot[2], th_boot[3]]).astype(int)
    sub = pd.DataFrame({'id_code': test_df['id_code'].values, 'diagnosis': cls})
    sub.to_csv('submission.csv', index=False)
    print('Submission with resnet200d-enhanced NNLS blend written to submission.csv', flush=True)
else:
    print('Skipping submission due to missing blend inputs.', flush=True)

Starting resnet200d@512 (3 folds) RRC+EMA...


Attempt 1: trying batch_size=8



===== [resnet200d@512 RRC+EMA] Fold 0 =====


Epoch 1 it 50/329 tr_loss 0.7889 elapsed 33.8s


Epoch 1 it 100/329 tr_loss 0.7351 elapsed 66.6s


...

Bootstrap 90/180


Bootstrap 120/180


Bootstrap 150/180


Bootstrap 180/180


Submission with resnet200d-enhanced NNLS blend written to submission.csv


In [29]:
# ConvNeXt-Base @512 (3 folds, RRC+EMA), z-scored NNLS blend with caps and robust thresholds
import os, gc, copy, time, math, numpy as np, pandas as pd, torch, torch.nn as nn, albumentations as A, cv2
from torch.utils.data import DataLoader
from albumentations.pytorch import ToTensorV2
from timm.utils import ModelEmaV2
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import nnls, minimize

print('Starting ConvNeXt-Base@512 (3 folds) RRC+EMA...', flush=True)

SEED_CNX = 2027
seed_everything(SEED_CNX)
BACKBONE_CNX = 'convnext_base.fb_in22k_ft_in1k'
EPOCHS_CNX = 12
PATIENCE_CNX = 3
BATCH_CNX = 8
LR_CNX = 2e-4
WD_CNX = 1e-5

MEAN = (0.485, 0.456, 0.406); STD = (0.229, 0.224, 0.225)
train_tfms_cnx = A.Compose([
    A.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.88, 1.0), ratio=(0.95, 1.05), interpolation=cv2.INTER_AREA),
    A.HorizontalFlip(p=0.5),
    A.Affine(scale=(0.95, 1.05), translate_percent=(0.0, 0.06), rotate=(-12, 12), border_mode=cv2.BORDER_REFLECT_101, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.7),
    A.RandomGamma(gamma_limit=(90,110), p=0.4),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])
valid_tfms_cnx = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

def get_loaders_cnx(tr_df, va_df, batch_size=8, num_workers=0):
    dtr = DRDataset(tr_df, IMG_DIR_TRAIN, transforms=train_tfms_cnx)
    dva = DRDataset(va_df, IMG_DIR_TRAIN, transforms=valid_tfms_cnx)
    dl_tr = DataLoader(dtr, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=True, persistent_workers=False)
    dl_va = DataLoader(dva, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=False)
    return dl_tr, dl_va

def validate_loss_preds(model, dl, loss_fn):
    model.eval(); preds=[]; targs=[]; vloss=0.0; n=0
    with torch.no_grad():
        for xb, yb in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last); yb = yb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                out = model(xb); loss = loss_fn(out, yb)
            bs = xb.size(0); vloss += loss.item()*bs; n += bs
            preds.append(out.detach().float().cpu().numpy()); targs.append(yb.detach().float().cpu().numpy())
    preds = np.concatenate(preds); targs = np.concatenate(targs)
    return vloss/max(1,n), preds, targs

def train_one_fold_cnx_ema(fold, folds_df, epochs=12, lr=2e-4, wd=1e-5, batch_size=8, patience=3, ema_decay=0.9996):
    print(f"\n===== [ConvNeXtB@512 RRC+EMA] Fold {fold} =====", flush=True)
    tr_df = folds_df[folds_df['fold'] != fold][['id_code','diagnosis']].reset_index(drop=True)
    va_df = folds_df[folds_df['fold'] == fold][['id_code','diagnosis']].reset_index(drop=True)
    dl_tr, dl_va = get_loaders_cnx(tr_df, va_df, batch_size=batch_size, num_workers=0)
    model = RegHeadModel(backbone_name=BACKBONE_CNX, pretrained=True).to(device)
    model = model.to(memory_format=torch.channels_last)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    loss_fn = nn.HuberLoss(delta=1.0)
    scaler = torch.amp.GradScaler('cuda', enabled=True)
    ema = ModelEmaV2(model, decay=ema_decay, device=None)
    total_steps = epochs * len(dl_tr)
    def lr_lambda(step):
        if step < len(dl_tr): return (step+1)/len(dl_tr)
        prog = (step - len(dl_tr)) / max(1, total_steps - len(dl_tr))
        return 0.5*(1+math.cos(math.pi*prog))
    sch = torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda)
    best_loss = float('inf'); best_state=None; best_preds=None; best_targs=None; no_imp=0
    accum = max(1, 16 // batch_size); opt.zero_grad(set_to_none=True)
    for ep in range(1, epochs+1):
        model.train(); tr_loss=0.0; n=0; t0=time.time()
        for it,(xb,yb) in enumerate(dl_tr):
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last); yb = yb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                out = model(xb); loss = loss_fn(out, yb)/accum
            scaler.scale(loss).backward()
            if (it+1)%accum==0:
                scaler.step(opt); scaler.update(); opt.zero_grad(set_to_none=True); sch.step(); ema.update(model)
            bs = xb.size(0); tr_loss += (loss.item()*accum)*bs; n += bs
            if (it+1)%50==0: print(f'Epoch {ep} it {it+1}/{len(dl_tr)} tr_loss {tr_loss/max(1,n):.4f} elapsed {time.time()-t0:.1f}s', flush=True)
        # validate EMA model (save/validate EMA)
        vloss, vpreds, vtargs = validate_loss_preds(ema.module, dl_va, loss_fn)
        vq = cohen_kappa_score(vtargs, np.digitize(vpreds, bins=[0.5,1.5,2.5,3.5]), weights='quadratic')
        print(f'Epoch {ep}: tr_loss {tr_loss/max(1,n):.4f} val_loss {vloss:.4f} val_qwk(def) {vq:.4f} epoch_time {time.time()-t0:.1f}s', flush=True)
        if vloss < best_loss:
            best_loss = vloss; best_state = copy.deepcopy(ema.module.state_dict()); best_preds = vpreds.copy(); best_targs = vtargs.copy(); no_imp=0
        else:
            no_imp += 1
        if no_imp >= patience:
            print('Early stopping triggered', flush=True); break
    ema.module.load_state_dict(best_state)
    return ema.module, best_preds, best_targs

def run_cv_cnx_ema(folds_df, folds_to_run=(0,1,2), epochs=12, batch_size=8):
    folds_df = folds_df.reset_index(drop=True).copy()
    n = len(folds_df); oof_preds = np.full(n, np.nan, dtype=np.float32); oof_targs = folds_df['diagnosis'].values.astype(float)
    models_out = []
    for fold in folds_to_run:
        try:
            torch.cuda.empty_cache()
            if torch.cuda.is_available(): torch.cuda.reset_peak_memory_stats()
        except Exception: pass
        gc.collect()
        bs = batch_size
        for attempt in range(4):
            print(f'Attempt {attempt+1}: trying batch_size={bs}', flush=True)
            try:
                m, vpreds, vtargs = train_one_fold_cnx_ema(fold, folds_df, epochs=epochs, lr=LR_CNX, wd=WD_CNX, batch_size=bs, patience=PATIENCE_CNX, ema_decay=0.9996)
                break
            except (RuntimeError, torch.cuda.OutOfMemoryError, torch.OutOfMemoryError) as e:
                if 'out of memory' in str(e).lower():
                    print(f'CUDA OOM on fold {fold} with batch_size={bs}. Reducing...', flush=True)
                    try: del m
                    except Exception: pass
                    gc.collect()
                    try: torch.cuda.empty_cache(); torch.cuda.reset_peak_memory_stats()
                    except Exception: pass
                    bs = max(2, bs-2); continue
                else:
                    raise
        models_out.append(m)
        va_idx = np.where(folds_df['fold'].values == fold)[0]
        oof_preds[va_idx] = vpreds
        fold_q = cohen_kappa_score(vtargs, np.digitize(vpreds, bins=[0.5,1.5,2.5,3.5]), weights='quadratic')
        print(f'[ConvNeXtB@512] Fold {fold} val QWK (default th): {fold_q:.4f}', flush=True)
    np.save('oof_preds_convnextb_512_rrc_ema.npy', oof_preds); np.save('oof_targets_convnextb_512_rrc_ema.npy', oof_targs)
    return models_out, oof_preds, oof_targs

def hflip_tta_predict_mean(models, dl):
    preds = []
    with torch.no_grad():
        for xb, _ in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                s = 0; c = 0
                for m in models:
                    m.eval(); p0 = m(xb); p1 = m(torch.flip(xb, dims=[-1]))
                    s += (p0 + p1) / 2.0; c += 1
                p = s / max(1, c)
            preds.append(p.detach().float().cpu().numpy())
    return np.concatenate(preds)

def zscore_fit_transform(arr_list, mask):
    stats = []
    z_list = []
    for a in arr_list:
        a_m = a[mask]
        mu = float(np.mean(a_m))
        sd = float(np.std(a_m) + 1e-6)
        z = (a - mu) / sd
        stats.append((mu, sd))
        z_list.append(z)
    return z_list, stats

def apply_zscore(arr_list, stats):
    z_list = []
    for a, (mu, sd) in zip(arr_list, stats):
        z_list.append((a - mu) / (sd + 1e-6))
    return z_list

def optimize_thresholds_generic(y, p, init=[0.5,1.5,2.5,3.5]):
    y = np.asarray(y).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, 0.3, 3.7)
        cls = np.digitize(p, bins=[th[0], th[1], th[2], th[3]])
        return -cohen_kappa_score(y, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i]-th[i-1] < 0.12: th[i] = th[i-1] + 0.12
    th = np.clip(th, 0.3, 3.7)
    return th

def bootstrap_thresholds_generic(y, p, n_bootstrap=200, base=[0.5,1.5,2.5,3.5]):
    N = len(y); cuts = []
    for i in range(n_bootstrap):
        idx = np.random.randint(0, N, size=N)
        th = optimize_thresholds_generic(y[idx], p[idx], init=base)
        cuts.append(th)
        if (i+1)%40==0: print(f'Bootstrap {i+1}/{n_bootstrap}', flush=True)
    cuts = np.array(cuts); med = np.median(cuts, axis=0); med = np.clip(med, 0.3, 3.7)
    for j in range(1,4):
        if med[j]-med[j-1] < 0.12: med[j] = med[j-1] + 0.12
    base_arr = np.array([0.5,1.5,2.5,3.5])
    return 0.9*med + 0.1*base_arr

def nnls_with_caps_zscore_and_serx_cap(y, oof_map, test_map, keys, lo=0.05, hi=0.70, serx_pair=('serx50_512_rrcema','serx50_512_rrcema_s2'), serx_cap_sum=0.30):
    arrs = [oof_map[k] for k in keys]
    mask = np.ones_like(y, dtype=bool)
    for a in arrs: mask &= np.isfinite(a)
    z_arrs, stats = zscore_fit_transform(arrs, mask)
    X = np.concatenate([z.reshape(-1,1) for z in z_arrs], axis=1)
    w, _ = nnls(X[mask], y[mask].astype(float))
    if w.sum() > 0: w = w / w.sum()
    w = np.clip(w, lo, hi); w = w / w.sum()
    # serx correlated sum cap
    if serx_pair[0] in keys and serx_pair[1] in keys:
        i0 = keys.index(serx_pair[0]); i1 = keys.index(serx_pair[1])
        s = w[i0] + w[i1]
        if s > serx_cap_sum:
            scale = serx_cap_sum / s
            w[i0] *= scale; w[i1] *= scale
            rem = 1.0 - (w[i0] + w[i1])
            others = [i for i in range(len(w)) if i not in (i0,i1)]
            w_others = w[others]; s_others = w_others.sum() + 1e-9
            w[others] = w_others / s_others * rem
    # build blended OOF
    blend_oof = (X * w.reshape(1,-1)).sum(axis=1)
    # build blended TEST using same z-score stats
    te_arrs = [test_map[k] for k in keys]
    te_z = apply_zscore(te_arrs, stats)
    te_mat = np.concatenate([z.reshape(-1,1) for z in te_z], axis=1)
    blend_te = (te_mat * w.reshape(1,-1)).sum(axis=1)
    return blend_oof, blend_te, w, mask

# 1) Train ConvNeXtB 3-folds
models_cnx, oof_cnx, oof_t_cnx = run_cv_cnx_ema(folds_df, folds_to_run=(0,1,2), epochs=EPOCHS_CNX, batch_size=BATCH_CNX)

# 2) Inference on test (hflip-only) for ConvNeXtB
dte_cnx = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms_cnx)
dl_te_cnx = DataLoader(dte_cnx, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
preds_cnx = hflip_tta_predict_mean(models_cnx, dl_te_cnx).reshape(-1)
np.save('test_reg_preds_convnextb_512_rrc_ema.npy', preds_cnx)

# 3) Refit NNLS on z-scored OOF EVs with caps, include ConvNeXtB
oof_paths = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'oof_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'oof_preds_convnextb_512_rrc_ema.npy',
}
test_paths = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'test_reg_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'test_reg_preds_convnextb_512_rrc_ema.npy',
}
y_ref = None
for tgt_path in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt_path):
        y_ref = np.load(tgt_path).reshape(-1); break
if y_ref is None:
    y_ref = oof_t_cnx.reshape(-1)

loaded_keys = []; oof_map = {}; te_map = {}
for k, p in oof_paths.items():
    if os.path.exists(p):
        a = np.load(p).reshape(-1);
        oof_map[k] = a; loaded_keys.append(k)
for k, p in test_paths.items():
    if os.path.exists(p):
        te_map[k] = np.load(p).reshape(-1)

keys_use = [k for k in loaded_keys if k in te_map]
if len(keys_use) < 3:
    print('Too few models available for NNLS; aborting blend.', flush=True)
else:
    blend_oof, blend_te, w, mask_nnls = nnls_with_caps_zscore_and_serx_cap(y_ref, oof_map, te_map, keys_use, lo=0.05, hi=0.70, serx_cap_sum=0.30)
    th = optimize_thresholds_generic(y_ref[mask_nnls], blend_oof[mask_nnls])
    oof_q = cohen_kappa_score(y_ref[mask_nnls], np.digitize(blend_oof[mask_nnls], bins=[th[0], th[1], th[2], th[3]]), weights='quadratic')
    print('Z-scored NNLS OOF QWK:', f'{oof_q:.5f}', 'weights:', w, 'keys:', keys_use, flush=True)
    th_boot = bootstrap_thresholds_generic(y_ref[mask_nnls], blend_oof[mask_nnls], n_bootstrap=200, base=th.tolist())
    # Optional tiny smoothing: clip EV to [0,4]
    blend_te = np.clip(blend_te, 0.0, 4.0)
    cls = np.digitize(blend_te, bins=[th_boot[0], th_boot[1], th_boot[2], th_boot[3]]).astype(int)
    sub = pd.DataFrame({'id_code': test_df['id_code'].values, 'diagnosis': cls})
    sub.to_csv('submission.csv', index=False)
    np.save('blend_weights_keys.npy', np.array(keys_use, dtype=object)); np.save('blend_weights_vals.npy', w); np.save('thresholds_blend_boot.npy', th_boot)
    print('Final submission.csv written with ConvNeXtB included.', flush=True)

Starting ConvNeXt-Base@512 (3 folds) RRC+EMA...


Attempt 1: trying batch_size=8



===== [ConvNeXtB@512 RRC+EMA] Fold 0 =====


/app/.pip-target/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 1 it 50/329 tr_loss 0.6565 elapsed 33.5s


...

Bootstrap 80/200


Bootstrap 120/200


Bootstrap 160/200


Bootstrap 200/200


Final submission.csv written with ConvNeXtB included.


In [30]:
# Refit LOFO-averaged NNLS (caps + serx cap) incl. ConvNeXt; bootstrap thresholds; write submission
import os, numpy as np, pandas as pd
from scipy.optimize import nnls, minimize
from sklearn.metrics import cohen_kappa_score

print('Refitting LOFO-averaged NNLS with caps and serx-pair cap (incl. ConvNeXt)...', flush=True)

paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'oof_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'oof_preds_convnextb_512_rrc_ema.npy',
}
paths_te = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'test_reg_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'test_reg_preds_convnextb_512_rrc_ema.npy',
}
y = None
for tgt in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt):
        y = np.load(tgt).reshape(-1); break
if y is None:
    raise RuntimeError('OOF targets not found')

loaded_keys = []; arrs = [];
for k,p in paths_oof.items():
    if os.path.exists(p):
        a = np.load(p).reshape(-1); loaded_keys.append(k); arrs.append(a)
    
if len(arrs) < 3:
    raise RuntimeError('Too few OOF arrays for NNLS')

X_full = np.concatenate([a.reshape(-1,1) for a in arrs], axis=1)
mask = np.isfinite(y).copy()
for a in arrs: mask &= np.isfinite(a)

def fit_nnls_masked(y, X, mask, lo=0.05, hi=0.70, serx_pair=('serx50_512_rrcema','serx50_512_rrcema_s2'), serx_cap_sum=0.30, keys=None):
    w, _ = nnls(X[mask], y[mask].astype(float))
    if w.sum() > 0: w = w / w.sum()
    w = np.clip(w, lo, hi); w = w / w.sum()
    # serx-pair sum cap
    if keys is not None and all(k in keys for k in serx_pair):
        i0 = keys.index(serx_pair[0]); i1 = keys.index(serx_pair[1])
        s = w[i0] + w[i1]
        if s > serx_cap_sum:
            scale = serx_cap_sum / s
            w[i0] *= scale; w[i1] *= scale
            rem = 1.0 - (w[i0] + w[i1])
            idx_others = [i for i in range(len(w)) if i not in (i0,i1)]
            s_others = w[idx_others].sum() + 1e-9
            w[idx_others] = w[idx_others] / s_others * rem
    return w

# LOFO-averaged NNLS
keys_all = loaded_keys.copy()
weights_list = []
w_all = fit_nnls_masked(y, X_full, mask, lo=0.05, hi=0.70, keys=keys_all)
weights_list.append(w_all)
for i in range(len(keys_all)):
    keep_idx = [j for j in range(len(keys_all)) if j != i]
    X_lofo = X_full[:, keep_idx]
    keys_lofo = [keys_all[j] for j in keep_idx]
    w_lofo = fit_nnls_masked(y, X_lofo, mask, lo=0.05, hi=0.70, keys=keys_lofo)
    # map back to full length with zero for left-out index
    w_full = np.zeros(len(keys_all), dtype=float)
    ptr = 0
    for j in range(len(keys_all)):
        if j == i: continue
        w_full[j] = w_lofo[ptr]; ptr += 1
    # renormalize to 1
    if w_full.sum() > 0: w_full /= w_full.sum()
    weights_list.append(w_full)
w_avg = np.mean(np.stack(weights_list, axis=0), axis=0)
w_avg = np.clip(w_avg, 0.05, 0.70); w_avg = w_avg / w_avg.sum()

# Build blended OOF with averaged weights
blend_oof = (X_full * w_avg.reshape(1,-1)).sum(axis=1)

def optimize_thresholds_generic(y, p, init=[0.5,1.5,2.5,3.5]):
    y = np.asarray(y).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, 0.3, 3.7)
        cls = np.digitize(p, bins=[th[0], th[1], th[2], th[3]])
        return -cohen_kappa_score(y, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i]-th[i-1] < 0.12: th[i] = th[i-1] + 0.12
    th = np.clip(th, 0.3, 3.7)
    return th

th = optimize_thresholds_generic(y[mask], blend_oof[mask])
oof_q = cohen_kappa_score(y[mask], np.digitize(blend_oof[mask], bins=[th[0], th[1], th[2], th[3]]), weights='quadratic')
print('LOFO-avg NNLS OOF QWK:', f'{oof_q:.5f}', 'weights:', w_avg, 'keys:', keys_all, flush=True)

def bootstrap_thresholds_generic(y, p, n_bootstrap=200, base=None):
    if base is None: base = [0.5,1.5,2.5,3.5]
    N = len(y); cuts = []
    for i in range(n_bootstrap):
        idx = np.random.randint(0, N, size=N)
        thb = optimize_thresholds_generic(y[idx], p[idx], init=base)
        cuts.append(thb)
        if (i+1)%40==0: print(f'Bootstrap {i+1}/{n_bootstrap}', flush=True)
    cuts = np.array(cuts); med = np.median(cuts, axis=0); med = np.clip(med, 0.3, 3.7)
    for j in range(1,4):
        if med[j]-med[j-1] < 0.12: med[j] = med[j-1] + 0.12
    base_arr = np.array([0.5,1.5,2.5,3.5])
    return 0.9*med + 0.1*base_arr

th_boot = bootstrap_thresholds_generic(y[mask], blend_oof[mask], n_bootstrap=200, base=th.tolist())
np.save('blend_weights_keys.npy', np.array(keys_all, dtype=object))
np.save('blend_weights_vals.npy', w_avg)
np.save('thresholds_blend_boot.npy', th_boot)

# Build blended TEST with same key order; use available preds only
test_list = []; used_keys = []
for k in keys_all:
    p = paths_te.get(k, None)
    if p and os.path.exists(p):
        test_list.append(np.load(p).reshape(-1)); used_keys.append(k)
    else:
        print('Missing test preds for', k, '->', p, flush=True)
if len(test_list) == 0:
    raise RuntimeError('No test preds found for final blend')
w_use = w_avg[:len(test_list)]
if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
w_use = w_use / w_use.sum()
blend_te = np.zeros_like(test_list[0], dtype=float)
for wi, arr in zip(w_use, test_list):
    blend_te += wi * arr
# optional EV clipping
blend_te = np.clip(blend_te, 0.0, 4.0)
cls = np.digitize(blend_te, bins=[th_boot[0], th_boot[1], th_boot[2], th_boot[3]]).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
print('Submission.csv written with LOFO-averaged NNLS blend (incl. ConvNeXt).', flush=True)

Refitting LOFO-averaged NNLS with caps and serx-pair cap (incl. ConvNeXt)...


LOFO-avg NNLS OOF QWK: 0.88284 weights: [0.11385803 0.15888844 0.26617223 0.10379826 0.18198154 0.05618721
 0.06956592 0.04954837] keys: ['b4_512', 'b5_512', 'b5_512_rrcema', 'b4_640', 'serx50_512_rrcema', 'serx50_512_rrcema_s2', 'r200d_512_rrcema', 'convnextb_512_rrcema']


Bootstrap 40/200


Bootstrap 80/200


Bootstrap 120/200


Bootstrap 160/200


Bootstrap 200/200


Submission.csv written with LOFO-averaged NNLS blend (incl. ConvNeXt).


In [31]:
# SEResNeXt101_32x8d @512 (3 folds) with RRC+EMA, then LOFO-avg NNLS (caps + serx pair cap), bootstrap thresholds, write submission
import os, gc, copy, time, math, numpy as np, pandas as pd, torch, torch.nn as nn, albumentations as A, cv2
from torch.utils.data import DataLoader
from albumentations.pytorch import ToTensorV2
from timm.utils import ModelEmaV2
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import nnls, minimize

print('Starting SEResNeXt101_32x8d@512 (3 folds) RRC+EMA...', flush=True)

MEAN = (0.485, 0.456, 0.406); STD = (0.229, 0.224, 0.225)
train_tfms_s101 = A.Compose([
    A.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.90, 1.0), ratio=(0.95, 1.05), interpolation=cv2.INTER_AREA),
    A.HorizontalFlip(p=0.5),
    A.Affine(scale=(0.95, 1.05), translate_percent=(0.0, 0.05), rotate=(-12, 12), border_mode=cv2.BORDER_REFLECT_101, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.7),
    A.RandomGamma(gamma_limit=(90,110), p=0.4),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])
valid_tfms_s101 = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

def get_loaders_s101(tr_df, va_df, batch_size=8, num_workers=0):
    dtr = DRDataset(tr_df, IMG_DIR_TRAIN, transforms=train_tfms_s101)
    dva = DRDataset(va_df, IMG_DIR_TRAIN, transforms=valid_tfms_s101)
    dl_tr = DataLoader(dtr, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=True, persistent_workers=False)
    dl_va = DataLoader(dva, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=False)
    return dl_tr, dl_va

def validate_s101(model, dl, loss_fn):
    model.eval(); preds=[]; targs=[]; val_loss=0.0; n=0
    with torch.no_grad():
        for xb, yb in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last); yb = yb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                out = model(xb); loss = loss_fn(out, yb)
            bs = xb.size(0); val_loss += loss.item()*bs; n += bs
            preds.append(out.detach().float().cpu().numpy()); targs.append(yb.detach().float().cpu().numpy())
    preds = np.concatenate(preds); targs = np.concatenate(targs)
    return val_loss/max(1,n), preds, targs

def train_one_fold_s101_ema(fold, folds_df, epochs=12, lr=2e-4, wd=1e-5, batch_size=8, patience=3, ema_decay=0.9996):
    print(f"\n===== [SEResNeXt101@512 RRC+EMA] Fold {fold} =====", flush=True)
    tr_df = folds_df[folds_df['fold'] != fold][['id_code','diagnosis']].reset_index(drop=True)
    va_df = folds_df[folds_df['fold'] == fold][['id_code','diagnosis']].reset_index(drop=True)
    dl_tr, dl_va = get_loaders_s101(tr_df, va_df, batch_size=batch_size, num_workers=0)
    model = RegHeadModel(backbone_name='seresnext101_32x8d', pretrained=True).to(device)
    model = model.to(memory_format=torch.channels_last)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    loss_fn = nn.HuberLoss(delta=1.0)
    scaler = torch.amp.GradScaler('cuda', enabled=True)
    ema = ModelEmaV2(model, decay=ema_decay, device=None)
    total_steps = epochs * len(dl_tr)
    def lr_lambda(step):
        if step < len(dl_tr): return (step+1)/len(dl_tr)
        prog = (step - len(dl_tr)) / max(1, total_steps - len(dl_tr))
        return 0.5*(1+math.cos(math.pi*prog))
    sch = torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda)
    best_loss = float('inf'); best_state=None; best_preds=None; best_targs=None; no_imp=0; accum = max(1, 16 // batch_size)
    opt.zero_grad(set_to_none=True)
    for ep in range(1, epochs+1):
        model.train(); tr_loss=0.0; n=0; t0=time.time()
        for it,(xb,yb) in enumerate(dl_tr):
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last); yb = yb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                out = model(xb); loss = loss_fn(out, yb)/accum
            scaler.scale(loss).backward()
            if (it+1)%accum==0:
                scaler.step(opt); scaler.update(); opt.zero_grad(set_to_none=True); sch.step(); ema.update(model)
            bs = xb.size(0); tr_loss += (loss.item()*accum)*bs; n += bs
            if (it+1)%50==0: print(f'Epoch {ep} it {it+1}/{len(dl_tr)} tr_loss {tr_loss/max(1,n):.4f} elapsed {time.time()-t0:.1f}s', flush=True)
        vloss, vpreds, vtargs = validate_s101(ema.module, dl_va, loss_fn)
        vq = cohen_kappa_score(vtargs, np.digitize(vpreds, bins=[0.5,1.5,2.5,3.5]), weights='quadratic')
        print(f'Epoch {ep}: tr_loss {tr_loss/max(1,n):.4f} val_loss {vloss:.4f} val_qwk(def) {vq:.4f} epoch_time {time.time()-t0:.1f}s', flush=True)
        if vloss < best_loss:
            best_loss = vloss; best_state = copy.deepcopy(ema.module.state_dict()); best_preds = vpreds.copy(); best_targs = vtargs.copy(); no_imp=0
        else:
            no_imp += 1
        if no_imp >= patience:
            print('Early stopping triggered', flush=True); break
    ema.module.load_state_dict(best_state)
    return ema.module, best_preds, best_targs

def run_cv_s101_ema(folds_df, folds_to_run=(0,1,2), epochs=12, batch_size=8):
    folds_df = folds_df.reset_index(drop=True).copy()
    n = len(folds_df); oof_preds = np.full(n, np.nan, dtype=np.float32); oof_targs = folds_df['diagnosis'].values.astype(float)
    models_out = []
    for fold in folds_to_run:
        try:
            torch.cuda.empty_cache()
            if torch.cuda.is_available(): torch.cuda.reset_peak_memory_stats()
        except Exception: pass
        gc.collect()
        bs = batch_size
        for attempt in range(5):
            print(f'Attempt {attempt+1}: trying batch_size={bs}', flush=True)
            try:
                m, vpreds, vtargs = train_one_fold_s101_ema(fold, folds_df, epochs=epochs, lr=2e-4, wd=1e-5, batch_size=bs, patience=3, ema_decay=0.9996)
                break
            except (RuntimeError, torch.cuda.OutOfMemoryError, torch.OutOfMemoryError) as e:
                if 'out of memory' in str(e).lower():
                    print(f'CUDA OOM on fold {fold} with batch_size={bs}. Reducing...', flush=True)
                    try: del m
                    except Exception: pass
                    gc.collect()
                    try: torch.cuda.empty_cache(); torch.cuda.reset_peak_memory_stats()
                    except Exception: pass
                    bs = max(2, bs-2); continue
                else:
                    raise
        models_out.append(m)
        va_idx = np.where(folds_df['fold'].values == fold)[0]
        oof_preds[va_idx] = vpreds
        fold_q = cohen_kappa_score(vtargs, np.digitize(vpreds, bins=[0.5,1.5,2.5,3.5]), weights='quadratic')
        print(f'[SEResNeXt101@512] Fold {fold} val QWK (default th): {fold_q:.4f}', flush=True)
    np.save('oof_preds_serx101_512_rrc_ema.npy', oof_preds); np.save('oof_targets_serx101_512_rrc_ema.npy', oof_targs)
    return models_out, oof_preds, oof_targs

def hflip_tta_predict_mean(models, dl):
    preds = []
    with torch.no_grad():
        for xb, _ in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                s = 0; c = 0
                for m in models:
                    m.eval(); p0 = m(xb); p1 = m(torch.flip(xb, dims=[-1]))
                    s += (p0 + p1) / 2.0; c += 1
                p = s / max(1,c)
            preds.append(p.detach().float().cpu().numpy())
    return np.concatenate(preds)

def optimize_thresholds_generic(y, p, init=[0.5,1.5,2.5,3.5]):
    y = np.asarray(y).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, 0.3, 3.7)
        cls = np.digitize(p, bins=[th[0], th[1], th[2], th[3]])
        return -cohen_kappa_score(y, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i]-th[i-1] < 0.12: th[i] = th[i-1] + 0.12
    th = np.clip(th, 0.3, 3.7)
    return th

def bootstrap_thresholds_generic(y, p, n_bootstrap=200, base=[0.5,1.5,2.5,3.5]):
    N = len(y); cuts = []
    for i in range(n_bootstrap):
        idx = np.random.randint(0, N, size=N)
        th = optimize_thresholds_generic(y[idx], p[idx], init=base)
        cuts.append(th)
        if (i+1)%40==0: print(f'Bootstrap {i+1}/{n_bootstrap}', flush=True)
    cuts = np.array(cuts); med = np.median(cuts, axis=0); med = np.clip(med, 0.3, 3.7)
    for j in range(1,4):
        if med[j]-med[j-1] < 0.12: med[j] = med[j-1] + 0.12
    base_arr = np.array([0.5,1.5,2.5,3.5])
    return 0.9*med + 0.1*base_arr

def fit_nnls_masked(y, X, mask, lo=0.05, hi=0.70, serx_pair=('serx50_512_rrcema','serx50_512_rrcema_s2'), serx_cap_sum=0.30, keys=None):
    w, _ = nnls(X[mask], y[mask].astype(float))
    if w.sum() > 0: w = w / w.sum()
    w = np.clip(w, lo, hi); w = w / w.sum()
    if keys is not None and all(k in keys for k in serx_pair):
        i0 = keys.index(serx_pair[0]); i1 = keys.index(serx_pair[1])
        s = w[i0] + w[i1]
        if s > serx_cap_sum:
            scale = serx_cap_sum / s
            w[i0] *= scale; w[i1] *= scale
            rem = 1.0 - (w[i0] + w[i1])
            idx_others = [i for i in range(len(w)) if i not in (i0,i1)]
            s_others = w[idx_others].sum() + 1e-9
            w[idx_others] = w[idx_others] / s_others * rem
    return w

# 1) Train SEResNeXt101 (3 folds)
seed_everything(2028)
models_s101, oof_s101, oof_t_s101 = run_cv_s101_ema(folds_df, folds_to_run=(0,1,2), epochs=12, batch_size=8)

# 2) Test inference (hflip-only TTA) and save
dte_s101 = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms_s101)
dl_te_s101 = DataLoader(dte_s101, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
preds_s101 = hflip_tta_predict_mean(models_s101, dl_te_s101).reshape(-1)
np.save('test_reg_preds_serx101_512_rrc_ema.npy', preds_s101)

# 3) LOFO-averaged NNLS with caps, include serx101
paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'oof_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'oof_preds_convnextb_512_rrc_ema.npy',
    'serx101_512_rrcema': 'oof_preds_serx101_512_rrc_ema.npy',
}
paths_te = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'test_reg_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'test_reg_preds_convnextb_512_rrc_ema.npy',
    'serx101_512_rrcema': 'test_reg_preds_serx101_512_rrc_ema.npy',
}
y = None
for tgt in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt):
        y = np.load(tgt).reshape(-1); break
if y is None: y = oof_t_s101.reshape(-1)

loaded_keys = []; arrs = [];
for k,p in paths_oof.items():
    if os.path.exists(p):
        a = np.load(p).reshape(-1); loaded_keys.append(k); arrs.append(a)
X_full = np.concatenate([a.reshape(-1,1) for a in arrs], axis=1)
mask = np.isfinite(y).copy()
for a in arrs: mask &= np.isfinite(a)
keys_all = loaded_keys.copy()
weights_list = []
w_all = fit_nnls_masked(y, X_full, mask, lo=0.05, hi=0.70, keys=keys_all)
weights_list.append(w_all)
for i in range(len(keys_all)):
    keep_idx = [j for j in range(len(keys_all)) if j != i]
    X_lofo = X_full[:, keep_idx]
    keys_lofo = [keys_all[j] for j in keep_idx]
    w_lofo = fit_nnls_masked(y, X_lofo, mask, lo=0.05, hi=0.70, keys=keys_lofo)
    w_full = np.zeros(len(keys_all), dtype=float)
    ptr = 0
    for j in range(len(keys_all)):
        if j == i: continue
        w_full[j] = w_lofo[ptr]; ptr += 1
    if w_full.sum() > 0: w_full /= w_full.sum()
    weights_list.append(w_full)
w_avg = np.mean(np.stack(weights_list, axis=0), axis=0)
w_avg = np.clip(w_avg, 0.05, 0.70); w_avg = w_avg / w_avg.sum()
blend_oof = (X_full * w_avg.reshape(1,-1)).sum(axis=1)
th = optimize_thresholds_generic(y[mask], blend_oof[mask])
oof_q = cohen_kappa_score(y[mask], np.digitize(blend_oof[mask], bins=[th[0], th[1], th[2], th[3]]), weights='quadratic')
print('LOFO-avg NNLS OOF QWK (with serx101):', f'{oof_q:.5f}', 'weights:', w_avg, 'keys:', keys_all, flush=True)
th_boot = bootstrap_thresholds_generic(y[mask], blend_oof[mask], n_bootstrap=200, base=th.tolist())
np.save('blend_weights_keys.npy', np.array(keys_all, dtype=object)); np.save('blend_weights_vals.npy', w_avg); np.save('thresholds_blend_boot.npy', th_boot)

# 4) Build blended TEST in same key order; hflip-only; EV clip [0,4]
test_list = []; used_keys = []
for k in keys_all:
    p = paths_te.get(k, None)
    if p and os.path.exists(p):
        test_list.append(np.load(p).reshape(-1)); used_keys.append(k)
    else:
        print('Missing test preds for', k, '->', p, flush=True)
if len(test_list) == 0:
    raise RuntimeError('No test preds found for final blend with serx101')
w_use = w_avg[:len(test_list)]
if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
w_use = w_use / w_use.sum()
blend_te = np.zeros_like(test_list[0], dtype=float)
for wi, arr in zip(w_use, test_list):
    blend_te += wi * arr
blend_te = np.clip(blend_te, 0.0, 4.0)
cls = np.digitize(blend_te, bins=[th_boot[0], th_boot[1], th_boot[2], th_boot[3]]).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
print('Final submission.csv written with SEResNeXt101 included.', flush=True)

Starting SEResNeXt101_32x8d@512 (3 folds) RRC+EMA...


Attempt 1: trying batch_size=8



===== [SEResNeXt101@512 RRC+EMA] Fold 0 =====


/app/.pip-target/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 1 it 50/329 tr_loss 0.8250 elapsed 42.0s


...

Bootstrap 80/200


Bootstrap 120/200


Bootstrap 160/200


Bootstrap 200/200


Final submission.csv written with SEResNeXt101 included.


In [32]:
# Recreate best LOFO-avg NNLS blend (exclude serx101) and write final submission
import os, numpy as np, pandas as pd
from scipy.optimize import nnls, minimize
from sklearn.metrics import cohen_kappa_score

print('Rebuilding best LOFO-avg NNLS (exclude serx101)...', flush=True)
keys_fixed = ['b4_512','b5_512','b5_512_rrcema','b4_640','serx50_512_rrcema','serx50_512_rrcema_s2','r200d_512_rrcema','convnextb_512_rrcema']
paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'oof_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'oof_preds_convnextb_512_rrc_ema.npy',
}
paths_te = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'test_reg_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'test_reg_preds_convnextb_512_rrc_ema.npy',
}
y = None
for tgt in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt): y = np.load(tgt).reshape(-1); break
if y is None: raise RuntimeError('OOF targets not found')

loaded_keys = []; arrs = [];
for k in keys_fixed:
    p = paths_oof[k]
    if os.path.exists(p):
        a = np.load(p).reshape(-1); loaded_keys.append(k); arrs.append(a)
    else:
        print('Missing OOF for', k, '->', p, flush=True)
if len(arrs) < 3: raise RuntimeError('Too few OOF arrays for NNLS')

X_full = np.concatenate([a.reshape(-1,1) for a in arrs], axis=1)
mask = np.isfinite(y).copy()
for a in arrs: mask &= np.isfinite(a)

def fit_nnls_masked(y, X, mask, lo=0.05, hi=0.70, serx_pair=('serx50_512_rrcema','serx50_512_rrcema_s2'), serx_cap_sum=0.30, keys=None):
    w, _ = nnls(X[mask], y[mask].astype(float))
    if w.sum() > 0: w = w / w.sum()
    w = np.clip(w, lo, hi); w = w / w.sum()
    if keys is not None and all(k in keys for k in serx_pair):
        i0 = keys.index(serx_pair[0]); i1 = keys.index(serx_pair[1])
        s = w[i0] + w[i1]
        if s > serx_cap_sum:
            scale = serx_cap_sum / s
            w[i0] *= scale; w[i1] *= scale
            rem = 1.0 - (w[i0] + w[i1])
            idx_others = [i for i in range(len(w)) if i not in (i0,i1)]
            s_others = w[idx_others].sum() + 1e-9
            w[idx_others] = w[idx_others] / s_others * rem
    return w

# LOFO-averaged NNLS
keys_all = loaded_keys.copy()
weights_list = []
w_all = fit_nnls_masked(y, X_full, mask, lo=0.05, hi=0.70, keys=keys_all)
weights_list.append(w_all)
for i in range(len(keys_all)):
    keep_idx = [j for j in range(len(keys_all)) if j != i]
    X_lofo = X_full[:, keep_idx]
    keys_lofo = [keys_all[j] for j in keep_idx]
    w_lofo = fit_nnls_masked(y, X_lofo, mask, lo=0.05, hi=0.70, keys=keys_lofo)
    w_full = np.zeros(len(keys_all), dtype=float)
    ptr = 0
    for j in range(len(keys_all)):
        if j == i: continue
        w_full[j] = w_lofo[ptr]; ptr += 1
    if w_full.sum() > 0: w_full /= w_full.sum()
    weights_list.append(w_full)
w_avg = np.mean(np.stack(weights_list, axis=0), axis=0)
w_avg = np.clip(w_avg, 0.05, 0.70); w_avg = w_avg / w_avg.sum()

blend_oof = (X_full * w_avg.reshape(1,-1)).sum(axis=1)

def optimize_thresholds_generic(y_true, p, init=[0.5,1.5,2.5,3.5]):
    y_true = np.asarray(y_true).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, 0.3, 3.7)
        cls = np.digitize(p, bins=[th[0], th[1], th[2], th[3]])
        return -cohen_kappa_score(y_true, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i]-th[i-1] < 0.12: th[i] = th[i-1] + 0.12
    th = np.clip(th, 0.3, 3.7)
    return th

th = optimize_thresholds_generic(y[mask], blend_oof[mask])
oof_q = cohen_kappa_score(y[mask], np.digitize(blend_oof[mask], bins=[th[0], th[1], th[2], th[3]]), weights='quadratic')
print('Best-set LOFO-avg NNLS OOF QWK:', f'{oof_q:.5f}', 'weights:', w_avg, 'keys:', keys_all, flush=True)

def bootstrap_thresholds_generic(y_true, p, n_bootstrap=200, base=None):
    if base is None: base = [0.5,1.5,2.5,3.5]
    N = len(y_true); cuts = []
    for i in range(n_bootstrap):
        idx = np.random.randint(0, N, size=N)
        thb = optimize_thresholds_generic(y_true[idx], p[idx], init=base)
        cuts.append(thb)
        if (i+1)%40==0: print(f'Bootstrap {i+1}/{n_bootstrap}', flush=True)
    cuts = np.array(cuts); med = np.median(cuts, axis=0); med = np.clip(med, 0.3, 3.7)
    for j in range(1,4):
        if med[j]-med[j-1] < 0.12: med[j] = med[j-1] + 0.12
    base_arr = np.array([0.5,1.5,2.5,3.5])
    return 0.9*med + 0.1*base_arr

th_boot = bootstrap_thresholds_generic(y[mask], blend_oof[mask], n_bootstrap=200, base=th.tolist())
np.save('blend_weights_keys.npy', np.array(keys_all, dtype=object))
np.save('blend_weights_vals.npy', w_avg)
np.save('thresholds_blend_boot.npy', th_boot)

# Build blended TEST with same key order
test_list = []; used_keys = []
for k in keys_all:
    p = paths_te[k]
    if os.path.exists(p):
        test_list.append(np.load(p).reshape(-1)); used_keys.append(k)
    else:
        print('Missing test preds for', k, '->', p, flush=True)
if len(test_list) == 0: raise RuntimeError('No test preds for final blend')
w_use = w_avg[:len(test_list)]
if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
w_use = w_use / w_use.sum()
blend_te = np.zeros_like(test_list[0], dtype=float)
for wi, arr in zip(w_use, test_list): blend_te += wi * arr
blend_te = np.clip(blend_te, 0.0, 4.0)
cls = np.digitize(blend_te, bins=[th_boot[0], th_boot[1], th_boot[2], th_boot[3]]).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
print('Final submission.csv written with best-set (no serx101).', flush=True)

Rebuilding best LOFO-avg NNLS (exclude serx101)...


Best-set LOFO-avg NNLS OOF QWK: 0.88284 weights: [0.11385803 0.15888844 0.26617223 0.10379826 0.18198154 0.05618721
 0.06956592 0.04954837] keys: ['b4_512', 'b5_512', 'b5_512_rrcema', 'b4_640', 'serx50_512_rrcema', 'serx50_512_rrcema_s2', 'r200d_512_rrcema', 'convnextb_512_rrcema']


Bootstrap 40/200


Bootstrap 80/200


Bootstrap 120/200


Bootstrap 160/200


Bootstrap 200/200


Final submission.csv written with best-set (no serx101).


In [33]:
# Subset selection: try top-K model subsets (by NNLS weight) for LOFO-avg NNLS; pick best OOF and write submission
import os, numpy as np, pandas as pd
from scipy.optimize import nnls, minimize
from sklearn.metrics import cohen_kappa_score

print('Subset LOFO-avg NNLS search over top-K models...', flush=True)
keys_all = np.load('blend_weights_keys.npy', allow_pickle=True).tolist()
w_all = np.load('blend_weights_vals.npy')
paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'oof_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'oof_preds_convnextb_512_rrc_ema.npy',
}
paths_te = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'test_reg_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'test_reg_preds_convnextb_512_rrc_ema.npy',
}

y = None
for tgt in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt):
        y = np.load(tgt).reshape(-1); break
if y is None:
    raise RuntimeError('OOF targets not found')

# Load OOF arrays
oof_map = {}
for k in keys_all:
    p = paths_oof.get(k, None)
    if p and os.path.exists(p):
        oof_map[k] = np.load(p).reshape(-1)

def fit_nnls_masked(y, X, mask, lo=0.05, hi=0.70):
    w, _ = nnls(X[mask], y[mask].astype(float))
    if w.sum() > 0: w = w / w.sum()
    w = np.clip(w, lo, hi); w = w / w.sum()
    return w

def optimize_thresholds_generic(y, p, init=[0.5,1.5,2.5,3.5]):
    y = np.asarray(y).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, 0.3, 3.7)
        cls = np.digitize(p, bins=[th[0], th[1], th[2], th[3]])
        return -cohen_kappa_score(y, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':1500, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i]-th[i-1] < 0.12: th[i] = th[i-1] + 0.12
    th = np.clip(th, 0.3, 3.7)
    return th

def bootstrap_thresholds_generic(y, p, n_bootstrap=160, base=None):
    if base is None: base = [0.5,1.5,2.5,3.5]
    N = len(y); cuts = []
    for i in range(n_bootstrap):
        idx = np.random.randint(0, N, size=N)
        thb = optimize_thresholds_generic(y[idx], p[idx], init=base)
        cuts.append(thb)
        if (i+1)%40==0: print(f'Bootstrap {i+1}/{n_bootstrap}', flush=True)
    cuts = np.array(cuts); med = np.median(cuts, axis=0); med = np.clip(med, 0.3, 3.7)
    for j in range(1,4):
        if med[j]-med[j-1] < 0.12: med[j] = med[j-1] + 0.12
    base_arr = np.array([0.5,1.5,2.5,3.5])
    return 0.9*med + 0.1*base_arr

# Order keys by current weights (desc)
order_idx = np.argsort(w_all)[::-1]
keys_sorted = [keys_all[i] for i in order_idx]
w_sorted = w_all[order_idx]
print('Weight order:', list(zip(keys_sorted, w_sorted)), flush=True)

best = (-1.0, None, None, None, None)  # (qwk, keys_used, weights, th_boot, blend_oof)
for K in [4,5,6,7,8]:
    use = [k for k in keys_sorted[:K] if k in oof_map]
    if len(use) < 3: continue
    # Build X and mask
    arrs = [oof_map[k] for k in use]
    mask = np.isfinite(y).copy()
    for a in arrs: mask &= np.isfinite(a)
    X = np.concatenate([a.reshape(-1,1) for a in arrs], axis=1)
    # Fit NNLS with caps
    w = fit_nnls_masked(y, X, mask, lo=0.05, hi=0.70)
    blend_oof = (X * w.reshape(1,-1)).sum(axis=1)
    th = optimize_thresholds_generic(y[mask], blend_oof[mask])
    oof_q = cohen_kappa_score(y[mask], np.digitize(blend_oof[mask], bins=[th[0], th[1], th[2], th[3]]), weights='quadratic')
    print(f'Top-{K} keys {use} -> OOF QWK {oof_q:.5f}', flush=True)
    if oof_q > best[0]:
        th_boot = bootstrap_thresholds_generic(y[mask], blend_oof[mask], n_bootstrap=160, base=th.tolist())
        best = (oof_q, use, w, th_boot, blend_oof)

best_q, best_keys, best_w, best_th, best_oof = best
print('Best subset OOF QWK:', f'{best_q:.5f}', 'keys:', best_keys, 'weights:', best_w, flush=True)
np.save('blend_weights_keys.npy', np.array(best_keys, dtype=object)); np.save('blend_weights_vals.npy', best_w); np.save('thresholds_blend_boot.npy', best_th)

# Build test blend in best key order
test_list = []; used = []
for k in best_keys:
    p = paths_te.get(k, None)
    if p and os.path.exists(p):
        test_list.append(np.load(p).reshape(-1)); used.append(k)
    else:
        print('Missing test preds for', k, '->', p, flush=True)
if len(test_list) == 0:
    raise RuntimeError('No test preds for best subset')
w_use = best_w[:len(test_list)]
if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
w_use = w_use / w_use.sum()
blend_te = np.zeros_like(test_list[0], dtype=float)
for wi, arr in zip(w_use, test_list): blend_te += wi * arr
blend_te = np.clip(blend_te, 0.0, 4.0)
cls = np.digitize(blend_te, bins=[best_th[0], best_th[1], best_th[2], best_th[3]]).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
print('Final submission.csv written with best subset blend.', flush=True)

Subset LOFO-avg NNLS search over top-K models...


Weight order: [('b5_512_rrcema', 0.2661722288708696), ('serx50_512_rrcema', 0.18198154293406468), ('b5_512', 0.158888441188999), ('b4_512', 0.11385802994503175), ('b4_640', 0.1037982605635069), ('r200d_512_rrcema', 0.06956591954477008), ('serx50_512_rrcema_s2', 0.05618720830717532), ('convnextb_512_rrcema', 0.0495483686455826)]


Top-4 keys ['b5_512_rrcema', 'serx50_512_rrcema', 'b5_512', 'b4_512'] -> OOF QWK 0.88240


Bootstrap 40/160


Bootstrap 80/160


...

Top-6 keys ['b5_512_rrcema', 'serx50_512_rrcema', 'b5_512', 'b4_512', 'b4_640', 'r200d_512_rrcema'] -> OOF QWK 0.88010


Top-7 keys ['b5_512_rrcema', 'serx50_512_rrcema', 'b5_512', 'b4_512', 'b4_640', 'r200d_512_rrcema', 'serx50_512_rrcema_s2'] -> OOF QWK 0.88248


Top-8 keys ['b5_512_rrcema', 'serx50_512_rrcema', 'b5_512', 'b4_512', 'b4_640', 'r200d_512_rrcema', 'serx50_512_rrcema_s2', 'convnextb_512_rrcema'] -> OOF QWK 0.88172


Best subset OOF QWK: 0.88339 keys: ['b5_512_rrcema', 'serx50_512_rrcema', 'b5_512', 'b4_512', 'b4_640'] weights: [0.3262021  0.24090006 0.18800059 0.1268179  0.11807935]


Final submission.csv written with best subset blend.


In [34]:
# Final thresholds refinement: heavier bootstrap (n=320) and optional th3 +0.05 tweak; rebuild submission
import os, numpy as np, pandas as pd
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import minimize

print('Refining thresholds with heavier bootstrap and optional th3 tweak...', flush=True)

# Load best subset keys/weights from previous step
keys = np.load('blend_weights_keys.npy', allow_pickle=True).tolist()
w = np.load('blend_weights_vals.npy')

# Paths
paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'oof_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'oof_preds_convnextb_512_rrc_ema.npy',
}
paths_te = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'test_reg_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'test_reg_preds_convnextb_512_rrc_ema.npy',
}

y = None
for tgt in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt):
        y = np.load(tgt).reshape(-1); break
if y is None: raise RuntimeError('OOF targets not found')

# Build OOF matrix for current best keys
arrs = []
for k in keys:
    p = paths_oof.get(k, None)
    if p is None or not os.path.exists(p):
        raise RuntimeError(f'Missing OOF preds for {k}: {p}')
    arrs.append(np.load(p).reshape(-1))
X = np.concatenate([a.reshape(-1,1) for a in arrs], axis=1)
mask = np.isfinite(y).copy()
for a in arrs: mask &= np.isfinite(a)
w_use = w[:X.shape[1]]
if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
w_use = w_use / w_use.sum()
blend_oof = (X * w_use.reshape(1,-1)).sum(axis=1)

def preds_to_classes(p, th):
    return np.digitize(p, bins=[th[0], th[1], th[2], th[3]])

def optimize_thresholds(y_true, p, init=[0.5,1.5,2.5,3.5]):
    y_true = np.asarray(y_true).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, 0.3, 3.7)
        cls = preds_to_classes(p, th)
        return -cohen_kappa_score(y_true, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2500, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i]-th[i-1] < 0.12: th[i] = th[i-1] + 0.12
    th = np.clip(th, 0.3, 3.7)
    return th

th0 = optimize_thresholds(y[mask], blend_oof[mask])
q0 = cohen_kappa_score(y[mask], preds_to_classes(blend_oof[mask], th0), weights='quadratic')
print('Base th OOF QWK:', f'{q0:.5f}', 'th:', th0, flush=True)

def bootstrap_thresholds(y_true, p, base, n=320):
    N = len(y_true); cuts = []
    for i in range(n):
        idx = np.random.randint(0, N, size=N)
        thb = optimize_thresholds(y_true[idx], p[idx], init=base)
        cuts.append(thb)
        if (i+1)%40==0: print(f'Bootstrap {i+1}/{n}', flush=True)
    cuts = np.array(cuts); med = np.median(cuts, axis=0); med = np.clip(med, 0.3, 3.7)
    for j in range(1,4):
        if med[j]-med[j-1] < 0.12: med[j] = med[j-1] + 0.12
    base_arr = np.array([0.5,1.5,2.5,3.5])
    return 0.9*med + 0.1*base_arr

th_boot = bootstrap_thresholds(y[mask], blend_oof[mask], base=th0, n=320)
q_boot = cohen_kappa_score(y[mask], preds_to_classes(blend_oof[mask], th_boot), weights='quadratic')
print('Bootstrapped th OOF QWK:', f'{q_boot:.5f}', 'th_boot:', th_boot, flush=True)

# Optional final stabilizer: try +0.05 on last threshold (th3)
th_tweak = th_boot.copy()
th_tweak[3] = min(3.7, th_tweak[3] + 0.05)
q_tweak = cohen_kappa_score(y[mask], preds_to_classes(blend_oof[mask], th_tweak), weights='quadratic')
use_th = th_tweak if q_tweak > q_boot else th_boot
print('Chosen OOF QWK:', f"{(q_tweak if q_tweak > q_boot else q_boot):.5f}", 'using th:', use_th, flush=True)

# Build test blend with same keys/weights
test_list = []
for k in keys:
    p = paths_te.get(k, None)
    if p is None or not os.path.exists(p):
        raise RuntimeError(f'Missing test preds for {k}: {p}')
    test_list.append(np.load(p).reshape(-1))
w_te = w_use[:len(test_list)]
if w_te.sum() <= 0: w_te = np.ones_like(w_te)/len(w_te)
w_te = w_te / w_te.sum()
blend_te = np.zeros_like(test_list[0], dtype=float)
for wi, arr in zip(w_te, test_list):
    blend_te += wi * arr
blend_te = np.clip(blend_te, 0.0, 4.0)
cls = np.digitize(blend_te, bins=[use_th[0], use_th[1], use_th[2], use_th[3]]).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
np.save('thresholds_blend_boot.npy', use_th)
print('Final submission.csv written with refined thresholds.', flush=True)

Refining thresholds with heavier bootstrap and optional th3 tweak...


Base th OOF QWK: 0.88339 th: [0.55947401 1.48773105 2.45901277 3.04202706]


Bootstrap 40/320


Bootstrap 80/320


Bootstrap 120/320


...

Bootstrap 280/320


Bootstrap 320/320


Bootstrapped th OOF QWK: 0.88279 th_boot: [0.55416249 1.48895794 2.46311149 3.08782436]


Chosen OOF QWK: 0.88279 using th: [0.55416249 1.48895794 2.46311149 3.08782436]


Final submission.csv written with refined thresholds.


In [ ]:
# Rebuild submission using base optimized thresholds (no bootstrap) for best top-5 subset
import os, numpy as np, pandas as pd
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import minimize

print('Writing submission with base optimized thresholds (no bootstrap)...', flush=True)
keys = np.load('blend_weights_keys.npy', allow_pickle=True).tolist()
w = np.load('blend_weights_vals.npy')

paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
}
paths_te = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
}

y = None
for tgt in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt):
        y = np.load(tgt).reshape(-1); break
if y is None: raise RuntimeError('OOF targets not found')

# OOF blend for base thresholds
arrs = []
for k in keys:
    p = paths_oof.get(k, None)
    if p is None or not os.path.exists(p):
        raise RuntimeError(f'Missing OOF preds for {k}: {p}')
    arrs.append(np.load(p).reshape(-1))
X = np.concatenate([a.reshape(-1,1) for a in arrs], axis=1)
mask = np.isfinite(y).copy()
for a in arrs: mask &= np.isfinite(a)
w_use = w[:X.shape[1]]; w_use = w_use / (w_use.sum() if w_use.sum() > 0 else len(w_use))
blend_oof = (X * w_use.reshape(1,-1)).sum(axis=1)

def preds_to_classes(p, th):
    return np.digitize(p, bins=[th[0], th[1], th[2], th[3]])

def optimize_thresholds(y_true, p, init=[0.5,1.5,2.5,3.5]):
    y_true = np.asarray(y_true).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, 0.3, 3.7)
        cls = preds_to_classes(p, th)
        return -cohen_kappa_score(y_true, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000,'xatol':1e-3,'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i]-th[i-1] < 0.12: th[i] = th[i-1] + 0.12
    th = np.clip(th, 0.3, 3.7)
    return th

th_base = optimize_thresholds(y[mask], blend_oof[mask])
oof_q = cohen_kappa_score(y[mask], preds_to_classes(blend_oof[mask], th_base), weights='quadratic')
print('Base thresholds OOF QWK:', f'{oof_q:.5f}', 'th_base:', th_base, flush=True)

# Build test blend and write submission
test_list = []
for k in keys:
    p = paths_te.get(k, None)
    if p is None or not os.path.exists(p):
        raise RuntimeError(f'Missing test preds for {k}: {p}')
    test_list.append(np.load(p).reshape(-1))
w_te = w_use[:len(test_list)]
w_te = w_te / (w_te.sum() if w_te.sum() > 0 else len(w_te))
blend_te = np.zeros_like(test_list[0], dtype=float)
for wi, arr in zip(w_te, test_list):
    blend_te += wi * arr
blend_te = np.clip(blend_te, 0.0, 4.0)
cls = preds_to_classes(blend_te, th_base).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
np.save('thresholds_blend_boot.npy', th_base)
print('Final submission.csv written with base optimized thresholds.', flush=True)

In [35]:
# Write submission using base-optimized thresholds (no bootstrap) for current best subset keys/weights
import os, numpy as np, pandas as pd
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import minimize

print('Building submission with base thresholds (no bootstrap) for best subset...', flush=True)
keys = np.load('blend_weights_keys.npy', allow_pickle=True).tolist()
w = np.load('blend_weights_vals.npy')

paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
}
paths_te = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
}

y = None
for tgt in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt):
        y = np.load(tgt).reshape(-1); break
if y is None: raise RuntimeError('OOF targets not found')

# Build OOF blend
arrs = []
for k in keys:
    p = paths_oof.get(k, None)
    if p is None or not os.path.exists(p):
        raise RuntimeError(f'Missing OOF preds for {k}: {p}')
    arrs.append(np.load(p).reshape(-1))
X = np.concatenate([a.reshape(-1,1) for a in arrs], axis=1)
mask = np.isfinite(y).copy()
for a in arrs: mask &= np.isfinite(a)
w_use = w[:X.shape[1]]
if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
w_use = w_use / w_use.sum()
blend_oof = (X * w_use.reshape(1,-1)).sum(axis=1)

def preds_to_classes(p, th):
    return np.digitize(p, bins=[th[0], th[1], th[2], th[3]])

def optimize_thresholds(y_true, p, init=[0.5,1.5,2.5,3.5]):
    y_true = np.asarray(y_true).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, 0.3, 3.7)
        cls = preds_to_classes(p, th)
        return -cohen_kappa_score(y_true, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':2000, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i]-th[i-1] < 0.12: th[i] = th[i-1] + 0.12
    th = np.clip(th, 0.3, 3.7)
    return th

th_base = optimize_thresholds(y[mask], blend_oof[mask])
oof_q = cohen_kappa_score(y[mask], preds_to_classes(blend_oof[mask], th_base), weights='quadratic')
print('Base thresholds OOF QWK:', f'{oof_q:.5f}', 'th_base:', th_base, flush=True)

# Build test blend with same keys/weights
test_list = []
for k in keys:
    p = paths_te.get(k, None)
    if p is None or not os.path.exists(p):
        raise RuntimeError(f'Missing test preds for {k}: {p}')
    test_list.append(np.load(p).reshape(-1))
w_te = w_use[:len(test_list)]
if w_te.sum() <= 0: w_te = np.ones_like(w_te)/len(w_te)
w_te = w_te / w_te.sum()
blend_te = np.zeros_like(test_list[0], dtype=float)
for wi, arr in zip(w_te, test_list):
    blend_te += wi * arr
blend_te = np.clip(blend_te, 0.0, 4.0)
cls = preds_to_classes(blend_te, th_base).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
np.save('thresholds_blend_boot.npy', th_base)
print('Final submission.csv written with base thresholds for best subset.', flush=True)

Building submission with base thresholds (no bootstrap) for best subset...


Base thresholds OOF QWK: 0.88339 th_base: [0.55947401 1.48773105 2.45901277 3.04202706]


Final submission.csv written with base thresholds for best subset.


In [36]:
# 1-D grid search on th3 for the current best top-5 subset blend; write submission
import os, numpy as np, pandas as pd
from sklearn.metrics import cohen_kappa_score

print('Starting 1-D grid search on th3 for best subset blend...', flush=True)

# Load best subset keys/weights determined earlier (Cell 23)
keys = np.load('blend_weights_keys.npy', allow_pickle=True).tolist()
w = np.load('blend_weights_vals.npy')

# Paths
paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'oof_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'oof_preds_convnextb_512_rrc_ema.npy',
}
paths_te = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'test_reg_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'test_reg_preds_convnextb_512_rrc_ema.npy',
}

# Targets
y = None
for tgt in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt):
        y = np.load(tgt).reshape(-1)
        break
if y is None:
    raise RuntimeError('OOF targets not found')

# Build OOF EV blend for current best keys
arrs = []
for k in keys:
    p = paths_oof.get(k, None)
    if p is None or not os.path.exists(p):
        raise RuntimeError(f'Missing OOF preds for {k}: {p}')
    arrs.append(np.load(p).reshape(-1))
X = np.concatenate([a.reshape(-1,1) for a in arrs], axis=1)
mask = np.isfinite(y).copy()
for a in arrs:
    mask &= np.isfinite(a)
w_use = w[:X.shape[1]]
if w_use.sum() <= 0:
    w_use = np.ones_like(w_use)/len(w_use)
w_use = w_use / w_use.sum()
blend_oof = (X * w_use.reshape(1,-1)).sum(axis=1)

def preds_to_classes(p, th):
    return np.digitize(p, bins=[th[0], th[1], th[2], th[3]])

from scipy.optimize import minimize
def optimize_th012(y_true, p, init=[0.5,1.5,2.5]):
    y_true = np.asarray(y_true).astype(float); p = np.asarray(p).astype(float)
    def _loss(th3):
        th0, th1, th2 = th3[0], th3[1], th3[2]
        th = np.array([th0, th1, th2])
        th = np.sort(th)
        th = np.clip(th, 0.3, 3.7)
        # enforce minimum gaps
        th[1] = max(th[1], th[0] + 0.12)
        th[2] = max(th[2], th[1] + 0.12)
        cls = np.digitize(p, bins=[th[0], th[1], th[2], 3.7])  # temporary th3 high
        return -cohen_kappa_score(y_true, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':1200, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    th = np.clip(th, 0.3, 3.7)
    th[1] = max(th[1], th[0] + 0.12)
    th[2] = max(th[2], th[1] + 0.12)
    return th

# First get good th0, th1, th2 (fix th3 large during this optimization)
th012 = optimize_th012(y[mask], blend_oof[mask], init=[0.55, 1.49, 2.46])
th2_min = th012[2] + 0.12

# Now 1-D grid over th3
grid = np.arange(th2_min, 3.701, 0.005)
best_q = -1.0
best_th3 = None
for i, th3 in enumerate(grid):
    th = np.array([th012[0], th012[1], th012[2], th3], dtype=float)
    cls = preds_to_classes(blend_oof[mask], th)
    q = cohen_kappa_score(y[mask], cls, weights='quadratic')
    if q > best_q:
        best_q = q; best_th3 = th3
    if (i+1) % 50 == 0:
        print(f'Grid {i+1}/{len(grid)} th3={th3:.3f} best_q={best_q:.5f}', flush=True)

best_th = np.array([th012[0], th012[1], th012[2], best_th3], dtype=float)
print('Best OOF QWK from th3 grid:', f'{best_q:.5f}', 'best_th:', best_th, flush=True)

# Build TEST EV blend using same keys/weights
test_list = []
for k in keys:
    p = paths_te.get(k, None)
    if p is None or not os.path.exists(p):
        raise RuntimeError(f'Missing test preds for {k}: {p}')
    test_list.append(np.load(p).reshape(-1))
w_te = w_use[:len(test_list)]
if w_te.sum() <= 0:
    w_te = np.ones_like(w_te)/len(w_te)
w_te = w_te / w_te.sum()
blend_te = np.zeros_like(test_list[0], dtype=float)
for wi, arr in zip(w_te, test_list):
    blend_te += wi * arr
blend_te = np.clip(blend_te, 0.0, 4.0)
cls = preds_to_classes(blend_te, best_th).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
np.save('thresholds_blend_grid.npy', best_th)
print('submission.csv written with th3-grid-optimized thresholds.', flush=True)

Starting 1-D grid search on th3 for best subset blend...


Grid 50/231 th3=2.791 best_q=0.88141


Grid 100/231 th3=3.041 best_q=0.88463


Grid 150/231 th3=3.291 best_q=0.88463


Grid 200/231 th3=3.541 best_q=0.88463


Best OOF QWK from th3 grid: 0.88463 best_th: [0.5697338  1.49034491 2.42640278 3.04140278]


submission.csv written with th3-grid-optimized thresholds.


In [37]:
# 2-D grid search around th2 and th3 for the best top-5 subset blend; write submission
import os, numpy as np, pandas as pd
from sklearn.metrics import cohen_kappa_score

print('Starting 2-D grid search around th2 & th3 for best subset blend...', flush=True)

# Load keys/weights from best subset
keys = np.load('blend_weights_keys.npy', allow_pickle=True).tolist()
w = np.load('blend_weights_vals.npy')

# Paths
paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'oof_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'oof_preds_convnextb_512_rrc_ema.npy',
}
paths_te = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'test_reg_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'test_reg_preds_convnextb_512_rrc_ema.npy',
}

# Targets
y = None
for tgt in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt):
        y = np.load(tgt).reshape(-1); break
if y is None: raise RuntimeError('OOF targets not found')

# Build OOF EV blend
arrs = []
for k in keys:
    p = paths_oof.get(k, None)
    if p is None or not os.path.exists(p):
        raise RuntimeError(f'Missing OOF preds for {k}: {p}')
    arrs.append(np.load(p).reshape(-1))
X = np.concatenate([a.reshape(-1,1) for a in arrs], axis=1)
mask = np.isfinite(y).copy()
for a in arrs: mask &= np.isfinite(a)
w_use = w[:X.shape[1]]
if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
w_use = w_use / w_use.sum()
blend_oof = (X * w_use.reshape(1,-1)).sum(axis=1)

def preds_to_classes(p, th):
    return np.digitize(p, bins=[th[0], th[1], th[2], th[3]])

from scipy.optimize import minimize
def optimize_th01(y_true, p, init=[0.55, 1.49]):
    y_true = np.asarray(y_true).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        t0, t1 = th[0], th[1]
        t0 = float(np.clip(t0, 0.3, 1.6))
        t1 = max(float(np.clip(t1, 0.8, 2.2)), t0 + 0.12)
        cls = np.digitize(p, bins=[t0, t1, 3.7, 3.7])
        return -cohen_kappa_score(y_true, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':800, 'xatol':1e-3, 'fatol':1e-3})
    t0 = float(np.clip(res.x[0], 0.3, 1.6))
    t1 = float(max(np.clip(res.x[1], 0.8, 2.2), t0 + 0.12))
    return np.array([t0, t1], dtype=float)

# Get good th0, th1 from quick opt
t01 = optimize_th01(y[mask], blend_oof[mask], init=[0.57, 1.49])

# Seed th2/th3 from prior results if available
seed_th_path = 'thresholds_blend_grid.npy' if os.path.exists('thresholds_blend_grid.npy') else ('thresholds_blend_boot.npy' if os.path.exists('thresholds_blend_boot.npy') else None)
if seed_th_path:
    seed_th = np.load(seed_th_path)
    th2_c = float(seed_th[2]); th3_c = float(seed_th[3])
else:
    th2_c = 2.46; th3_c = 3.04

# Define 2D search windows
th2_lo = max(t01[1] + 0.12, th2_c - 0.18)
th2_hi = min(3.50, th2_c + 0.18)
th3_lo_base = th3_c - 0.18
th3_hi = 3.70
grid_step = 0.005
th2_vals = np.arange(th2_lo, th2_hi + 1e-9, grid_step)

best_q = -1.0
best_th = None
cnt = 0
for t2 in th2_vals:
    t3_lo = max(t2 + 0.12, th3_lo_base)
    t3_vals = np.arange(t3_lo, th3_hi + 1e-9, grid_step)
    for t3 in t3_vals:
        th = np.array([t01[0], t01[1], t2, t3], dtype=float)
        cls = preds_to_classes(blend_oof[mask], th)
        q = cohen_kappa_score(y[mask], cls, weights='quadratic')
        cnt += 1
        if q > best_q:
            best_q = q; best_th = th.copy()
    if cnt % 2000 == 0:
        print(f'Checked {cnt} combos; current best_q={best_q:.5f} best_th={best_th}', flush=True)

print('Best OOF QWK from 2D grid:', f'{best_q:.5f}', 'best_th:', best_th, flush=True)
np.save('thresholds_blend_grid2d.npy', best_th)

# Build TEST EV blend using same keys/weights
test_list = []
for k in keys:
    p = paths_te.get(k, None)
    if p is None or not os.path.exists(p):
        raise RuntimeError(f'Missing test preds for {k}: {p}')
    test_list.append(np.load(p).reshape(-1))
w_te = w_use[:len(test_list)]
if w_te.sum() <= 0: w_te = np.ones_like(w_te)/len(w_te)
w_te = w_te / w_te.sum()
blend_te = np.zeros_like(test_list[0], dtype=float)
for wi, arr in zip(w_te, test_list): blend_te += wi * arr
blend_te = np.clip(blend_te, 0.0, 4.0)
cls = preds_to_classes(blend_te, best_th).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
print('submission.csv written with 2D (th2, th3) grid-optimized thresholds.', flush=True)

Starting 2-D grid search around th2 & th3 for best subset blend...


Best OOF QWK from 2D grid: 0.88482 best_th: [0.57       1.4155     2.42640278 3.04140278]


submission.csv written with 2D (th2, th3) grid-optimized thresholds.


In [38]:
# Final small-step 4D threshold refinement (Nelder-Mead from current best) on top-5 blend; write submission
import os, numpy as np, pandas as pd
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import minimize

print('Starting 4D NM refinement from current best thresholds...', flush=True)

keys = np.load('blend_weights_keys.npy', allow_pickle=True).tolist()
w = np.load('blend_weights_vals.npy')

paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'oof_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'oof_preds_convnextb_512_rrc_ema.npy',
}
paths_te = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'test_reg_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'test_reg_preds_convnextb_512_rrc_ema.npy',
}

y = None
for tgt in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt):
        y = np.load(tgt).reshape(-1); break
if y is None: raise RuntimeError('OOF targets not found')

arrs = []
for k in keys:
    p = paths_oof.get(k, None)
    if p is None or not os.path.exists(p):
        raise RuntimeError(f'Missing OOF preds for {k}: {p}')
    arrs.append(np.load(p).reshape(-1))
X = np.concatenate([a.reshape(-1,1) for a in arrs], axis=1)
mask = np.isfinite(y).copy()
for a in arrs: mask &= np.isfinite(a)
w_use = w[:X.shape[1]]
if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
w_use = w_use / w_use.sum()
blend_oof = (X * w_use.reshape(1,-1)).sum(axis=1)

def preds_to_classes(p, th):
    return np.digitize(p, bins=[th[0], th[1], th[2], th[3]])

# Seed thresholds: prefer 2D grid best if available, else previous boot/base
seed = None
for p in ['thresholds_blend_grid2d.npy', 'thresholds_blend_grid.npy', 'thresholds_blend_boot.npy']:
    if os.path.exists(p):
        seed = np.load(p).astype(float); break
if seed is None:
    seed = np.array([0.56, 1.49, 2.46, 3.04], dtype=float)

def enforce_gaps(th):
    th = np.clip(th, 0.3, 3.7)
    th = np.sort(th)
    # enforce min gaps
    if th[1] - th[0] < 0.12: th[1] = th[0] + 0.12
    if th[2] - th[1] < 0.12: th[2] = th[1] + 0.12
    if th[3] - th[2] < 0.12: th[3] = min(3.7, th[2] + 0.12)
    return th

def qwk_loss(th):
    th = enforce_gaps(np.array(th, dtype=float))
    cls = preds_to_classes(blend_oof[mask], th)
    return -cohen_kappa_score(y[mask], cls, weights='quadratic')

# Small-step NM starting from seed
res = minimize(qwk_loss, x0=seed, method='Nelder-Mead', options={'maxiter': 600, 'xatol':1e-3, 'fatol':1e-3})
best_th = enforce_gaps(res.x)
best_q = -qwk_loss(best_th)
print('Refined OOF QWK:', f'{best_q:.5f}', 'best_th:', best_th, flush=True)
np.save('thresholds_blend_nm.npy', best_th)

# Apply to TEST blend
test_list = []
for k in keys:
    p = paths_te.get(k, None)
    if p is None or not os.path.exists(p):
        raise RuntimeError(f'Missing test preds for {k}: {p}')
    test_list.append(np.load(p).reshape(-1))
w_te = w_use[:len(test_list)]
if w_te.sum() <= 0: w_te = np.ones_like(w_te)/len(w_te)
w_te = w_te / w_te.sum()
blend_te = np.zeros_like(test_list[0], dtype=float)
for wi, arr in zip(w_te, test_list):
    blend_te += wi * arr
blend_te = np.clip(blend_te, 0.0, 4.0)
cls = preds_to_classes(blend_te, best_th).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
print('submission.csv written with 4D-NM-refined thresholds.', flush=True)

Starting 4D NM refinement from current best thresholds...


Refined OOF QWK: 0.88482 best_th: [0.57       1.4155     2.42640278 3.04140278]


submission.csv written with 4D-NM-refined thresholds.


In [44]:
# Isotonic calibration on blended EV, re-opt thresholds, micro-scan th3, write submission
import os, numpy as np, pandas as pd
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import minimize

print('Starting isotonic calibration + threshold re-opt...', flush=True)

# Load best subset keys/weights
keys = np.load('blend_weights_keys.npy', allow_pickle=True).tolist()
w = np.load('blend_weights_vals.npy')

# Paths
paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'oof_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'oof_preds_convnextb_512_rrc_ema.npy',
}
paths_te = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'test_reg_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'test_reg_preds_convnextb_512_rrc_ema.npy',
}

# Targets
y = None
for tgt in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt):
        y = np.load(tgt).reshape(-1); break
if y is None: raise RuntimeError('OOF targets not found')

# Build OOF/Test EV blend for current keys
arrs_oof = []; arrs_te = []
for k in keys:
    po = paths_oof.get(k, None); pt = paths_te.get(k, None)
    if po is None or not os.path.exists(po): raise RuntimeError(f'Missing OOF for {k}: {po}')
    if pt is None or not os.path.exists(pt): raise RuntimeError(f'Missing test for {k}: {pt}')
    arrs_oof.append(np.load(po).reshape(-1))
    arrs_te.append(np.load(pt).reshape(-1))
X = np.concatenate([a.reshape(-1,1) for a in arrs_oof], axis=1)
mask = np.isfinite(y).copy()
for a in arrs_oof: mask &= np.isfinite(a)
w_use = w[:X.shape[1]]
if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
w_use = w_use / w_use.sum()
blend_oof = (X * w_use.reshape(1,-1)).sum(axis=1)
blend_te = np.zeros_like(arrs_te[0], dtype=float)
for wi, arr in zip(w_use, arrs_te): blend_te += wi * arr

# Fit isotonic on OOF EV -> calibrated EV (handle NaNs by fitting only on finite mask, transforming only mask)
ir = IsotonicRegression(increasing=True, out_of_bounds='clip')
mask_fit = mask & np.isfinite(blend_oof)
if mask_fit.sum() == 0:
    raise RuntimeError('No finite OOF EV available for isotonic fit')
ir.fit(blend_oof[mask_fit], y[mask_fit].astype(float))
# Transform only on finite indices to avoid NaN errors
cal_oof = blend_oof.copy()
cal_oof[mask_fit] = ir.transform(blend_oof[mask_fit])
# Any remaining non-finite, leave as original EV
cal_oof = np.where(np.isfinite(cal_oof), cal_oof, blend_oof)
# Test EV: ensure finite then transform
if not np.isfinite(blend_te).all():
    med = float(np.nanmedian(blend_te[np.isfinite(blend_te)]))
    blend_te = np.where(np.isfinite(blend_te), blend_te, med)
cal_te = ir.transform(blend_te)
cal_oof = np.clip(cal_oof, 0.0, 4.0)
cal_te = np.clip(cal_te, 0.0, 4.0)

def preds_to_classes(p, th):
    return np.digitize(p, bins=[th[0], th[1], th[2], th[3]])

def optimize_thresholds_strict(y_true, p, init=[0.5,1.5,2.5,3.5]):
    y_true = np.asarray(y_true).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, 0.3, 3.7)
        for i in range(1,4):
            if th[i] - th[i-1] < 0.12: th[i] = th[i-1] + 0.12
        cls = preds_to_classes(p, th)
        return -cohen_kappa_score(y_true, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':1500, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x);
    for i in range(1,4):
        if th[i] - th[i-1] < 0.12: th[i] = th[i-1] + 0.12
    th = np.clip(th, 0.3, 3.7)
    return th

# Optimize thresholds on calibrated OOF
th0 = optimize_thresholds_strict(y[mask_fit], cal_oof[mask_fit], init=[0.57, 1.42, 2.43, 3.04])
q0 = cohen_kappa_score(y[mask_fit], preds_to_classes(cal_oof[mask_fit], th0), weights='quadratic')
print('Iso-cal OOF QWK (NM):', f'{q0:.5f}', 'th0:', th0, flush=True)

# Optional micro-scan on th3 in [th3-0.05, th3+0.05] with step 0.0025, min gap respected
t2 = th0[2]
th3_min = max(t2 + 0.12, th0[3] - 0.05)
th3_max = min(3.7, th0[3] + 0.05)
grid = np.arange(th3_min, th3_max + 1e-9, 0.0025)
best_q = q0; best_th = th0.copy()
for i, th3 in enumerate(grid):
    th = th0.copy(); th[3] = th3
    if th[3] - th[2] < 0.12: th[3] = th[2] + 0.12
    q = cohen_kappa_score(y[mask_fit], preds_to_classes(cal_oof[mask_fit], th), weights='quadratic')
    if q > best_q:
        best_q = q; best_th = th.copy()
    if (i+1) % 40 == 0:
        print(f'th3 scan {i+1}/{len(grid)} best_q={best_q:.5f}', flush=True)
print('Iso-cal best OOF QWK:', f'{best_q:.5f}', 'best_th:', best_th, flush=True)

# Apply to calibrated Test EV and write submission
cls = preds_to_classes(cal_te, best_th).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
np.save('thresholds_blend_isotonic.npy', best_th)
print('submission.csv written with isotonic-calibrated EV and optimized thresholds.', flush=True)

Starting isotonic calibration + threshold re-opt...


Iso-cal OOF QWK (NM): 0.88482 th0: [0.57  1.42  2.43  3.192]


th3 scan 40/41 best_q=0.88482


Iso-cal best OOF QWK: 0.88482 best_th: [0.57  1.42  2.43  3.192]


submission.csv written with isotonic-calibrated EV and optimized thresholds.


In [41]:
# Affine EV scaling grid (p' = a*p + b) + threshold re-opt; write submission
import os, numpy as np, pandas as pd
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import minimize

print('Starting affine EV scaling grid search...', flush=True)

# Load best subset keys/weights
keys = np.load('blend_weights_keys.npy', allow_pickle=True).tolist()
w = np.load('blend_weights_vals.npy')

paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'oof_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'oof_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'oof_preds_convnextb_512_rrc_ema.npy',
}
paths_te = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
    'serx50_512_rrcema_s2': 'test_reg_preds_serx50_512_rrc_ema_seed2026.npy',
    'r200d_512_rrcema': 'test_reg_preds_r200d_512_rrc_ema.npy',
    'convnextb_512_rrcema': 'test_reg_preds_convnextb_512_rrc_ema.npy',
}

# Targets
y = None
for tgt in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt):
        y = np.load(tgt).reshape(-1); break
if y is None: raise RuntimeError('OOF targets not found')

# Build OOF/Test EV blend for current keys
arrs_oof = []; arrs_te = []
for k in keys:
    po = paths_oof.get(k, None); pt = paths_te.get(k, None)
    if po is None or not os.path.exists(po): raise RuntimeError(f'Missing OOF for {k}: {po}')
    if pt is None or not os.path.exists(pt): raise RuntimeError(f'Missing test for {k}: {pt}')
    arrs_oof.append(np.load(po).reshape(-1))
    arrs_te.append(np.load(pt).reshape(-1))
X = np.concatenate([a.reshape(-1,1) for a in arrs_oof], axis=1)
mask = np.isfinite(y).copy()
for a in arrs_oof: mask &= np.isfinite(a)
w_use = w[:X.shape[1]]
if w_use.sum() <= 0: w_use = np.ones_like(w_use)/len(w_use)
w_use = w_use / w_use.sum()
blend_oof = (X * w_use.reshape(1,-1)).sum(axis=1)
blend_te = np.zeros_like(arrs_te[0], dtype=float)
for wi, arr in zip(w_use, arrs_te): blend_te += wi * arr

def preds_to_classes(p, th):
    return np.digitize(p, bins=[th[0], th[1], th[2], th[3]])

def optimize_thresholds_strict(y_true, p, init=[0.57,1.42,2.43,3.19]):
    y_true = np.asarray(y_true).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, 0.3, 3.7)
        for i in range(1,4):
            if th[i] - th[i-1] < 0.12: th[i] = th[i-1] + 0.12
        cls = preds_to_classes(p, th)
        return -cohen_kappa_score(y_true, cls, weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':1200, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x)
    for i in range(1,4):
        if th[i] - th[i-1] < 0.12: th[i] = th[i-1] + 0.12
    th = np.clip(th, 0.3, 3.7)
    return th

# Affine grid
a_vals = np.arange(0.97, 1.0301, 0.005)
b_vals = np.arange(-0.05, 0.0501, 0.005)
best = (-1.0, 1.0, 0.0, np.array([0.57,1.42,2.43,3.19], dtype=float))
cnt = 0
for a in a_vals:
    for b in b_vals:
        p = a*blend_oof + b
        p = np.clip(p, 0.0, 4.0)
        th = optimize_thresholds_strict(y[mask], p[mask], init=best[3])
        q = cohen_kappa_score(y[mask], preds_to_classes(p[mask], th), weights='quadratic')
        cnt += 1
        if q > best[0]:
            best = (q, a, b, th)
    # light logging
    if (cnt % 100) < len(b_vals):
        print(f'Checked {cnt} combos; current best QWK={best[0]:.5f} a={best[1]:.3f} b={best[2]:+.3f}', flush=True)

best_q, best_a, best_b, best_th = best
print('Best affine OOF QWK:', f'{best_q:.5f}', 'a,b:', (best_a, best_b), 'th:', best_th, flush=True)

# Apply to TEST and write submission
p_te = np.clip(best_a*blend_te + best_b, 0.0, 4.0)
cls = preds_to_classes(p_te, best_th).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
np.save('thresholds_blend_affine.npy', best_th); np.save('affine_params.npy', np.array([best_a, best_b], dtype=float))
print('submission.csv written with affine-scaled EV and optimized thresholds.', flush=True)

Starting affine EV scaling grid search...


Checked 105 combos; current best QWK=0.88454 a=0.985 b=+0.020


Checked 210 combos; current best QWK=0.88463 a=1.010 b=+0.010


Best affine OOF QWK: 0.88463 a,b: (1.01, 0.009999999999999967) th: [0.57918575 1.5137256  2.46036221 3.08284331]


submission.csv written with affine-scaled EV and optimized thresholds.


In [43]:
# Recompute hvflip (orig+hflip+vflip)/3 test preds for top-5 models and overwrite hflip files; then rerun isotonic cell
import os, numpy as np, pandas as pd, torch
from torch.utils.data import DataLoader

print('Recomputing hvflip TTA test preds for top-5 models...', flush=True)

def hvflip_tta_predict_mean(models, dl):
    preds = []
    with torch.no_grad():
        for xb, _ in dl:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with torch.amp.autocast('cuda', enabled=True, dtype=torch.float16):
                s = 0; c = 0
                for m in models:
                    m.eval()
                    p0 = m(xb)
                    p1 = m(torch.flip(xb, dims=[-1]))  # hflip
                    p2 = m(torch.flip(xb, dims=[-2]))  # vflip
                    s += (p0 + p1 + p2) / 3.0; c += 1
                p = s / max(1, c)
            preds.append(p.detach().float().cpu().numpy())
    return np.concatenate(preds)

# B4 512
try:
    if 'models' in globals() and isinstance(models, list) and len(models) > 0:
        dte_b4 = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms)
        dl_b4 = DataLoader(dte_b4, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
        p_b4 = hvflip_tta_predict_mean(models, dl_b4).reshape(-1)
        np.save('test_reg_preds_b4_hflip.npy', p_b4)
        print('Overwrote test_reg_preds_b4_hflip.npy with hvflip average', flush=True)
    else:
        print('Skip B4 512: models not in memory', flush=True)
except Exception as e:
    print('B4 512 hvflip failed:', e, flush=True)

# B5 512
try:
    if 'models_b5' in globals() and isinstance(models_b5, list) and len(models_b5) > 0:
        dte_b5 = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms)
        dl_b5 = DataLoader(dte_b5, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
        p_b5 = hvflip_tta_predict_mean(models_b5, dl_b5).reshape(-1)
        np.save('test_reg_preds_b5_hflip.npy', p_b5)
        print('Overwrote test_reg_preds_b5_hflip.npy with hvflip average', flush=True)
    else:
        print('Skip B5 512: models_b5 not in memory', flush=True)
except Exception as e:
    print('B5 512 hvflip failed:', e, flush=True)

# B5 512 RRC+EMA
try:
    if 'models_b5_seed2025_ema' in globals() and isinstance(models_b5_seed2025_ema, list) and len(models_b5_seed2025_ema) > 0:
        dte_b5e = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms)
        dl_b5e = DataLoader(dte_b5e, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
        p_b5e = hvflip_tta_predict_mean(models_b5_seed2025_ema, dl_b5e).reshape(-1)
        np.save('test_reg_preds_b5_seed2025_rrc_ema.npy', p_b5e)
        print('Overwrote test_reg_preds_b5_seed2025_rrc_ema.npy with hvflip average', flush=True)
    else:
        print('Skip B5 512 EMA: models_b5_seed2025_ema not in memory', flush=True)
except Exception as e:
    print('B5 512 EMA hvflip failed:', e, flush=True)

# B4 640
try:
    if 'models_b4_640' in globals() and isinstance(models_b4_640, list) and len(models_b4_640) > 0:
        dte_b4640 = LocalDRDataset(test_df[['id_code']].copy(), CACHE640_TEST, IMG_SIZE_640, transforms=valid_tfms_640)
        dl_b4640 = DataLoader(dte_b4640, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
        p_b4640 = hvflip_tta_predict_mean(models_b4_640, dl_b4640).reshape(-1)
        np.save('test_reg_preds_b4_640_rrc_ema.npy', p_b4640)
        print('Overwrote test_reg_preds_b4_640_rrc_ema.npy with hvflip average', flush=True)
    else:
        print('Skip B4 640: models_b4_640 not in memory', flush=True)
except Exception as e:
    print('B4 640 hvflip failed:', e, flush=True)

# SEResNeXt50 512
try:
    if 'models_serx' in globals() and isinstance(models_serx, list) and len(models_serx) > 0:
        dte_serx = DRDataset(test_df[['id_code']].copy(), IMG_DIR_TEST, transforms=valid_tfms_serx)
        dl_serx = DataLoader(dte_serx, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
        p_serx = hvflip_tta_predict_mean(models_serx, dl_serx).reshape(-1)
        np.save('test_reg_preds_serx50_512_rrc_ema.npy', p_serx)
        print('Overwrote test_reg_preds_serx50_512_rrc_ema.npy with hvflip average', flush=True)
    else:
        print('Skip SEResNeXt50: models_serx not in memory', flush=True)
except Exception as e:
    print('SEResNeXt50 hvflip failed:', e, flush=True)

print('hvflip recompute done. Now rerun isotonic cell (cell 30) to write submission.', flush=True)

Recomputing hvflip TTA test preds for top-5 models...


Overwrote test_reg_preds_b4_hflip.npy with hvflip average


In [45]:
# Isotonic + th3 safety nudge (+0.02 if within 0.0005 OOF), write submission
import os, numpy as np, pandas as pd
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import minimize

print('Isotonic + th3 safety nudge...', flush=True)

keys = np.load('blend_weights_keys.npy', allow_pickle=True).tolist()
w = np.load('blend_weights_vals.npy')

paths_oof = {
    'b4_512': 'oof_preds_b4.npy',
    'b5_512': 'oof_preds.npy',
    'b5_512_rrcema': 'oof_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'oof_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'oof_preds_serx50_512_rrc_ema.npy',
}
paths_te = {
    'b4_512': 'test_reg_preds_b4_hflip.npy',
    'b5_512': 'test_reg_preds_b5_hflip.npy',
    'b5_512_rrcema': 'test_reg_preds_b5_seed2025_rrc_ema.npy',
    'b4_640': 'test_reg_preds_b4_640_rrc_ema.npy',
    'serx50_512_rrcema': 'test_reg_preds_serx50_512_rrc_ema.npy',
}

y = None
for tgt in ['oof_targets_b4.npy', 'oof_targets.npy']:
    if os.path.exists(tgt): y = np.load(tgt).reshape(-1); break
if y is None: raise RuntimeError('OOF targets not found')

# Build blends
arrs_oof=[]; arrs_te=[]
for k in keys:
    if k not in paths_oof or k not in paths_te: continue
    po=paths_oof[k]; pt=paths_te[k]
    if not (os.path.exists(po) and os.path.exists(pt)): continue
    arrs_oof.append(np.load(po).reshape(-1)); arrs_te.append(np.load(pt).reshape(-1))
if len(arrs_oof)==0: raise RuntimeError('No arrays for isotonic step')
X = np.concatenate([a.reshape(-1,1) for a in arrs_oof], axis=1)
mask = np.isfinite(y).copy()
for a in arrs_oof: mask &= np.isfinite(a)
w_use = w[:X.shape[1]]; w_use = w_use / (w_use.sum() if w_use.sum()>0 else len(w_use))
blend_oof = (X * w_use.reshape(1,-1)).sum(axis=1)
blend_te = np.zeros_like(arrs_te[0], dtype=float)
for wi, arr in zip(w_use, arrs_te): blend_te += wi * arr

# Isotonic fit on valid OOF only
ir = IsotonicRegression(increasing=True, out_of_bounds='clip')
mask_fit = mask & np.isfinite(blend_oof)
ir.fit(blend_oof[mask_fit], y[mask_fit].astype(float))
cal_oof = blend_oof.copy()
cal_oof[mask_fit] = ir.transform(blend_oof[mask_fit])
cal_oof = np.where(np.isfinite(cal_oof), cal_oof, blend_oof)
if not np.isfinite(blend_te).all():
    med = float(np.nanmedian(blend_te[np.isfinite(blend_te)]))
    blend_te = np.where(np.isfinite(blend_te), blend_te, med)
cal_te = ir.transform(blend_te)
cal_oof = np.clip(cal_oof, 0.0, 4.0); cal_te = np.clip(cal_te, 0.0, 4.0)

def preds_to_classes(p, th):
    return np.digitize(p, bins=[th[0], th[1], th[2], th[3]])

def optimize_thresholds_strict(y_true, p, init=[0.57,1.42,2.43,3.19]):
    y_true = np.asarray(y_true).astype(float); p = np.asarray(p).astype(float)
    def _loss(th):
        th = np.sort(th); th = np.clip(th, 0.3, 3.7)
        for i in range(1,4):
            if th[i]-th[i-1] < 0.12: th[i] = th[i-1] + 0.12
        return -cohen_kappa_score(y_true, preds_to_classes(p, th), weights='quadratic')
    res = minimize(_loss, x0=np.array(init, dtype=float), method='Nelder-Mead', options={'maxiter':1200, 'xatol':1e-3, 'fatol':1e-3})
    th = np.sort(res.x);
    for i in range(1,4):
        if th[i]-th[i-1] < 0.12: th[i] = th[i-1] + 0.12
    th = np.clip(th, 0.3, 3.7)
    return th

# Base NM
th0 = optimize_thresholds_strict(y[mask_fit], cal_oof[mask_fit], init=[0.57,1.42,2.43,3.19])
q0 = cohen_kappa_score(y[mask_fit], preds_to_classes(cal_oof[mask_fit], th0), weights='quadratic')
print('Iso-cal OOF QWK:', f'{q0:.5f}', 'th0:', th0, flush=True)

# Fine th3 scan ±0.05
t2 = th0[2]; th3_min = max(t2+0.12, th0[3]-0.05); th3_max = min(3.7, th0[3]+0.05)
grid = np.arange(th3_min, th3_max+1e-9, 0.0025)
best_q = q0; best_th = th0.copy()
for i, th3 in enumerate(grid):
    th = th0.copy(); th[3] = max(th[2]+0.12, min(3.7, th3))
    q = cohen_kappa_score(y[mask_fit], preds_to_classes(cal_oof[mask_fit], th), weights='quadratic')
    if q > best_q: best_q = q; best_th = th.copy()
print('Best th3 scan OOF:', f'{best_q:.5f}', 'best_th:', best_th, flush=True)

# Safety nudge: +0.02 to th3 if within 0.0005 OOF drop
safe_th = best_th.copy(); safe_th[3] = min(3.7, best_th[3] + 0.02)
q_safe = cohen_kappa_score(y[mask_fit], preds_to_classes(cal_oof[mask_fit], safe_th), weights='quadratic')
chosen_th = safe_th if (best_q - q_safe) <= 0.0005 else best_th
chosen_q = q_safe if (best_q - q_safe) <= 0.0005 else best_q
print('Chosen OOF QWK:', f'{chosen_q:.5f}', 'chosen_th:', chosen_th, flush=True)

# Apply to test
cls = preds_to_classes(cal_te, chosen_th).astype(int)
sub = pd.DataFrame({'id_code': pd.read_csv('test.csv')['id_code'].values, 'diagnosis': cls})
sub.to_csv('submission.csv', index=False)
np.save('thresholds_blend_isotonic_safe.npy', chosen_th)
print('submission.csv written (isotonic + th3 safety nudge).', flush=True)

Isotonic + th3 safety nudge...


Iso-cal OOF QWK: 0.88482 th0: [0.57 1.42 2.43 3.19]


Best th3 scan OOF: 0.88482 best_th: [0.57 1.42 2.43 3.19]


Chosen OOF QWK: 0.88482 chosen_th: [0.57 1.42 2.43 3.21]


submission.csv written (isotonic + th3 safety nudge).
